# 导入模块

In [228]:
import numpy as np
import pandas as pd
import xlwings as xw
from openpyxl import load_workbook
import matplotlib.pyplot as plt
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False

import warnings
warnings.filterwarnings("ignore")

import pymysql
from sqlalchemy import create_engine
import gc
import json
import random
import datetime
from datetime import timedelta ,time, datetime
from dateutil.relativedelta import relativedelta
import re
import os
import glob
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean

In [229]:
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# 实例化All_Model类
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()

# 导入订单数据

In [230]:
sql1 = ''' -- 订单&风控信息  近10日数据   
SELECT date(om.create_time) as create_date,om.create_time,om.id as order_id ,om.order_number,om.all_money 
,om.status, om.user_id, om.overdue, om.alipay_order_id
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
,case when locate('租物',pa.name)>0 or locate('租物',cc.name)>0 or locate('芝麻',pa.name)>0 or locate('芝麻',cc.name)>0  then '芝麻租物' when locate('抖音',pa.name)>0 then '抖音渠道' when locate('搜索',cc.name)>0 then '搜索渠道' else '其他渠道' end as channel_type 
,tod.sku_attributes,tod.product_name,tod.new_actual_money, tod.product_id
,case when  locate('租完即送',tod.sku_attributes)>0 then '租完即送' else '租物归还' end as back_type
,om.user_mobile,tmu.true_name,tmu.id_card_num
,top.total_describes,tor.decision_result,om.cancel_reason
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.traceid') end,'"','') as trace_id 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.rejected') end,'"','') as rejected 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.result') end,'"','') as result 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.tips') end,'"','') as tips  
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.is_vip') end,'"','') as is_vip
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.status') end,'"','') as status_result
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.queue_verify_type') end,'"','') as qvt_risk
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.queue_verify_type') end,'"','') as qvt_result
,cc.name as channel_name         -- 来源渠道
,cc.channel_type_id              -- 渠道id
,pa.name as activity_name        -- 活动名称
,om.merchant_id,om.merchant_name, om.has_actual
,topay.total_freeze_fund_amount , tod.dy_order_item_json, pa.type
,om.buy_service_product
,tso.status as service_status 
,om.order_method, om.activity_id, om.appid, tprm.max_overdue_days
, tor.update_time, tprm.purchase_amount, tprm.liquidated_damages 违约金
, tomt.reason
, om.order_type, om.union_rent_tag, tprm.rent_received 已收租金 , tprm.paid_buyout_balance_payment 已付买断尾款, tprm.relet_days
, tolog.status 物流状态, tojo.app_type

, replace(case when JSON_VALID(risk.data) THEN JSON_EXTRACT(risk.data, '$.rejected_ato_risk') end,'"','') as rejected_ato_risk  # 联合拒量

, replace(case when JSON_VALID(risk.data) THEN JSON_EXTRACT(risk.data, '$.union_rent_rejected') end,'"','') as union_rent_rejected   # 联合运营

, replace(case when JSON_VALID(risk.data) THEN JSON_EXTRACT(risk.data, '$.strategy_241011') end,'"','') as strategy_241011                    # 拒量
, replace(case when JSON_VALID(risk.data) THEN JSON_EXTRACT(risk.data, '$.strategy_241205_rejected') end,'"','') as strategy_241205_rejected  # 拒量
, replace(case when JSON_VALID(risk.data) THEN JSON_EXTRACT(risk.data, '$.strategy_241212_rejected') end,'"','') as strategy_241212_rejected  # 拒量
, replace(case when JSON_VALID(risk.data) THEN JSON_EXTRACT(risk.data, '$.strategy_250120_rejected') end,'"','') as strategy_250120_rejected  # 拒量
, replace(case when JSON_VALID(risk.data) THEN JSON_EXTRACT(risk.data, '$.strategy_250330_rejected') end,'"','') as strategy_250330_rejected  # 拒量

-- , ymos.money, ymos.part_payment, ymos.refund_date, ymos.reality_refund_date, ymos.sort, ymos.status 分期租金支付状态
from  db_digua_business.t_order  om
left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=om.id
left join db_digua_business.t_order_risk tor on om.id = tor.order_id
-- 备注信息合并 
left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top 
on om.id = top.order_id 
-- 服务信息
left join  db_digua_business.t_service_order tso  on om.id = tso.order_id 
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
-- 用户信息 
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
-- 商品信息
left join db_digua_business.t_order_details tod on om.id = tod.order_id
-- 免押信息  
left join (SELECT t.*,row_number() over(partition by t.order_id order by t.pay_date desc) as rn 
from db_digua_business.t_order_pay t 

where t.pay_type = 'ZFBYSQ' and t.item_type=1 and t.`status` in (2,5) and t.trade_no is not null )  topay 
on topay.order_id=om.id   and  topay.rn = 1 
-- 商家订单转移表
left join db_digua_business.t_order_merchant_transfer tomt on tomt.order_id=om.id
-- 京东外部订单关联表
left join db_digua_business.t_order_jd_out_no tojo on tojo.order_id=om.id
-- 订单物流表
left join db_digua_business.t_order_logistics tolog on tolog.order_id=om.id
-- 租后分期表
-- left join db_rent.ya_merchant_order_stages ymos on ymos.order_id=om.id
left join db_credit.risk risk on risk.order_id=om.id


where om.user_mobile is not null 
-- 曙光计划订单会在去重逻辑中剔除，这里不做过滤，否则存在非线下小店订单被剔除的情况
-- and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
-- ,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静','陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静', '李珍珍')
-- and pa.type!=4 号卡活动，已过期，不统计
-- and  om.create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -15 day )               -- 近20天数据
-- and  om.create_time <= DATE_ADD(CURRENT_DATE,INTERVAL -0 day )
and  DATE_FORMAT(om.create_time, '%Y-%m-%d')>='2025-10-01'
and  DATE_FORMAT(om.create_time, '%Y-%m-%d')<='2025-10-23'
-- and TIME(om.create_time) <= '17:00:00'

'''

df_order = clean.query(sql1)
df_order.shape

(282521, 63)

In [231]:
sql_modelname = '''
-- 商品表
select
tp.id, tp.classify_id, tp.id 商品ID, tpmn.name phone_name
from db_digua_business.t_product tp
left join db_digua_business.t_product_model_number tpmn on tpmn.id=tp.model_number_id
'''
df_modelname = clean.query(sql_modelname)
df_order = df_order.merge(df_modelname, left_on='product_id', right_on='id',how='left')

In [232]:
sql_risk = ''' -- risk等级
select trace_id, id_card as id_card_r, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status_r 
-- , replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.union_rent_tag') end,'"','') as union_rent_tag
-- , replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.union_rent_rejected') end,'"','') as union_rent_rejected

from db_credit.risk
'''
df_risk = clean.query(sql_risk)
# df_risk.loc[:, 'time_r'] = pd.to_datetime(df_risk.time.dt.date,errors="coerce")
# df_risk = df_risk.sort_values(by='time', ascending=False).groupby('id_card_r').head(1)

In [233]:
sql3 = '''
SELECT risk_trace_id trace_id, id_card, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status   FROM `db_credit`.risk_examine
'''
df_risk_examine = clean.query(sql3)
df_re = df_risk_examine.copy()
df_re.loc[:, 'time_re'] = pd.to_datetime(df_re.time.dt.date,errors="coerce")
df_re.rename(columns={'id_card': 'id_card_re', 'status': 'status_re'}, inplace=True)
# df_re = df_re.sort_values(by='time', ascending=False).groupby('id_card_re').head(1)

In [234]:
sql_ra = ''' -- 996强拒表
select order_id, time, replace(case when JSON_VALID(result) THEN JSON_EXTRACT(result, '$.status') end,'"','') as status_ra  from db_credit.risk_alipay_interactive_prod_result
'''
df_ra = clean.query(sql_ra)
df_ra.loc[:, 'time_ra'] = pd.to_datetime(df_ra.time.dt.date,errors="coerce")
df_ra = df_ra.sort_values(by='time', ascending=False).groupby('order_id').head(1)

In [235]:
# sql_name = '''
# SELECT tuvor.order_id, tu.nick_name 分配人, tuvor.update_time 
# FROM db_digua_business.t_user_verify_order_record tuvor
# left join db_digua_business.t_user tu on tuvor.user_id = tu.id 
# where tuvor.del_flag = 0 ORDER BY tuvor.update_time
# '''
# df_name = clean.query(sql_name)

In [236]:
# # 首逾
# sql_stages = '''
# select 
# tos.order_id, tos.sort, tos.refund_date, tos.reality_refund_date
# from db_digua_business.t_order_stages tos
# where tos.sort = 2
# '''
# df_stages = clean.query(sql_stages)

In [237]:
sql_wl = '''
select
date_format(om.create_time, '%Y-%m-%d') 下单日期, tort.order_num order_number, tort.task_status
from db_digua_business.t_order_retention_task tort
left join db_digua_business.t_order om on om.id=tort.order_id
'''
df_wl = clean.query(sql_wl)
df_wl.shape

(4194, 3)

In [238]:
sql_ywl = '''
select

from db_digua_business.t_order om
left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=om.id
'''

In [239]:
# df_order[df_order.order_number=='A20250806150830177']

# 提取订单风控数据

# 合并数据

In [240]:
df = df_order.copy()
df = df[df.type!=4]
# df.order_id.notnull().sum()
# df = df[df['product_name'].str.contains(r'iPhone 17|iPhone Air')]

# 处理日期

In [241]:
df["下单日期"]=df["create_time"].dt.date
df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
df["月份"]=df["下单日期"].dt.month
df["下单月份"]=df["create_time"].dt.strftime('%Y-%m')
df['年份'] = df["下单日期"].dt.year
df['hour'] = df['create_time'].dt.hour


In [242]:
# 机审强拒策略
# 提取拒绝理由字段并清理特殊字符
df['拒绝理由'] = df["rejected"].fillna('').str.replace(r'[\[\]"]', '', regex=True)
# df['拒绝理由'] = df["rejected"].str.replace("[","").str.replace("]","").str.replace('"','')

df['拒绝理由_risk']= df['rejected'].fillna('')+df['strategy_241205_rejected'].fillna('')+df['strategy_241212_rejected'].fillna('')+df['strategy_241011'].fillna('')+df['strategy_250120_rejected'].fillna('')+df['strategy_250330_rejected'].fillna('')+df['union_rent_rejected'].fillna('')+df['rejected_ato_risk'].fillna('')

df.loc[:, '拒绝理由'] = np.where(df['拒绝理由'] == '', df['拒绝理由_risk'], df['拒绝理由'])
df['拒绝理由'] = df["拒绝理由"].str.replace("[","").str.replace("]","").str.replace('"','')

In [243]:
df["下单日期"].unique()


<DatetimeArray>
['2025-10-01 00:00:00', '2025-10-02 00:00:00', '2025-10-03 00:00:00',
 '2025-10-04 00:00:00', '2025-10-05 00:00:00', '2025-10-06 00:00:00',
 '2025-10-07 00:00:00', '2025-10-08 00:00:00', '2025-10-09 00:00:00',
 '2025-10-10 00:00:00', '2025-10-11 00:00:00', '2025-10-12 00:00:00',
 '2025-10-13 00:00:00', '2025-10-14 00:00:00', '2025-10-15 00:00:00',
 '2025-10-16 00:00:00', '2025-10-17 00:00:00', '2025-10-18 00:00:00',
 '2025-10-19 00:00:00', '2025-10-20 00:00:00', '2025-10-21 00:00:00',
 '2025-10-22 00:00:00', '2025-10-23 00:00:00']
Length: 23, dtype: datetime64[ns]

# 渠道补充

In [244]:
# qudao_name_df = pd.read_excel('F:/myfile/租后数据/历史订单渠道名称补充.xlsx')
# qudao_name_df.drop(columns=['渠道id','下单时间'],inplace = True)
# qudao_name_df.rename(columns={'订单号': '订单号1'}, inplace=True)
# df =  df.merge(qudao_name_df,left_on = 'order_number',right_on = '订单号1',how = 'left')
# df["channel_name"] = np.where(df.渠道名称.notnull(),df.渠道名称,df["channel_name"])

# 处理备注信息

In [245]:
df = df[df['sku_attributes'].notnull()]
df["取消原因"]=df["total_describes"].str.split("客户申请取消：").str[1].str.split("$").str[0].str.strip().str.split('"').str[0]
# # 修复后的代码：增强字符串类型转换的鲁棒性,total_describes为空时使用下面这个
# df["取消原因"] = df["total_describes"].fillna('').astype(str).apply(
#     lambda x: x.split("客户申请取消：")[1].split("$")[0].strip().split('"')[0] 
#     if "客户申请取消：" in x else ""
# )
df["电审拒绝原因"]=df["total_describes"].str.split("审核不通过：").str[1].str.split("$").str[0].str.strip().str.split('"').str[0]
df["无法联系原因"]=df["total_describes"].str.split("用户无法联系：").str[1].str.split("$").str[0].str.strip()
df.loc[:,"商品类型"]=np.where(df["product_name"].str.contains('99新')|df["product_name"].str.contains('95新')|df["product_name"].str.contains('准新'),"二手","全新")  ##  准新
df.loc[:,"租赁方案"]=np.where(df["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")
df.loc[:,"押金类型"]=np.where(df["total_freeze_fund_amount"]>0,"部分免押","全免押")
df.loc[:,"优惠券使用否"]=np.where((df["new_actual_money"]-df["all_money"]>0)&(df["租赁方案"]=='租完即送'),"已使用","未使用")
df.loc[:,"是否进行预授权"]=np.where(df["total_freeze_fund_amount"].isnull(),"未预授权","已预授权")
df.loc[:,"颜色"]=df.apply(lambda x:clean.getcolor(x["sku_attributes"]),axis=1)
df.loc[:,"内存"]=df.apply(lambda x: clean.getneicun(x["sku_attributes"]),axis=1)
df.loc[:,"进件"]=np.where((df["status2"]=="待支付")|(df["status2"]=="订单取消"),"未进件","进件")
df.loc[:,"来源渠道"]=df["channel_name"].fillna("未知渠道")
df.loc[:, '商品ID'] = df.商品ID.astype(str)+'_'+df.phone_name
# 物流状态=5，退货拒签
df.loc[:,"退货拒签"] = np.where(df['物流状态']==5, 1, 0)
df.loc[:,'机型内存'] = df['phone_name'] + df['内存']

# 渠道归属

In [246]:

df.loc[:,"归属渠道"]=df.apply(lambda x: clean.qudao_type(x["来源渠道"],x["activity_name"],x["order_method"], x['channel_type_id'], x['order_type']),axis=1)


In [247]:
# df[df['来源渠道'].isin(['搜索下挂-商品', '搜索-vivo'])][['order_number','来源渠道', '归属渠道', 'channel_type_id', 'order_type']].drop_duplicates()
# df[(df.channel_type_id.isin ([48, 81])) & (df.order_type=='ZFB_ORDER') & (df['来源渠道'] != '灯火联投测试')].来源渠道.value_counts()
# df.归属渠道.value_counts()

# 订单去重

In [248]:
# 保留不去重的数据
df_no_drop = df.copy()
# 订单去重
# to_do 京东创建订单数不去重
df = clean.order_drop_duplicates(df)
df.shape


(138354, 90)

# 定义状态

In [249]:
# note 2025-02-24添加
df = df.merge(df_risk[['trace_id', 'status_r']], on='trace_id', how='left').merge(df_re[['trace_id', 'status_re']], on='trace_id', how='left').merge(df_ra[['order_id', 'time_ra', 'status_ra']], left_on=['order_id', '下单日期'], right_on=['order_id', 'time_ra'], how='left')
# , 'union_rent_tag', 'union_rent_rejected'
# 判断 前置拦截   机审强拒   出库前风控强拒
df['是否前置拦截'] = np.where((df.result.str.contains('id_card不得为空') )|(df.result.str.contains('mobile校验不通过')) |(df.result.str.contains('name校验不通过'))
                        |(df.result.str.contains('年龄超过49岁或低于18岁'))|(df.result=='风控未通过')|(df.result=='黑名单用户'),1,0)
# df['是否机审强拒'] = np.where((df.result.str.contains('风控拒绝') & (~df.result.str.contains('命中出库前风控流强拒').fillna(False)) ),1,0)
# df['是否出库前风控强拒'] = np.where((df.result.str.contains('命中出库前风控流强拒').fillna(False)) |((df.total_describes.str.contains('蚂蚁数控风险等级').fillna(False)) & (~df.result.str.contains('黑名单用户').fillna(False))) ,1,0)
# note 2025-02-24修改
df['是否机审强拒'] = np.where(df.status_r=='1', 1, 0)
df['是否出库前风控强拒'] = np.where((df.status_re=='1')|(df.status_ra=='1'), 1, 0)

In [250]:

df.loc[:,"审核状态"]=df.apply(lambda x: clean.reject_type(x["拒绝理由"],x["进件"],x["电审拒绝原因"],x["取消原因"],x["status2"],x["无法联系原因"],x["total_describes"],x['是否前置拦截'],x['是否机审强拒'],x['是否出库前风控强拒']),axis=1)


# 保留商家的数据

In [251]:
df_contain = df.copy()
df_contain.shape

(138354, 98)

# 剔除商家数据

In [252]:
# 调用Data_Clean类中的drop_merchant函数
df = clean.drop_merchant(df)
df.shape

(131388, 98)

# 去重订单明细导出

In [253]:
from datetime import datetime  
today = datetime.today() 
t_date = today.date().strftime('%Y-%m-%d')
t_date

'2025-10-24'

# 导入服务订单

In [254]:
df["下单时段"]=df["create_time"].astype(str).str[:14]

df_j = df[df["进件"]=="进件"]
df_j.shape

(38324, 99)

In [255]:
df = clean.status_node(df)

# 导入台账数据

In [256]:
#读取导入出库台账数据
# f_path_ck = "F:/myfile/p站数据/台账数据/维客壹佰2023&2024年台账.xlsx"
# df_ck = pd.read_excel(f_path_ck,sheet_name="2023")  

f_path_ck = "F:/myfile/p站数据/台账数据/维客壹佰2025年台账.xlsx"
df_ck = pd.read_excel(f_path_ck,sheet_name="2025") 
df_ck.shape

(39979, 18)

In [257]:
dfck = pd.merge(df_ck,df,left_on="订单号", right_on="order_number")
dfck.drop_duplicates(subset=["订单号"],inplace=True)
dfck.shape

(3181, 132)

In [258]:
dfck.drop(dfck[dfck["status2"]=="已退款"].index,inplace=True)
# 删除 露营设备 出库 
try:
    dfck.drop(dfck[dfck["类目"]=="露营设备"].index,inplace=True)
except:
    dfck.drop(dfck[dfck["类型"]=="露营设备"].index,inplace=True)
# 将序列号转换为日期
base_date = pd.to_datetime('1899-12-30')  # Excel的基准日期
dfck['日期'] = base_date + pd.to_timedelta(dfck['日期'], unit='D')


In [259]:
dfck.日期.value_counts().sort_index(ascending=False)

日期
2025-10-23    151
2025-10-22    155
2025-10-21    157
2025-10-20    158
2025-10-19    128
2025-10-18    141
2025-10-17    134
2025-10-16    142
2025-10-15    137
2025-10-14    151
2025-10-13    160
2025-10-12    117
2025-10-11    154
2025-10-10    175
2025-10-09    170
2025-10-08    157
2025-10-07    112
2025-10-06     96
2025-10-05    118
2025-10-04    115
2025-10-03    111
2025-10-02    100
2025-10-01     80
Name: count, dtype: int64

# 每日各渠道出库单数

In [260]:
# 按转化日期看每日各渠道出库单数
df_weekday_zh = pd.crosstab(dfck["日期"],dfck["归属渠道"],margins=True)
# 按下单日期看每日各渠道出库单数
df_weekday = pd.crosstab(dfck["下单日期"],dfck["归属渠道"],margins=True)
# 按下单日期看每日各渠道出库单数（纯租物）
df_ly_ck = pd.crosstab(dfck["下单日期"],dfck["来源渠道"],margins=True)
# 按下单日期看每日支付宝直播商品出库单数
df_zfb_ck = pd.crosstab(dfck["下单日期"],dfck["activity_name"],margins=True)
# 按下单日期看 免押 出库单数
df_my = pd.crosstab(dfck["下单日期"],dfck["押金类型"],margins=True)

In [261]:
# dfck['归属渠道'].unique()

# 剔除拒量订单

In [262]:
df2 = df.copy()
df2 = clean.drop_rejected_merchant(df2)
df2 = df2[df2.是否拒量==0]
day_name_cn_mapping = {
    'Monday': '星期一',
    'Tuesday': '星期二',
    'Wednesday': '星期三',
    'Thursday': '星期四',
    'Friday': '星期五',
    'Saturday': '星期六',
    'Sunday': '星期日'
}

# 临时需求

## 获取前5的spu机型、前10的sku机型内存转化

In [263]:
# df_spu = df.groupby('phone_name').agg({'是否出库':'sum'}).rename(columns={'是否出库':'出库'})

# df_spu.loc[:, '出库占比'] = df_spu['出库'] / df_spu['出库'].sum()
# df_spu.loc[:, '出库占比'] = df_spu['出库占比'].apply(lambda x:format(x,".2%"))
# # 出库数降序排序
# df_spu = df_spu.sort_values(by='出库', ascending=False)

# df_spu_new = df_spu.reset_index()
# df_spu_new[:10]

In [264]:
# df_sku = df.groupby('机型内存').agg({'是否出库':'sum'}).rename(columns={'是否出库':'出库'})
# df_sku.loc[:, '出库占比'] = df_sku['出库'] / df_sku['出库'].sum()
# df_sku.loc[:, '出库占比'] = df_sku['出库占比'].apply(lambda x:format(x,".2%"))
# # 出库数降序排序
# df_sku = df_sku.sort_values(by='出库', ascending=False)
# df_sku_new = df_sku.reset_index()
# df_sku_new[:15]

In [265]:
# with pd.ExcelWriter(f'F:/需求/七月需求/2025年10月01spu、sku出库占比.xlsx', engine='xlsxwriter') as writer:
#     df_spu_new[:10].to_excel(writer, sheet_name='spu')
#     df_sku_new[:15].to_excel(writer, sheet_name='sku')

## 迪瓜租手机芝麻租赁频道异常交易订单匹配

In [266]:
# df_12 =  df[df.order_number == 'A20250807124004660'][['order_number','拒绝理由','电审拒绝原因','审核状态','tips','是否前置拦截','是否机审强拒','是否出库前风控强拒']]
# df_12
# df.下单日期.unique()

In [267]:
# df_ali_order = pd.read_excel('F:/需求/瑞彤需求/迪瓜租手机芝麻租赁频道异常交易核对.xlsx')
# df_ali_orderid_list = df_ali_order['order_id'].astype(str).tolist()
# df_ali_new = df[df['alipay_order_id'].isin(df_ali_orderid_list)]

# df_ali_merge = df_ali_order.merge(df_ali_new[['order_number','alipay_order_id','下单日期', '来源渠道', '归属渠道', 'status2', 'all_money']], left_on='order_id', right_on='alipay_order_id', how='left')
# df_ali_merge.rename(columns={'order_number': '订单号', 'status2': '订单状态', 'all_money': '总租金', 'alipay_order_id': '支付宝订单号'}, inplace=True)
# df_ali_merge.to_excel('F:/需求/瑞彤需求/迪瓜租手机芝麻租赁频道异常交易核对结果.xlsx', index=False)




A20250301093707114 正常被拒订单  拒绝理由:命中自有模型强拒, 命中融安分低于680强拒, 评分等级低于D,命中拒绝  审核状态：机审强拒  是否机审强拒：1  是否出库前风控强拒: 0

A20250821195141539 拒量订单（回捞241205） 拒绝理由:null  审核状态：人审拒绝  是否机审强拒：0, 是否出库前风控强拒: 0   日报会回加到机审强拒中

A20250916223149153 联合拒量订单 拒绝理由:null  审核状态：人审拒绝  是否机审强拒：0  是否出库前风控强拒：0

A20250807124004660 联合运营被拒订单(支付宝联合运营订单) 拒绝理由:null  审核状态：人审拒绝  是否机审强拒：0  是否出库前风控强拒：0  [命中银联模型及Fico联合规则强拒(青云分豁免), 已有在租, 支付宝联合运营订单, 用户下单次数:2]   日报不会回加到机审强拒中


## 统计联合拒量出库订单明细；

In [268]:
# df.drop_duplicates(subset=["id_card_num", "下单日期"], keep="last", inplace=True)# 75
# df[(df.tips.str.contains(r'联合拒量', regex=True, na=False)==True)&(df['是否出库'] == 1)][['order_number', '下单日期', '是否出库']]

In [269]:
# def year_sum(df, year):
#     df2 = df[df.年份==year]176
#     df2_ck = df2[df2.是否出库==1]
#     df2_group = df2_ck.groupby('下单月份').agg({'是否出库': 'sum', 'purchase_amount': 'sum', 'new_actual_money': 'sum'}).rename(columns={'是否出库': '出库数', 'purchase_amount': '采购金额', 'new_actual_money': '签约价'})
#     return df2_group
# df_group_2023 = year_sum(df, 2023)
# df_group_2024 = year_sum(df, 2024)
# df_group_2025 = year_sum(df, 2025)
# df_group_2025
# with pd.ExcelWriter(f'F:/需求/七月需求/23-25年出库数据1.xlsx', engine='xlsxwriter') as writer:
#     df_group_2023.to_excel(writer, sheet_name='2023年出库数据')
#     df_group_2024.to_excel(writer, sheet_name='2024年出库数据')
#     df_group_2025.to_excel(writer, sheet_name='2025年出库数据')

# df_new = df[~df.merchant_name.isin(['小蚂蚁租机', '兴鑫兴通讯', '人人享租', '崇胜数码', '喜卓灵租机'])]
# df_new_g = df_new[df_new.是否出库==1].groupby('下单月份').agg({'是否出库': 'sum', 'new_actual_money': 'sum'}).rename(columns={'new_actual_money': '签约价', '是否出库': '出库'})
# df_g = df.groupby('下单月份').agg({'是否进件': 'sum'}).rename(columns={'是否进件': '进件'})
# df_group = df_g.merge(df_new_g, on='下单月份', how='left').reset_index()
# df_group[['进件', '出库', '签约价']].set_index('进件')

In [270]:
# # 我们业务开始每个月的出库数据进件数据，进件出库率（出库/进件），每个月的采购价purchase_amount汇总，以及签约价new_actual_money汇总吧
# # 未完成订单的签约价-已收new_actual_money-rent_received
# # 剔除商家拒量的订单
# df = df[~df.merchant_name.isin(["小蚂蚁租机","兴鑫兴通讯","人人享租","崇胜数码","喜卓灵租机","喜卓灵新租机"])]
# df_copy = df[['下单月份', 'status', '是否进件' ,'是否出库' ,'purchase_amount' ,'new_actual_money', '已收租金']]
# df_copy
# # 应收未收金额：未完成订单（status in [4,5]）的签约价-已收(new_actual_money-rent_received)
# df_copy.loc[:,'应收未收金额' ] = np.where(df_copy['status'].isin([4,5]), df_copy['new_actual_money']-df_copy['已收租金'], 0)
# # 按下单月份分组选取出库、进件、采购价、签约价、应收未收金额的汇总
# df_res = df_copy.groupby('下单月份').agg({'是否进件': 'sum', '是否出库': 'sum', 'purchase_amount': 'sum', 'new_actual_money': 'sum', '应收未收金额': 'sum'}).rename(columns={'是否进件': '进件', '是否出库': '出库', 'purchase_amount': '采购价','new_actual_money': '签约价'})
# # 计算进件出库率（出库/进件）
# df_res['进件出库率'] = df_res['出库']/df_res['进件']
# # 进件出库率百分比
# df_res['进件出库率'] = df_res['进件出库率'].apply(lambda x: '{:.2%}'.format(x))
# # 写入excel
# # with pd.ExcelWriter(f'F:/需求/倩姐需求/业务量数据.xlsx', engine='openpyxl') as writer:
# #     df_res.to_excel(writer, sheet_name='业务量数据')

## 按小时分组聚合转化

In [93]:
# # 将create_time一天24个小时从0点开始，每2个小时为一组，生成一个新列，表示小时段：0-1，2-3，4-5，...，22-23
# # 1. 确保create_time是datetime类型（若不是，先转换）
# df['create_time'] = pd.to_datetime(df['create_time'])  # 转换为datetime格式
# # 2. 提取小时数（0-23）
# hours = df['create_time'].dt.hour  # 得到每个时间的小时部分
# # 3. 计算时间段（如0-1、2-3...）
# start_hour = (hours // 2) * 2  # 计算每组起始小时（0,2,4,...,22）
# end_hour = start_hour + 1      # 计算每组结束小时（1,3,5,...,23）
# time_periods = start_hour.astype(str) + '-' + end_hour.astype(str)  # 格式化为字符串
# # 4. 将时间段添加到原数据框（核心步骤）
# df['下单时间段'] = time_periods  # 直接赋值，新列会自动加入df

# df2['create_time'] = pd.to_datetime(df2['create_time'])  # 转换为datetime格式
# # 2. 提取小时数（0-23）
# hours = df2['create_time'].dt.hour  # 得到每个时间的小时部分
# # 3. 计算时间段（如0-1、2-3...）
# start_hour = (hours // 2) * 2  # 计算每组起始小时（0,2,4,...,22）
# end_hour = start_hour + 1      # 计算每组结束小时（1,3,5,...,23）
# time_periods = start_hour.astype(str) + '-' + end_hour.astype(str)  # 格式化为字符串
# # 4. 将时间段添加到原数据框（核心步骤）
# df2['下单时间段'] = time_periods  # 直接赋值，新列会自动加入df


# df_hour = all_models.data_group(df, df2, df_risk_examine, '下单时间段')
# df_hour['出库（按下单时间）'] = df_hour['出库']

# df_hour = df_hour[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                     "出库前风控强拒","待审核",'出库（按下单时间）', '拒量出库', '进件出库率','总体进件出库率（含拒量）','拒量进件出库率增加','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
# df_hour_new = df_hour.reset_index()
# df_hour_new#.to_excel(f'F:/需求/瑞彤需求/小时分组转化数据_{t_date}.xlsx')

In [223]:
df[df.归属渠道=='搜索渠道'].来源渠道.unique()

array(['顶部搜索框的搜索结果页', '搜索下挂-商品', '搜索直达banner', '搜索-租手机', '搜索-租苹果手机',
       '搜索邦道-小程序推广', '搜索侠客行-苹果17', '搜索侠客行-核心词', '邦道月活BD23', '公域搜索带券',
       '搜索-租手机苹果', '邦道-销售线索-电脑', '邦道月活BD22', '搜索-租真我手机', '搜索-租二手苹果',
       '搜索-3C数码租赁', '付费流量(通过商家数字推广平台，灯火等投放的广告)', '搜索-租机', '搜索-租手表',
       '搜索-友好物租手机', '租苹果17', '搜索-租笔记本电脑', '搜索-租华为手机', '搜索-租二手手机',
       '搜索侠客行-苹果新词', '邦道-销售线索2电脑', '搜索-租游戏手机', '搜索-租相机'], dtype=object)

# 总体转化

In [94]:
# 总体数据
# df = df[df.归属渠道!='京东渠道']
# df2 = df2[df2.归属渠道!='京东渠道'] 
df_all = all_models.data_group(df, df2, df_risk_examine, '下单日期')
df_all['出库（按下单时间）'] = df_all['出库']
df_all['出库（按转化时间）'] = df_weekday_zh['All'][:-1]
df_all = df_all[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
                    "出库前风控强拒","待审核",'出库（按下单时间）','出库（按转化时间）', '拒量出库', '进件出库率','总体进件出库率（含拒量）','拒量进件出库率增加','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
df_all_new = df_all.reset_index()
df_all_new.insert(0, '星期', df_all_new['下单日期'].apply(lambda x: day_name_cn_mapping[x.day_name()]))
df_all_new.insert(1, '月份', df_all_new['下单日期'].astype(str).str.split('-').str[0]+'-'+df_all_new['下单日期'].astype(str).str.split('-').str[1])
df_all_new.tail()

,星期,月份,下单日期,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审强拒,强拒比例,机审通过件,人审拒绝,风控通过件,风控通过率,客户取消,无法联系,出库前风控强拒,待审核,出库（按下单时间）,出库（按转化时间）,拒量出库,进件出库率,总体进件出库率（含拒量）,拒量进件出库率增加,取消率,人审拒绝率,出库前强拒比例,无法联系占比,订单出库率
11,星期一,2025-10,2025-10-20,7168,2334,32.56%,1995,27.83%,1447,72.53%,567.0,299,268.0,13.43%,25,45,19.0,1,178,158.0,0,8.92%,8.92%,0.00%,1.25%,14.99%,0.95%,2.26%,2.48%
12,星期二,2025-10,2025-10-21,6768,2161,31.93%,1873,27.67%,1392,74.32%,518.0,295,223.0,11.91%,15,34,14.0,9,151,157.0,0,8.06%,8.06%,0.00%,0.80%,15.75%,0.75%,1.82%,2.23%
13,星期三,2025-10,2025-10-22,6324,2179,34.46%,1750,27.67%,1293,73.89%,495.0,267,228.0,13.03%,20,26,18.0,22,142,155.0,0,8.11%,8.11%,0.00%,1.14%,15.26%,1.03%,1.49%,2.25%
14,星期四,2025-10,2025-10-23,6443,2205,34.22%,1724,26.76%,1243,72.10%,517.0,245,272.0,15.78%,24,9,8.0,105,126,151.0,0,7.31%,7.31%,0.00%,1.39%,14.21%,0.46%,0.52%,1.96%
15,星期五,2025-10,2025-10-24,902,399,44.24%,166,18.40%,116,69.88%,54.0,9,45.0,27.11%,0,0,0.0,45,0,0.0,0,0.00%,0.00%,0.00%,0.00%,5.42%,0.00%,0.00%,0.00%


In [95]:
# df_all.to_excel(f'F:/需求/瑞彤需求/支付宝数据_{t_date}.xlsx')
# df2[((df2['客户取消']==1)|(df2['已退款']==1))&(df2['下单日期']=='2025-09-09')][['order_id','客户取消','取消原因']].shape
# df[(df['下单日期']=='2025-09-15')&(df['待审核']==1)]

# 搜索渠道转化

In [96]:
df_ss = df[df['归属渠道']=='搜索渠道']
df_ss2 = df2[df2['归属渠道']=='搜索渠道']
df_ss_group = all_models.data_group(df_ss, df_ss2, df_risk_examine, '下单日期')

# df_ss_group['出库（按下单时间）'] = df_weekday['搜索渠道'][:-1]
df_ss_group['出库（按下单时间）'] = df_ss_group['出库']
df_ss_group['出库（按转化时间）'] = df_weekday_zh['搜索渠道'][:-1]
df_ss_group = df_ss_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系", "出库前风控强拒","待审核",'出库（按下单时间）','出库（按转化时间）','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
df_ss_group_new = df_ss_group.reset_index()
df_ss_group.tail()


,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审强拒,强拒比例,机审通过件,人审拒绝,风控通过件,风控通过率,客户取消,无法联系,出库前风控强拒,待审核,出库（按下单时间）,出库（按转化时间）,进件出库率,取消率,人审拒绝率,出库前强拒比例,无法联系占比,订单出库率
下单日期,,,,,,,,,,,,,,,,,,,,,,,
2025-10-20,2171,241,11.10%,564,25.98%,390,69.15%,174.0,106,68.0,12.06%,9,12,0.0,0,47,38.0,8.33%,1.60%,18.79%,0.00%,2.13%,2.16%
2025-10-21,2120,227,10.71%,541,25.52%,400,73.94%,141.0,90,51.0,9.43%,3,11,3.0,2,32,32.0,5.91%,0.55%,16.64%,0.55%,2.03%,1.51%
2025-10-22,1707,176,10.31%,451,26.42%,317,70.29%,134.0,84,50.0,11.09%,2,7,0.0,6,35,45.0,7.76%,0.44%,18.63%,0.00%,1.55%,2.05%
2025-10-23,1614,158,9.79%,399,24.72%,265,66.42%,135.0,80,55.0,13.78%,4,3,1.0,16,31,37.0,7.77%,1.00%,20.05%,0.25%,0.75%,1.92%
2025-10-24,210,30,14.29%,35,16.67%,26,74.29%,9.0,3,6.0,17.14%,0,0,0.0,6,0,0.0,0.00%,0.00%,8.57%,0.00%,0.00%,0.00%


## 按周分组，搜索渠道非机审拒绝的订单

In [97]:
# df_ss = df[df['归属渠道']=='搜索渠道']
# # 将下单周转换为时间范围格式，例如"2025.8.20-2025.8.24"
# def get_week_range(date):
#     # 获取该日期所在周的周一
#     monday = date - pd.Timedelta(days=date.weekday())
#     # 获取该日期所在周的周日
#     sunday = monday + pd.Timedelta(days=6)
#     # 格式化为"年.月.日-年.月.日"格式
#     return f"{monday.year}.{monday.month}.{monday.day}-{sunday.year}.{sunday.month}.{sunday.day}"

# df_ss_quyy = df_ss[((df_ss['机审强拒'] == 0)&((df_ss['人审拒绝']==1)|(df_ss['客户取消']==1)|(df_ss['无法联系']==1)|(df_ss['出库前风控强拒']==1)))]
# # 按星期设置列：下单周，周一至周日为一周，未满一周按一周算，eg:8月20到8月24，9月1日到9月4日都算一周
# df_ss_quyy['下单周范围'] = df_ss_quyy['下单日期'].apply(get_week_range)

# df_ss_quyy['取消原因2'] = df_ss_quyy['cancel_reason'].str.split('：').apply(lambda x: x[-1] if x is not None else x)
# df_ss_quyy['取消原因2'].drop_duplicates()
# df_ss_quyy_group = df_ss_quyy.groupby(['下单周范围', '取消原因2']).agg({'order_number': 'count'}).rename(columns={'order_number': '取消数'})
# df_ss_quyy_group = df_ss_quyy_group.sort_values(['下单周范围', '取消数'], ascending=[True, False])
# df_ss_quyy_group['取消率'] = df_ss_quyy_group['取消数']/(df_ss_quyy['是否进件'].sum())
# df_ss_quyy_group['取消率'] = df_ss_quyy_group['取消率'].apply(lambda x: format(x, '.2%'))
# # df_ss_quyy_group.to_excel('F:/需求/瑞彤需求/搜索非机审拒绝的取消原因.xlsx', sheet_name='订单取消原因')
# df_ss_quyy_group

In [98]:
# df_ss_jg = df_ss_quyy[df_ss_quyy.取消原因2=='不接受监管设备'][['下单日期','order_number','机型内存']]
# df_ss_jg.to_excel('F:/需求/瑞彤需求/搜索不接受监管设备.xlsx', sheet_name='订单明细',index=False)

# 单人聊天 转化

In [99]:
df_dr = df[df['来源渠道']=='单人聊天会话中的小程序消息卡片（分享）']
df_dr2 = df2[df2['来源渠道']=='单人聊天会话中的小程序消息卡片（分享）']
df_dr_group = all_models.data_group(df_dr, df_dr2, df_risk_examine,  '下单日期')

# df_dr_group['出库（按下单时间）'] = df_weekday['单人聊天会话中的小程序消息卡片（分享）'][:-1]
df_dr_group['出库（按下单时间）'] = df_dr_group['出库']
try:
    df_dr_group['出库（按转化时间）'] = df_weekday_zh['单人聊天会话中的小程序消息卡片（分享）'][:-1]
except:
    df_dr_group['出库（按转化时间）'] = 0
df_dr_group = df_dr_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例",'出库（按下单时间）','出库（按转化时间）','进件出库率','订单出库率']].fillna(0)
df_dr_group_new = df_dr_group.reset_index()

In [100]:
df_dr_group_new.tail()

,下单日期,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审强拒,强拒比例,出库（按下单时间）,出库（按转化时间）,进件出库率,订单出库率
11,2025-10-20,244,38,15.57%,72,29.51%,43,59.72%,0,0.0,0.00%,0.00%
12,2025-10-21,242,38,15.70%,58,23.97%,43,74.14%,0,0.0,0.00%,0.00%
13,2025-10-22,247,40,16.19%,65,26.32%,48,73.85%,0,0.0,0.00%,0.00%
14,2025-10-23,218,25,11.47%,52,23.85%,35,67.31%,0,0.0,0.00%,0.00%
15,2025-10-24,18,2,11.11%,3,16.67%,2,66.67%,0,0.0,0.00%,0.00%


# 芝麻渠道转化

In [101]:
df_zm = df[df['归属渠道']=='芝麻租物']
df_zm2 = df2[df2['归属渠道']=='芝麻租物']
df_zm_group = all_models.data_group(df_zm, df_zm2, df_risk_examine, '下单日期')

# df_zm_group['出库（按下单时间）'] = df_weekday['芝麻租物'][:-1]
df_zm_group['出库（按下单时间）'] = df_zm_group['出库']
df_zm_group['出库（按转化时间）'] = df_weekday_zh['芝麻租物'][:-1]
df_zm_group = df_zm_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系","出库前风控强拒","待审核",'出库（按下单时间）','出库（按转化时间）','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
df_zm_group_new = df_zm_group.reset_index()
df_zm_group.tail()

,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审强拒,强拒比例,机审通过件,人审拒绝,风控通过件,风控通过率,客户取消,无法联系,出库前风控强拒,待审核,出库（按下单时间）,出库（按转化时间）,进件出库率,取消率,人审拒绝率,出库前强拒比例,无法联系占比,订单出库率
下单日期,,,,,,,,,,,,,,,,,,,,,,,
2025-10-20,2646,1391,52.57%,395,14.93%,212.0,53.67%,202.0,78.0,124.0,31.39%,9,21,0.0,1,93,82.0,23.54%,2.28%,19.75%,0.00%,5.32%,3.51%
2025-10-21,2544,1336,52.52%,404,15.88%,230.0,56.93%,211.0,109.0,102.0,25.25%,9,16,0.0,4,73,81.0,18.07%,2.23%,26.98%,0.00%,3.96%,2.87%
2025-10-22,2450,1319,53.84%,369,15.06%,217.0,58.81%,189.0,89.0,100.0,27.10%,8,10,0.0,9,73,75.0,19.78%,2.17%,24.12%,0.00%,2.71%,2.98%
2025-10-23,2603,1362,52.32%,356,13.68%,190.0,53.37%,201.0,77.0,124.0,34.83%,12,5,1.0,48,58,73.0,16.29%,3.37%,21.63%,0.28%,1.40%,2.23%
2025-10-24,473,284,60.04%,58,12.26%,34.0,58.62%,28.0,3.0,25.0,43.10%,0,0,0.0,25,0,0.0,0.00%,0.00%,5.17%,0.00%,0.00%,0.00%


In [102]:
df_zm = df[(df['归属渠道']=='芝麻租物')&(df['是否出库']==1)]
df_zm.loc[:, '已买断'] = np.where((df_zm.status==8)&(df_zm.has_actual==1)&(df_zm.relet_days==0), 1, 0)
df_zm.loc[:, '已归还'] = np.where((df_zm.status.isin([8, 15, 16]))&(df_zm.has_actual==0)&(df_zm.relet_days==0), 1, 0)
df_zm.loc[:, '已续租'] = np.where(df_zm.relet_days>0, 1, 0)
df_zm_group = df_zm.groupby(['下单月份']).agg({'已买断': 'sum', '已归还': 'sum', '已续租': 'sum','是否出库': 'sum'})
df_zm_group['买断率'] = (df_zm_group['已买断']/df_zm_group['是否出库']).apply(lambda x: format(x, ".2%"))
df_zm_group['归还率'] = (df_zm_group['已归还']/df_zm_group['是否出库']).apply(lambda x: format(x, ".2%"))
df_zm_group['续租率'] = (df_zm_group['已续租']/df_zm_group['是否出库']).apply(lambda x: format(x, ".2%"))
df_zm_group

,已买断,已归还,已续租,是否出库,买断率,归还率,续租率
下单月份,,,,,,,
2025-10,1,20,0,1035,0.10%,1.93%,0.00%


# 纯租物 转化

In [103]:
#逻辑是否需要优化   
df_zw = df[df['来源渠道']=='芝麻信用']
df_zw2 = df2[df2['来源渠道']=='芝麻信用']
df_zw_group = all_models.data_group(df_zw, df_zw2, df_risk_examine, '下单日期')

# df_zw_group['出库（按下单时间）'] = df_ly_ck['芝麻租物'][:-1]
df_zw_group['出库（按下单时间）'] = df_zw_group['出库']
df_zw_group = df_zw_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例",'出库（按下单时间）','进件出库率','订单出库率']]
df_zw_group_new = df_zw_group.reset_index()
df_zw_group_new.tail()

,下单日期,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审强拒,强拒比例,出库（按下单时间）,进件出库率,订单出库率
11,2025-10-20,2624,1380,52.59%,390,14.86%,211.0,54.10%,92,23.59%,3.51%
12,2025-10-21,2528,1334,52.77%,398,15.74%,228.0,57.29%,73,18.34%,2.89%
13,2025-10-22,2433,1307,53.72%,367,15.08%,216.0,58.86%,73,19.89%,3.00%
14,2025-10-23,2580,1353,52.44%,354,13.72%,188.0,53.11%,58,16.38%,2.25%
15,2025-10-24,469,283,60.34%,56,11.94%,32.0,57.14%,0,0.00%,0.00%


In [104]:
# df_zm.是否进件.sum()

# 抖音转化

In [105]:
df_dy = df[df['归属渠道']=='抖音渠道']
df_dy2 = df2[df2['归属渠道']=='抖音渠道']
# df_dy_group = all_models.data_group(df_dy, df_dy2, df_risk_examine, '下单日期')
# df_dy_group = clean.continuous_dates(df_dy_group)
# df_dy_group['出库（按下单时间）'] = df_dy_group['出库']
# try:
#     df_dy_group['出库（按转化时间）'] = df_weekday_zh['抖音渠道'][:-1]
# except:
#     df_dy_group['出库（按转化时间）'] = 0
# df_dy_group = df_dy_group[["去重订单数","进件前取消","进件前取消率","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","客户取消","取消率","人审拒绝","人审拒绝率","待审核","出库前风控强拒",'出库（按下单时间）','出库（按转化时间）','进件出库率','订单出库率']].fillna(0) 
# df_dy_group_new = df_dy_group.reset_index()


In [106]:
classify_id_list = [125,126,127,185]
dfck.loc[:, '抖音电脑'] = np.where((dfck.归属渠道=='抖音渠道')&(dfck.classify_id.isin(classify_id_list)), 1, 0)
# 按转化日期看抖音电脑品类出库
dfck_dy_pc = pd.crosstab(index=dfck['日期'], columns=dfck['抖音电脑'],margins=True)
# 电脑
df_dy_classify = df_dy[df_dy.classify_id.isin(classify_id_list)]
df_d2y_classify = df_dy2[df_dy2.classify_id.isin(classify_id_list)]
df_dy_group = all_models.data_group(df_dy_classify, df_d2y_classify, df_risk_examine, '下单日期')
df_dy_group = clean.continuous_dates(df_dy_group)
df_dy_group['出库（按下单时间）'] = df_dy_group['出库']
try:
    df_dy_group['出库（按转化时间）'] = dfck_dy_pc[1][:-1]
except:
    df_dy_group['出库（按转化时间）'] = 0
df_dy_group = df_dy_group[["去重订单数","进件前取消","进件前取消率","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","客户取消","取消率","人审拒绝","人审拒绝率","待审核","出库前风控强拒",'出库（按下单时间）','出库（按转化时间）', '进件出库率','订单出库率']].fillna(0) 
df_dy_group_new = df_dy_group.reset_index()

In [107]:
# df_dy_group_new = all_models.data_group(df_dy_classify, df_d2y_classify, df_risk_examine, '下单月份')
# df_dy_group_new#.tail()

# 支付宝直播转化

In [108]:
# df_zfb2[(df_zfb2['下单日期']=='2025-07-19')&(df_zfb2['是否出库']==1)][['下单日期','order_number','activity_name','是否拒量','是否出库']]
# 先被拒了，然后进入回捞再出库的，不是原先的出库

In [109]:
# df_zfb[(df_zfb['下单日期']=='2025-07-19')&(df_zfb['是否出库']==1)][['下单日期','order_number','activity_name','是否拒量','是否出库']]

In [110]:
# # 支付宝直播
df_zfb = df[df['activity_name'].str.contains('支付宝直播商品')==True]
df_zfb2 = df2[df2['activity_name'].str.contains('支付宝直播商品')==True]
df_zfb_group = all_models.data_group(df_zfb, df_zfb2, df_risk_examine, '下单日期')
df_zfb_group = clean.continuous_dates(df_zfb_group)

# df_zfb_group['出库（按下单时间）'] = df_zfb_ck['支付宝直播商品'][:-1]
df_zfb_group['出库（按下单时间）'] = df_zfb_group['出库']
df_zfb_group['出库（按转化时间）'] = df_zfb_group['出库']
df_zfb_group = df_zfb_group[["去重订单数","进件前取消","进件前取消率","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","人审拒绝率","风控通过件","客户取消","取消率",'无法联系',"出库前风控强拒","待审核",'出库（按下单时间）', '出库（按转化时间）','进件出库率','订单出库率']]
df_zfb_group1 = df_zfb_group[["去重订单数","进件前取消","进件前取消率","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","人审拒绝率","风控通过件","客户取消","取消率",'无法联系',"出库前风控强拒","待审核",'出库（按下单时间）','进件出库率','订单出库率']].reset_index(names=['下单日期'])
df_zfb_group1.tail()

,下单日期,去重订单数,进件前取消,进件前取消率,前置拦截,拦截率,进件数,预授权通过率,机审强拒,强拒比例,机审通过件,人审拒绝,人审拒绝率,风控通过件,客户取消,取消率,无法联系,出库前风控强拒,待审核,出库（按下单时间）,进件出库率,订单出库率
11,2025-10-20,71,37,52.11%,24,33.80%,10,14.08%,6.0,60.00%,4.0,4.0,40.00%,0.0,0,0.00%,0,0.0,0,0,0.00%,0.00%
12,2025-10-21,58,40,68.97%,10,17.24%,8,13.79%,7.0,87.50%,1.0,0.0,0.00%,1.0,0,0.00%,0,1.0,0,0,0.00%,0.00%
13,2025-10-22,74,45,60.81%,18,24.32%,11,14.86%,9.0,81.82%,2.0,2.0,18.18%,0.0,0,0.00%,0,0.0,0,0,0.00%,0.00%
14,2025-10-23,90,61,67.78%,21,23.33%,8,8.89%,7.0,87.50%,1.0,1.0,12.50%,0.0,0,0.00%,0,0.0,0,0,0.00%,0.00%
15,2025-10-24,17,11,64.71%,3,17.65%,3,17.65%,1.0,33.33%,2.0,0.0,0.00%,2.0,0,0.00%,0,0.0,2,0,0.00%,0.00%


In [225]:
# df_zfb.下单日期.unique()
df_zfb.来源渠道.unique()

array(['芝麻信用', '顶部搜索框的搜索结果页', '其他渠道场景渠道。', '未知渠道', '支付宝直播', '我的小程序入口',
       '支付宝客户端首页', '八派信息', '芝麻信用频道', '支群-社群专享', '社群-每日信息首页', '卡包\r\n',
       '生活号-商品组件', '搜索侠客行-苹果17', '搜索侠客行-核心词', '单人聊天会话中的小程序消息卡片（分享）',
       '邦道-销售线索-电脑', '搜索邦道-小程序推广', '邦道-自定义苹果17', '生活号-J-苹果旗舰', '系统桌面图标',
       '支群2-社群专属福利', '小程序商家消息（服务提醒）', '付费流量(通过商家数字推广平台，灯火等投放的广告)',
       '搜索下挂-商品', '搜索-租手机', '京东出游季', '搜索-租苹果手机', '搜索直达banner', '搜索-租手机苹果'],
      dtype=object)

## 支付宝直播进件前取消原因

In [112]:

# df_zfb_jjqqx = df_zfb[(df_zfb['进件前取消']==1)]
# df_zfb_jjqqx['取消原因2'] = df_zfb_jjqqx['cancel_reason'].str.split('：').apply(lambda x: x[-1] if x is not None else x)
# df_zfb_jjqqx['取消原因2'].drop_duplicates()
# df_zfb_jjqqx_group = df_zfb_jjqqx.groupby(['取消原因2']).agg({'order_number': 'count'}).rename(columns={'order_number': '取消数'})
# df_zfb_jjqqx_group['取消率'] = df_zfb_jjqqx_group['取消数']/(df_zfb_jjqqx_group['取消数'].sum())
# df_zfb_jjqqx_group['取消率'] = df_zfb_jjqqx_group['取消率'].apply(lambda x: format(x, '.2%'))
# df_zfb_jjqqx_group.to_excel('F:/需求/崔婷玉需求/支付宝直播进件前订单取消原因2.xlsx', sheet_name='进件前订单取消原因')



## 支付宝直播取消原因、拒绝理由及占比

In [227]:
# df_zfb['取消原因2'] = df_zfb['cancel_reason'].str.split('：').apply(lambda x: x[-1] if x is not None else x)
# df_zfb['取消原因2'].drop_duplicates()
# df_zfb_qx_group = df_zfb.groupby(['取消原因2']).agg({'order_number': 'count'}).rename(columns={'order_number': '取消数'})
# df_zfb_qx_group['取消率'] = df_zfb_qx_group['取消数']/(df_zfb_qx_group['取消数'].sum())
# df_zfb_qx_group['取消率'] = df_zfb_qx_group['取消率'].apply(lambda x: format(x, '.2%'))
# # 按取消数降序排序
# df_zfb_qx_group = df_zfb_qx_group.sort_values(by='取消数', ascending=False)
# df_zfb_jj_group = df_zfb.groupby(['拒绝理由']).agg({'order_number': 'count'}).rename(columns={'order_number': '拒绝数'})
# df_zfb_jj_group['拒绝率'] = (df_zfb_jj_group['拒绝数']/(df_zfb_jj_group['拒绝数'].sum())).apply(lambda x: format(x, '.2%'))
# df_zfb_jj_group = df_zfb_jj_group.sort_values(by='拒绝数', ascending=False)
# with pd.ExcelWriter('F:/需求/崔婷玉需求/支付宝直播取消拒绝理由占比.xlsx') as writer:
    
#     df_zfb_qx_group.to_excel(writer, sheet_name='取消原因')
#     df_zfb_jj_group.to_excel(writer, sheet_name='拒绝理由')
# df_zfb_qx_group

,取消数,取消率
取消原因2,,
下错型号/重新下单/更换型号,4,0.57%
不同意多预收租金,1,0.14%
不接受监管设备,7,1.00%
信息填写错误,25,3.58%
其他原因,45,6.45%
命中蚂蚁数控风险等级=998强拒,1,0.14%
命中高风险地址,4,0.57%
命中高风险渠道,18,2.58%
多拍/下单轨迹异常,3,0.43%


In [113]:
df1_ck = df_zfb[df_zfb['是否出库']==1]
Today_zfb = datetime.now().strftime('%Y%m%d%H')

# 总体剔除直播数据

In [114]:
df_tc = df_all[['去重订单数', '进件数', '机审强拒', '出库（按下单时间）', '出库（按转化时间）']]-df_dr_group[['去重订单数', '进件数', '机审强拒', '出库（按下单时间）', '出库（按转化时间）']]-df_dy_group[['去重订单数', '进件数', '机审强拒', '出库（按下单时间）', '出库（按转化时间）']]-df_zfb_group[['去重订单数', '进件数', '机审强拒', '出库（按下单时间）', '出库（按转化时间）']]
df_tc.fillna(df_all[['去重订单数', '进件数', '机审强拒', '出库（按下单时间）', '出库（按转化时间）']]-df_dr_group[['去重订单数', '进件数', '机审强拒', '出库（按下单时间）', '出库（按转化时间）']]-df_zfb_group[['去重订单数', '进件数', '机审强拒', '出库（按下单时间）', '出库（按转化时间）']], inplace=True)
df_tc['预授权通过率'] = df_tc['进件数'] / df_tc['去重订单数']
df_tc['预授权通过率'] = df_tc['预授权通过率'].apply(lambda x: format(x, '.2%'))
df_tc['强拒比例'] = df_tc['机审强拒'] / df_tc['进件数']
df_tc['强拒比例'] = df_tc['强拒比例'].apply(lambda x: format(x, '.2%'))
df_tc['进件出库率'] = df_tc['出库（按下单时间）'] / df_tc['进件数']
df_tc['进件出库率'] = df_tc['进件出库率'].apply(lambda x: format(x, '.2%'))
df_tc['订单出库率'] = df_tc['出库（按下单时间）'] / df_tc['去重订单数']
df_tc['订单出库率'] = df_tc['订单出库率'].apply(lambda x: format(x, '.2%'))
df_tc = df_tc[['去重订单数', '进件数', '预授权通过率', '机审强拒', '强拒比例', '出库（按下单时间）', '出库（按转化时间）', '进件出库率', '订单出库率']].reset_index()
df_tc.tail()

,下单日期,去重订单数,进件数,预授权通过率,机审强拒,强拒比例,出库（按下单时间）,出库（按转化时间）,进件出库率,订单出库率
11,2025-10-20,6846,1913,27.94%,1398.0,73.08%,178,158.0,9.30%,2.60%
12,2025-10-21,6451,1800,27.90%,1338.0,74.33%,151,157.0,8.39%,2.34%
13,2025-10-22,5989,1669,27.87%,1231.0,73.76%,142,155.0,8.51%,2.37%
14,2025-10-23,6103,1651,27.05%,1194.0,72.32%,126,151.0,7.63%,2.06%
15,2025-10-24,866,159,18.36%,112.0,70.44%,0,0.0,0.00%,0.00%


# 京享租 2025-05-20正式上线

In [115]:
df_no_drop_jd = df_no_drop[df_no_drop.归属渠道=='京东渠道']
df_no_drop_jd = df_no_drop_jd[df_no_drop_jd.下单日期>='2025-05-20']
df_no_drop_jd.loc[:, '创建进件数'] = np.where(df_no_drop_jd.进件=='进件', 1, 0)
df_no_drop_jd_g = df_no_drop_jd.groupby('下单日期').agg({'order_id': 'count', '创建进件数': 'sum'}).rename(columns={'order_id': '创建订单数'})
df_no_drop_jdsc_g = df_no_drop_jd[df_no_drop_jd.app_type=='JD_APP'].groupby('下单日期').agg({'order_id': 'count'}).rename(columns={'order_id': '创建订单数'})
# df_no_drop_jd_g.head()


In [116]:
df_jd = df[df['归属渠道']=='京东渠道']
df_jd2 = df2[df2['归属渠道']=='京东渠道']
df_jd = df_jd[df_jd.下单日期>='2025-05-20']
df_jd2 = df_jd2[df_jd2.下单日期>='2025-05-20']
df_jd_group = all_models.data_group(df_jd, df_jd2, df_risk_examine, '下单日期')


df_jd_group['出库（按下单时间）'] = df_jd_group['出库']
try:
    df_jd_group['出库（按转化时间）'] = df_weekday_zh['京东渠道'][:-1]
except:
    df_jd_group['出库（按转化时间）'] = 0
df_jd_group = df_jd_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系","出库前风控强拒","待审核",'出库（按下单时间）','出库（按转化时间）','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
df_jd_group_new = df_jd_group.reset_index()
df_jd_group_new = df_no_drop_jd_g.merge(df_jd_group_new, on='下单日期', how='inner')
df_jd_group_new.tail()

,下单日期,创建订单数,创建进件数,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审强拒,强拒比例,机审通过件,人审拒绝,风控通过件,风控通过率,客户取消,无法联系,出库前风控强拒,待审核,出库（按下单时间）,出库（按转化时间）,进件出库率,取消率,人审拒绝率,出库前强拒比例,无法联系占比,订单出库率
11,2025-10-20,2048,987,1465,559,38.16%,838,57.20%,712.0,84.96%,126.0,78.0,48.0,5.73%,4,11,0.0,0,33,33.0,3.94%,0.48%,9.31%,0.00%,1.31%,2.25%
12,2025-10-21,1696,845,1286,484,37.64%,750,58.32%,628.0,83.73%,122.0,70.0,52.0,6.93%,2,4,0.0,3,43,41.0,5.73%,0.27%,9.33%,0.00%,0.53%,3.34%
13,2025-10-22,1774,842,1321,541,40.95%,730,55.26%,612.0,83.84%,118.0,68.0,50.0,6.85%,6,7,0.0,7,30,31.0,4.11%,0.82%,9.32%,0.00%,0.96%,2.27%
14,2025-10-23,1944,916,1444,565,39.13%,809,56.02%,680.0,84.05%,129.0,63.0,66.0,8.16%,5,1,0.0,26,34,39.0,4.20%,0.62%,7.79%,0.00%,0.12%,2.35%
15,2025-10-24,234,85,142,74,52.11%,59,41.55%,47.0,79.66%,12.0,3.0,9.0,15.25%,0,0,0.0,9,0,0.0,0.00%,0.00%,5.08%,0.00%,0.00%,0.00%


In [117]:
df_jd.机型内存.value_counts()

机型内存
iPhone16ProMax256GB             14785
iPhone16ProMax512GB              1912
iPhone 17 Pro Max256GB           1056
iPhone16Pro256GB                  661
iPhone 17 Pro Max512GB            387
                                ...  
iPad11 2025款 11英寸512GB WiFi版        1
OPPO FindX8Pro16GB+512GB            1
OPPO Reno1416GB+512GB               1
iPhone Air512GB                     1
OPPO Reno1412GB+256GB               1
Name: count, Length: 70, dtype: int64

## 临时需求

### 京东二手商品转化数据及取消原因，含商家

In [118]:
# df_jd_es = df[(df['归属渠道']=='京东渠道')&(df['商品类型']=='二手')]
# df_jd_es.shape
# df_jd_es2 = df2[(df2['归属渠道']=='京东渠道')&(df2['商品类型']=='二手')&(df2['商品ID'].isin([7979, 7975]))]
# # df_jd_es.loc[:, '日期范围'] = '0809-0934'
# # df_jd_es2.loc[:, '日期范围'] = '0809-0934'
# df_jdes_group = all_models.data_group(df_jd_es, df_jd_es2, df_risk_examine, '下单日期')

# df_jdes_group = df_jdes_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                     "出库前风控强拒","待审核",'出库', '拒量出库', '进件出库率','总体进件出库率（含拒量）','拒量进件出库率增加','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
# df_jdes_group.loc['汇总', :] = df_jdes_group.select_dtypes(include=[np.number]).sum()
# df_jdes_group_new = df_jdes_group.reset_index()
# # 添加汇总行，仅计算数值型

# df_jdes_group_new.tail()#.to_excel('F:/需求/崔婷玉需求/京东二手商品总体转化数据.xlsx')


In [119]:
# df[(df['归属渠道']=='京东渠道')&(df['商品类型']=='二手')].商品ID.unique()

In [120]:
# df_jdes_jj = df_jd_es[(df_jd_es['是否进件']==1)]
# df_jdes_jj['取消原因2'] = df_jdes_jj['cancel_reason'].str.split('：').apply(lambda x: x[-1] if x is not None else x)
# df_jdes_jj['取消原因2'].drop_duplicates()
# df_jdes_qx = df_jdes_jj.groupby('取消原因2').agg({'order_number': 'count'}).rename(columns={'order_number': '取消数'})
# df_jdes_qx['取消率'] = df_jdes_qx['取消数']/(df_jdes_jj['是否进件'].sum())
# df_jdes_qx['取消率'] = df_jdes_qx['取消率'].apply(lambda x: format(x, '.2%'))
# df_jdes_qx#.to_excel('F:/需求/崔婷玉需求/京东二手商品总体机审强拒理由.xlsx')
# with pd.ExcelWriter('F:/需求/崔婷玉需求/京东二手特定商品ID转化数据及取消原因.xlsx', engine='xlsxwriter') as writer:
#     df_jdes_group_new.to_excel(writer, sheet_name='京东二手商品转化数据', index=False)
#     df_jdes_qx.to_excel(writer, sheet_name='京东二手商品取消原因')

In [121]:
# df_jd_qs = df[(df['归属渠道']=='京东渠道')&(df['是否出库']==1)]
# df_jd_qs.shape
# df_jd_qs.loc[:, '订单状态'] = np.where(df_jd_qs['退货拒签']==1, '已退货',np.where((df_jd_qs['status']==4)&(df_jd_qs.overdue==1), '逾期', df_jd_qs['status2']))
# # df_jd_qs.loc[:, '是否逾期'] = np.where((df_jd_qs['status']==4)&(df_jd_qs.overdue==1), '逾期', '未逾期')#(df['退货拒签']==0)&
# # df_jd_qs.loc[:, '是否买断'] = np.where(df_jd_qs.has_actual==1, '买断', '未买断')
# df_jd_qs[['order_number', '订单状态']].to_excel('F:/需求/崔婷玉需求/京东渠道8月出库订单号.xlsx', index=False)

In [122]:
# df_no_drop_jd = df_no_drop[df_no_drop.来源渠道 == '京东618活动']
# df_no_drop_jd.loc[:, '创建进件数'] = np.where(df_no_drop_jd.进件 == '进件', 1, 0)
# df_no_drop_jd_g = df_no_drop_jd.groupby('机型内存').agg({'order_id': 'count', '创建进件数': 'sum'}).rename(columns={'order_id': '创建订单数'})
# df_jd_yk = df[df['来源渠道'] == '京东618活动']
# df_jd2_yk = df2[df2['来源渠道'] == '京东618活动']
# df_jd_group = all_models.data_group(df_jd_yk, df_jd2_yk, df_risk_examine, '机型内存')
# df_jd_group = df_jd_group[
#     ["去重订单数", "前置拦截", "拦截率", "进件数", "预授权通过率", "机审强拒", "强拒比例", "机审通过件",
#         "人审拒绝", "风控通过件", "风控通过率", "客户取消", "无法联系", "出库前风控强拒", "待审核",
#         '出库', '进件出库率', '取消率', '人审拒绝率', '出库前强拒比例',
#         '无法联系占比', '订单出库率']].fillna(0)
# df_jd_group_new = df_jd_group.reset_index()
# df_jd_group_new2 = df_no_drop_jd_g.merge(df_jd_group_new, on='机型内存', how='inner')

# df_jd_group_new2.to_excel('F:\需求\瑞彤需求\京东618活动机型出库数据.xlsx', index=False)


## 统计京东订单，苹果17系列所有机型转化数据

In [123]:
# # df_jd17 = df_jd[df_jd['product_name'].str.contains(r'iPhone 17|iPhone Air')]
# df_jd17 = all_models.data_group(df_jd, df_jd2, df_risk_examine, '机型内存')

# df_jd17 = df_jd17[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系","出库前风控强拒","待审核",'出库','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
# df_jd17

## 按下单周拉取转化数据

In [124]:
# # 按下单日期所在本周的周四到下周三为一周，eg:，8月7日到8月13日为一周，8月7日到8月13日的下单周范围都为2025.8.7-2025.8.13，由此逻辑设置列：下单周范围
# def get_week_range_v2(date):
#     # 获取该日期所在周的周四
#     # weekday(): 周一=0, 周二=1, ..., 周四=3, ..., 周日=6
#     days_since_thursday = (date.weekday() - 3) % 7
#     thursday = date - pd.Timedelta(days=days_since_thursday)
#     # 获取下周的周三（周四+6天）
#     next_wednesday = thursday + pd.Timedelta(days=6)
#     # 格式化为"年.月.日-年.月.日"格式
#     return f"{thursday.year}.{thursday.month}.{thursday.day}-{next_wednesday.year}.{next_wednesday.month}.{next_wednesday.day}"

# df_no_drop_jd = df_no_drop[df_no_drop.归属渠道=='京东渠道']
# df_jd = df[df['归属渠道']=='京东渠道']
# df_jd2 = df2[df2['归属渠道']=='京东渠道']

# df_no_drop_jd.loc[:, '创建进件数'] = np.where(df_no_drop_jd.进件=='进件', 1, 0)

# df_no_drop_jd.loc[:, '下单周范围'] = df_no_drop_jd['下单日期'].apply(get_week_range_v2)
# df_jd.loc[:, '下单周范围'] = df_jd['下单日期'].apply(get_week_range_v2)
# df_jd2.loc[:, '下单周范围'] = df_jd2['下单日期'].apply(get_week_range_v2)

# # 京东商城
# df_no_drop_jdsc_g = df_no_drop_jd[df_no_drop_jd.app_type=='JD_APP'].groupby('下单日期').agg({'order_id': 'count','创建进件数': 'sum'}).rename(columns={'order_id': '创建订单数'})
# df_jdsc = df_jd[df_jd.app_type=='JD_APP']
# df_jdsc2 = df_jd2[df_jd2.app_type=='JD_APP']
# df_jdsc_group = all_models.data_group(df_jdsc, df_jdsc2, df_risk_examine, '下单日期')
# df_jdsc_group['出库（按下单时间）'] = df_jdsc_group['出库']

# df_jdsc_group = df_jdsc_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系","出库前风控强拒","待审核",'出库（按下单时间）','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
# df_jdsc_group_new = df_jdsc_group.reset_index()
# df_jdsc_group_new = df_no_drop_jdsc_g.merge(df_jdsc_group_new, on='下单日期', how='left')

# # 京东金融
# df_no_drop_jdjr_g = df_no_drop_jd[df_no_drop_jd.app_type=='JR_APP'].groupby('下单日期').agg({'order_id': 'count','创建进件数': 'sum'}).rename(columns={'order_id': '创建订单数'})
# df_jdjr = df_jd[df_jd.app_type=='JR_APP']
# df_jdjr2 = df_jd2[df_jd2.app_type=='JR_APP']
# df_jdjr_group = all_models.data_group(df_jdjr, df_jdjr2, df_risk_examine, '下单日期')
# df_jdjr_group['出库（按下单时间）'] = df_jdjr_group['出库']

# df_jdjr_group = df_jdjr_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系","出库前风控强拒","待审核",'出库（按下单时间）','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
# df_jdjr_group_new = df_jdjr_group.reset_index()
# df_jdjr_group_new = df_no_drop_jdjr_g.merge(df_jdjr_group_new, on='下单日期', how='left')
# df_jdsc_group_new

In [125]:
# path = 'F:\需求\崔婷玉需求/'
# with pd.ExcelWriter(path+f'京东渠道各子渠道出库数据_{t_date}.xlsx', engine='openpyxl') as writer:
#     df_jdsc_group_new.to_excel(writer, sheet_name='京东商城', index=False)
#     df_jdjr_group_new.to_excel(writer, sheet_name='京东金融', index=False)

## 客户取消

In [126]:
# df_jd.columns

### 取消原因1：创建订单后处于订单取消状态

In [127]:
# df_no_drop_jd = df_no_drop[(df_no_drop.归属渠道=='京东渠道')&(df_no_drop['product_name'].str.contains(r'iPhone 17|iPhone Air'))]
# df_no_drop_jd.loc[:, '取消1'] = np.where((df_no_drop_jd['status2'] == '订单取消')&(df_no_drop_jd['order_id'].notna()), 1, 0)
# df_no_drop_jd_group = df_no_drop_jd.groupby('下单日期').agg({'取消1':'sum', 'order_id':'count'}).rename(columns={'取消1':'取消数', 'order_id':'创建订单数'})
# df_no_drop_jd_group['取消率'] = (df_no_drop_jd_group['取消数']/df_no_drop_jd_group['创建订单数']).apply(lambda x: format(x, '.2%'))
# df_no_drop_jd_group

### 取消原因2：进件后，存在机审强拒、人审拒绝、客户取消、无法联系、出库前风控强拒中至少有一个为1

In [128]:
# df_jd = df[(df['归属渠道']=='京东渠道')&(df['product_name'].str.contains(r'iPhone 17|iPhone Air'))]
# df_jd.loc[:, '取消2'] = np.where(((df_jd['机审强拒'] == 1)|(df_jd['人审拒绝']==1)|(df_jd['客户取消']==1)|(df_jd['无法联系']==1)|(df_jd['出库前风控强拒']==1)), 1, 0)
# df_jd_group = df_jd.groupby('下单日期').agg({'取消2':'sum', '是否进件':'sum'}).rename(columns={'取消2':'取消数', '是否进件':'进件数'})
# df_jd_group['取消率'] = (df_jd_group['取消数']/df_jd_group['进件数']).apply(lambda x: format(x, '.2%'))
# df_jd_group

### 取消原因4：审核通过后或有支付流水且状态为已退款

In [129]:
# df_jd.loc[:, '审核通过'] = np.where(df_jd['total_describes'].str.contains('msg=审核通过'), 1, 0)
# df_jd.loc[:, '取消4'] = np.where((df_jd['审核通过']==1)&(df_jd['status2']=='已退款'), 1, 0)
# df_jd_group_sh = df_jd.groupby('下单日期').agg({'取消4':'sum', '审核通过':'sum'}).rename(columns={'取消4':'取消数', '审核通过':'审核通过数'})
# df_jd_group_sh.loc[:, '取消率'] = (df_jd_group_sh['取消数']/df_jd_group_sh['审核通过数']).apply(lambda x: format(x, '.2%'))
# df_jd_group_sh

In [130]:
# # 写入Excel
# with pd.ExcelWriter('F:/需求/瑞彤需求/京东取消原因订单_月份.xlsx', engine='openpyxl') as writer:
#     df_no_drop_jd_group.to_excel(writer, sheet_name='取消原因1')
#     df_jd_group.to_excel(writer, sheet_name='取消原因2')
#     df_jd_group_sh.to_excel(writer, sheet_name='取消原因4')

### 发货前订单取消原因

In [131]:
# df_jd_shtg = df_jd#[(df_jd['风控通过件']==1)&(df_jd['status2']=='已退款')]
# df_jd_shtg.loc[:,'发货前取消'] = np.where((df_jd['status2']=='已退款')&(df_jd['风控通过件']==1)&(df_jd['出库前风控强拒']==0),1,0)
# # 物流状态=5，退货拒签
# # df.loc[:,"退货拒签"] = np.where(df['物流状态']==5, 1, 0)
# df_jd_shtg.loc[:,'发货拒签'] = np.where((df_jd['物流状态']==5)&(df_jd['是否出库']==1),1,0)
# df_jd_shtg_group = df_jd_shtg.groupby('下单月份').agg({'发货前取消':'sum', '发货拒签':'sum', '是否进件':'sum'})#.rename(columns={'发货前取消':'发货前取消数', '发货拒签':'发货拒签数'})
# df_jd_shtg_group['发货前取消率'] = (df_jd_shtg_group['发货前取消']/df_jd_shtg_group['是否进件']).apply(lambda x: format(x, ".2%"))
# df_jd_shtg_group['发货拒签率'] = (df_jd_shtg_group['发货拒签']/df_jd_shtg_group['是否进件']).apply(lambda x: format(x, ".2%"))
# df_jd_shtg_group

In [132]:
# df_xxx = df_jd_shtg[['下单日期', '取消原因2', 'order_number']]
# df_xxx#.取消原因2.

In [133]:
# # [df_jd['product_name'].str.contains(r'iPhone 17|iPhone Air')]
# df_jd = df[df['归属渠道']=='京东渠道']
# df_jd = df_jd[df_jd['product_name'].str.contains(r'iPhone 17|iPhone Air')]
# # df_jd_shtg = df_jd[(df_jd['风控通过件']==1)&(df_jd['status2']=='已退款')&(df_jd['出库前风控强拒']==0)]
# # df_jd_shtg = df_jd[(df_jd['是否进件'] == 1) & (df_jd['机审强拒'] == 0) & (df_jd['人审拒绝'] == 0)&(df_jd['status2']=='已退款')&(df_jd['出库前风控强拒']==0)]
# df_jd_shtg = df_jd[(df_jd['是否进件'] == 1)]
# df_jd_shtg['取消原因2'] = df_jd_shtg['cancel_reason'].str.split('：').apply(lambda x: x[-1] if x is not None else x)
# df_jd_shtg['取消原因2'].drop_duplicates()
# df_jd_shtg_group = df_jd_shtg.groupby(['下单日期', 'rejected']).agg({'order_number': 'count'}).rename(columns={'order_number': '取消数'})
# df_jd_shtg_group['取消率'] = df_jd_shtg_group['取消数']/(df_jd_shtg['是否进件'].sum())
# df_jd_shtg_group['取消率'] = df_jd_shtg_group['取消率'].apply(lambda x: format(x, '.2%'))
# df_jd_shtg_group.shape#.to_excel('F:/需求/崔婷玉需求/iPhone17取消原因.xlsx', sheet_name='取消原因')

In [134]:
# # 写入Excel
# with pd.ExcelWriter('F:/需求/瑞彤需求/京东取消订单_月份.xlsx', engine='openpyxl') as writer:
#     df_tkjd_kh_group.to_excel(writer, sheet_name='客户取消')
#     df_jdqj_group.to_excel(writer, sheet_name='机审人审拒绝')
#     df_jd_shtg_group.to_excel(writer, sheet_name='审核通过取消订单')

## 京东商城

In [135]:
df_jdsc = df_jd[df_jd.app_type=='JD_APP']
df_jdsc2 = df_jd2[df_jd2.app_type=='JD_APP']
df_jdsc_group = all_models.data_group(df_jdsc, df_jdsc2, df_risk_examine, '下单日期')
df_jdsc_group['出库（按下单时间）'] = df_jdsc_group['出库']

df_jdsc_group = df_jdsc_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系","出库前风控强拒","待审核",'出库（按下单时间）','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
df_jdsc_group_new = df_jdsc_group.reset_index()
df_jdsc_group_new = df_no_drop_jdsc_g.merge(df_jdsc_group_new, on='下单日期', how='left')
df_jdsc_group_new.tail()

,下单日期,创建订单数,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审强拒,强拒比例,机审通过件,人审拒绝,风控通过件,风控通过率,客户取消,无法联系,出库前风控强拒,待审核,出库（按下单时间）,进件出库率,取消率,人审拒绝率,出库前强拒比例,无法联系占比,订单出库率
11,2025-10-20,650,533,0,0.00%,521,97.75%,445.0,85.41%,76.0,43.0,33.0,6.33%,3,6,0.0,0,24,4.61%,0.58%,8.25%,0.00%,1.15%,4.50%
12,2025-10-21,601,515,0,0.00%,501,97.28%,418.0,83.43%,83.0,46.0,37.0,7.39%,1,3,0.0,2,31,6.19%,0.20%,9.18%,0.00%,0.60%,6.02%
13,2025-10-22,570,476,0,0.00%,465,97.69%,393.0,84.52%,72.0,42.0,30.0,6.45%,4,3,0.0,2,21,4.52%,0.86%,9.03%,0.00%,0.65%,4.41%
14,2025-10-23,620,521,0,0.00%,500,95.97%,428.0,85.60%,72.0,33.0,39.0,7.80%,1,0,0.0,15,23,4.60%,0.20%,6.60%,0.00%,0.00%,4.41%
15,2025-10-24,57,39,0,0.00%,36,92.31%,29.0,80.56%,7.0,2.0,5.0,13.89%,0,0,0.0,5,0,0.00%,0.00%,5.56%,0.00%,0.00%,0.00%


# 渠道的退货订单数

In [136]:
# 输出df归属渠道的种类
# df['归属渠道'].unique()
# 获取df中所以退货拒签的订单
df_qudao = df2[df2['是否出库']==1]
# df_qudao = df_qudao[df_qudao.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True]#无用
# df_qudao = df_qudao[~df_qudao.merchant_name.isin(["小蚂蚁租机","兴鑫兴通讯","人人享租","崇胜数码","喜卓灵租机"])]

# 将归属渠道中的渠道数据依次取出芝麻租物， 搜索渠道，单人会话，S量，抖音渠道，，京东渠道（且前面六个渠道都需排除拒量），拒量(已存在是否拒量)的退货订单数
df_qudao['芝麻租物'] = np.where((df_qudao.归属渠道=='芝麻租物')&(df_qudao.是否拒量!=1)&(df_qudao.退货拒签==1), 1, 0)
df_qudao['搜索渠道'] = np.where((df_qudao.归属渠道=='搜索渠道')&(df_qudao.是否拒量!=1)&(df_qudao.退货拒签==1), 1, 0)
df_qudao['单人会话'] = np.where((df_qudao.归属渠道=='单人会话')&(df_qudao.是否拒量!=1)&(df_qudao.退货拒签==1), 1, 0)
conditions_s = ['八派信息', 'CPS直播', '派金花', '九州信息', '勉丫租', '哈银', '租瓜直播', '美仑美奂',
                        '分期乐1', '分期乐2', '98租超', '推一推', '硬派抖音']
df_qudao['S量'] = np.where((df_qudao.归属渠道.isin(conditions_s))&(df_qudao.是否拒量!=1)&(df_qudao.退货拒签==1), 1, 0)
df_qudao['抖音渠道'] = np.where((df_qudao.归属渠道=='抖音渠道')&(df_qudao.是否拒量!=1)&(df_qudao.退货拒签==1), 1, 0)

df_qudao['拒量'] = np.where((df_qudao['是否拒量']==1)&(df_qudao.退货拒签==1), 1, 0)

df_qudao['京东渠道'] = np.where((df_qudao.归属渠道=='京东渠道')&(df_qudao.是否拒量!=1)&(df_qudao.退货拒签==1), 1, 0)
# 将各个渠道的订单数相加
df_qudao_group = df_qudao.groupby('下单日期').agg({'是否出库':'sum','退货拒签':'sum','芝麻租物': 'sum', '搜索渠道': 'sum', '单人会话': 'sum', 'S量': 'sum', '抖音渠道': 'sum', '拒量': 'sum', '京东渠道': 'sum'}).rename(columns={'是否出库': '总体出库订单数', '退货拒签': '拒收订单数'})
df_qudao_group = df_qudao_group.reset_index(names=['下单日期'])

# df_qudao_group#.tail()#.to_excel('F:\日报\日报\拒收.xlsx', index=False, header=True)

df_qudao['下单日期'].unique()

<DatetimeArray>
['2025-10-09 00:00:00', '2025-10-10 00:00:00', '2025-10-11 00:00:00',
 '2025-10-12 00:00:00', '2025-10-13 00:00:00', '2025-10-14 00:00:00',
 '2025-10-15 00:00:00', '2025-10-16 00:00:00', '2025-10-17 00:00:00',
 '2025-10-18 00:00:00', '2025-10-19 00:00:00', '2025-10-20 00:00:00',
 '2025-10-21 00:00:00', '2025-10-22 00:00:00', '2025-10-23 00:00:00']
Length: 15, dtype: datetime64[ns]

## 验证退货拒签的订单数是否正确

In [137]:
# 计算df_qudao_group所有下单日期的拒收订单数
# df_qudao_group['拒收订单数'].sum()


In [138]:
# sql_th = """
# select om.order_number,om.create_time
# from db_digua_business.t_order om
# left join db_digua_business.t_order_logistics tolog on om.id = tolog.order_id
# where tolog.status =  5
# and  DATE_FORMAT(om.create_time, '%Y-%m-%d')>='2025-06-01'
# and  DATE_FORMAT(om.create_time, '%Y-%m-%d')<='2025-06-30'
# """
# df_th = clean.query(sql_th)
# # 统计订单总数
# df_th


# 免审转化

判断免人审的单可以用t_order_risk表中的risk json串中的is_vip=1和status=0

In [139]:
df_ms = df[(df.is_vip=='1')&(df.status_result=='0')]
df_ms2 = df2[(df2.is_vip=='1')&(df2.status_result=='0')]
df_ms_group = all_models.data_group(df_ms, df_ms2, df_risk_examine,  '下单日期')

df_ms_group = df_ms_group[["进件数","人审拒绝", "客户取消","出库","待审核","进件出库率","取消率","人审拒绝率"]] 
# df_ms_group = df_ms_group[["进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
                    # "出库前风控强拒","待审核",'出库', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']]
df_ms_group_new = df_ms_group.reset_index()
df_ms_group.tail()


,进件数,人审拒绝,客户取消,出库,待审核,进件出库率,取消率,人审拒绝率
下单日期,,,,,,,,
2025-10-20,515,240.0,24,172,1,33.40%,4.66%,46.60%
2025-10-21,473,240.0,14,148,7,31.29%,2.96%,50.74%
2025-10-22,453,215.0,18,137,22,30.24%,3.97%,47.46%
2025-10-23,456,184.0,22,126,94,27.63%,4.82%,40.35%
2025-10-24,53,7.0,0,0,42,0.00%,0.00%,13.21%


## 信审需求（倩姐）

In [140]:

# # df_ms_list = df_ms[(df_ms['下单日期']=='2025-08-18')&(df_ms['机审通过件']==1)]['order_id'].tolist()
# # df2[df2['机审通过件']==1].groupby('下单日期').agg({'order_id':'count'})
# # 免审订单号列表
# df_ms_list = df[(df['下单日期']=='2025-08-18')&(df['机审通过件']==1)&(df.is_vip=='1')&(df.status_result=='0')]['order_id'].tolist()
# len(df_ms_list)
# # df_ms_list
# # 机审通过件中缺少拒量的通过件，信审订单数
# # df2[(df2['下单日期']=='2025-08-18')&(df2['机审通过件']==1)&(~df2['order_id'].isin(df_ms_list))&((df2['is_vip'] == '0'))]['order_id'].count()
# # 机审通过件，不含拒量的通过件
# df2[(df2['下单日期']=='2025-08-18')&(df2['机审通过件']==1)]['order_id'].count()

### 获取机审通过订单中拒量的订单

In [141]:
# def xmy(df, df_risk_examine):
#         '''
#         计算小蚂蚁租机的总体转移，机审强拒，人审拒绝，出库前风控强拒数据
#         :param df: 总体数据的df
#         :param df_risk_examine: 出库前风控强拒数据的df
#         :param model: 传入的分组参数
#         :return: 返回小蚂蚁租机的数据
#         '''
#         # 匹配小蚂蚁（拒绝）数据
#         df_xmy = df[df['merchant_name'].isin(['小蚂蚁租机', '兴鑫兴通讯', '人人享租', '崇胜数码', '喜卓灵租机', '喜卓灵新租机'])]
#         # 出库前强拒数据重命名
#         df_risk_examine.rename(columns={'time': 'time_risk_ex', 'status': 'status_risk_ex'}, inplace=True)
#         # 对小蚂蚁数据和出库前强拒数据进行拼接
#         df_risk_examine_all = pd.merge(df_xmy, df_risk_examine, left_on='id_card_num', right_on='id_card', how='inner')
#         # 计算出库前强拒的订单数
#         df_risk_examine_all2 = df_risk_examine_all[(df_risk_examine_all['time_risk_ex'] < df_risk_examine_all['update_time'])]
#         # 进行排序并取到最近的一个订单
#         df_risk_examine_all2 = df_risk_examine_all2.sort_values(['order_id', 'time_risk_ex'], ascending=[True, False]).groupby('order_id').head(1)
#         df_risk_examine_all2 = df_risk_examine_all2[df_risk_examine_all2['status_risk_ex'] == '1']
#         # df_risk_examine_all2_g = df_risk_examine_all2.groupby(model).agg(order_risk_ex=('order_id', 'size'))
#         # df_risk_examine_all2_g
#         # 排除出库前风控强拒的订单
#         df_xmy_new = df_xmy[~df_xmy['order_id'].isin(df_risk_examine_all2['order_id'].to_list())]
#         # 定义人审拒绝订单
#         df_xmy_new['小蚂蚁人审拒绝'] = np.where(df_xmy_new['reason'] != '系统风控拒绝转移', 1, 0)
#         print(" 小蚂蚁人审拒绝：",df_xmy_new[(df_xmy_new['下单日期']=='2025-08-18')&(df_xmy_new['小蚂蚁人审拒绝']==1)]['order_id'].to_list())
#         print(df_xmy_new[(df_xmy_new['下单日期']=='2025-08-18')&(df_xmy_new['小蚂蚁人审拒绝']==1)]['order_id'].count())
#         print("order_risk_ex:", df_risk_examine_all2[(df_risk_examine_all2['下单日期']=='2025-08-18')]['order_id'].to_list())
#         print(df_risk_examine_all2[(df_risk_examine_all2['下单日期']=='2025-08-18')]['order_id'].count())
        
# xmy(df, df_risk_examine)
# df_241205 = df[df.tips.str.contains(r'策略241205|策略241212|命中自有模型回捞策略|回捞策略250330命中', regex=True)==True]
# # 拒绝
# df_241205 = df_241205[~df_241205.merchant_name.isin(['小蚂蚁租机', '兴鑫兴通讯', '人人享租', '崇胜数码', '喜卓灵租机', '喜卓灵新租机'])]
# df_241205.loc[:, '出库前强拒_拒量'] = np.where(((df_241205.qvt_risk == '0')|(df_241205.qvt_risk.isna()))&(df_241205.qvt_result == '1'), 1, 0)
# df_241205[(df_241205['出库前强拒_拒量']==1)&(df_241205['下单日期']=='2025-08-18')]['order_id'].count()


### 信审订单，按下单日期统计

In [142]:
# # 信审订单数
# df_credit = df2[(df2['机审通过件']==1)&((df2['is_vip'] == '0')&(df.status_result=='0'))]#  | (df['is_vip'].isna())
# # 筛选未命中发货需上锁的订单
# # df_credit_examine = df_credit_examine[~(df_credit_examine.tips.str.contains('发货需上锁', na=False))]
# # df_credit = df_credit[~(df_credit.tips.str.contains('发货需上锁', na=False))]
# # # 基于下单日期和id_card_num去重
# df_credit = df_credit.drop_duplicates(subset=['下单日期', 'id_card_num'],keep="first")
# df_credit_group = df_credit.groupby('下单日期').agg({'order_id':'count'})
# # df_creadit_list = df_credit[df_credit['下单日期']=='2025-08-18']['order_id'].tolist()
# # df_credit.tail()
# # len(df_creadit_list)
# df_credit_group

## 各渠道每日免审出库单数统计

In [143]:
# 剔除据量的数据
# dfck_ms = dfck[(~dfck.tips.str.contains(r'策略2412|命中自有模型回捞策略')==True)&(dfck.status!=12)]
dfck_ms = clean.drop_rejected_merchant(dfck)
dfdfck=dfck_ms[(dfck_ms.is_vip=='1')&(dfck_ms.status_result=='0')]

dfdfck_ck = pd.crosstab(dfdfck["下单日期"],dfdfck["归属渠道"],margins=True)[['芝麻租物','搜索渠道']][:-1]

# 删除原先索引
# dfdfck_ck = dfdfck_ck.reset_index(drop=True)
dfdfck_ck.rename(columns={'芝麻租物': '芝麻租物免审出库', '搜索渠道': '搜索免审出库'}, inplace=True)
dfdfck_ck = clean.continuous_dates(dfdfck_ck)
dfdfck_ck = dfdfck_ck.reset_index(names=['下单日期'])[['芝麻租物免审出库', '搜索免审出库']]#.set_index('下单日期')
dfdfck_ck.tail()


归属渠道,芝麻租物免审出库,搜索免审出库
11,87.0,47.0
12,72.0,32.0
13,69.0,35.0
14,56.0,31.0
15,0.0,0.0


## 租物  搜索的免审进件

In [144]:
dfms = df[(df.is_vip=='1')&(df.status_result=='0')]
dfms_jj = pd.crosstab(dfms["下单日期"],dfms["归属渠道"])
# 删除原先索引
dfms_jj = dfms_jj.reset_index(drop=True)
# 添加下单日期
dfms_jj['下单日期'] = df_ms_group_new['下单日期']
dfms_jj = dfms_jj[['下单日期', "芝麻租物","搜索渠道"]]
dfms_jj.rename(columns={'芝麻租物': '芝麻租物免审进件', '搜索渠道': '搜索渠道免审进件'}, inplace=True)
dfms_jj.tail()

归属渠道,下单日期,芝麻租物免审进件,搜索渠道免审进件
11,2025-10-20,207,166
12,2025-10-21,218,127
13,2025-10-22,202,115
14,2025-10-23,202,112
15,2025-10-24,28,10


In [145]:
# 整合
df_ms_new = dfms_jj.copy()
df_ms_new.insert(0, '月份', df_ms_new['下单日期'].astype(str).str.split('-').str[0]+'-'+df_ms_new['下单日期'].astype(str).str.split('-').str[1])
df_ms_new.insert(2, '总体进件', df_all_new['进件数'])
df_ms_new.insert(3, '芝麻租物进件', df_zm_group_new['进件数'])
df_ms_new.insert(4, '免审进件', df_ms_group_new['进件数'])
df_ms_new.loc[:, '免审进件占比'] = df_ms_new['免审进件'] / df_ms_new['总体进件']
df_ms_new.loc[:, '免审进件占比'] = df_ms_new['免审进件占比'].apply(lambda x: format(x, '.2%'))
df_ms_new.loc[:, '芝麻租物免审进件占比'] = df_ms_new['芝麻租物免审进件'] / df_ms_new['芝麻租物进件']
df_ms_new.loc[:, '芝麻租物免审进件占比'] = df_ms_new['芝麻租物免审进件占比'].apply(lambda x: format(x, '.2%'))
df_ms_new.loc[:, '搜索免审进件占比'] = df_ms_new['搜索渠道免审进件'] / df_ss_group_new['进件数']
df_ms_new.loc[:, '搜索免审进件占比'] = df_ms_new['搜索免审进件占比'].apply(lambda x: format(x, '.2%'))
df_ms_new.loc[:, '总体出库'] = df_all_new['出库（按下单时间）']
df_ms_new.loc[:, '芝麻租物出库'] = df_zm_group_new['出库（按下单时间）']
df_ms_new.loc[:, '免审出库'] = df_ms_group_new['出库']
df_ms_new2 = pd.concat([df_ms_new, dfdfck_ck], axis=1)
# df_ms_new2
df_ms_new2.loc[:, '免审转化率'] = df_ms_new2['免审出库'] / df_ms_new2['免审进件']
df_ms_new2.loc[:, '免审转化率'] = df_ms_new2['免审转化率'].apply(lambda x: format(x, '.2%'))
df_ms_new2.loc[:, '免审出库占比'] = df_ms_new2['免审出库'] / df_ms_new2['总体出库']
df_ms_new2.loc[:, '免审出库占比'] = df_ms_new2['免审出库占比'].apply(lambda x: format(x, '.2%'))
df_ms_new2.loc[:, '芝麻租物免审出库占比'] = df_ms_new2['芝麻租物免审出库'] / df_ms_new2['芝麻租物出库']
df_ms_new2.loc[:, '芝麻租物免审出库占比'] = df_ms_new2['芝麻租物免审出库占比'].apply(lambda x: format(x, '.2%'))
df_ms_new2.loc[:, '搜索免审出库占比'] = df_ms_new2['搜索免审出库'] / df_ss_group_new['出库（按下单时间）']
df_ms_new2.loc[:, '搜索免审出库占比'] = df_ms_new2['搜索免审出库占比'].apply(lambda x: format(x, '.2%'))
df_ms_new2.loc[:, '机审通过'] = df_all_new['机审通过件']
df_ms_new2.loc[:, '非免审进件'] = df_ms_new2['机审通过'] - df_ms_new2['免审进件']
df_ms_new2.loc[:, '非免审出库'] = df_ms_new2['总体出库'] - df_ms_new2['免审出库']
df_ms_new2.loc[:, '非免审转化率'] = df_ms_new2['非免审出库'] / df_ms_new2['非免审进件']
df_ms_new2.loc[:, '非免审转化率'] = df_ms_new2['非免审转化率'].apply(lambda x: format(x, '.2%'))
df_ms_new3 = pd.concat([df_ms_new2, df_ms_group_new], axis=1)
# df_ms_new3.set_index('月份')
df_ms_new3.tail()

,月份,下单日期,总体进件,芝麻租物进件,免审进件,芝麻租物免审进件,搜索渠道免审进件,免审进件占比,芝麻租物免审进件占比,搜索免审进件占比,总体出库,芝麻租物出库,免审出库,芝麻租物免审出库,搜索免审出库,免审转化率,免审出库占比,芝麻租物免审出库占比,搜索免审出库占比,机审通过,非免审进件,非免审出库,非免审转化率,下单日期,进件数,人审拒绝,客户取消,出库,待审核,进件出库率,取消率,人审拒绝率
11,2025-10,2025-10-20,1995,395,515,207,166,25.81%,52.41%,29.43%,178,93,172,87.0,47.0,33.40%,96.63%,93.55%,100.00%,567.0,52.0,6,11.54%,2025-10-20,515,240.0,24,172,1,33.40%,4.66%,46.60%
12,2025-10,2025-10-21,1873,404,473,218,127,25.25%,53.96%,23.48%,151,73,148,72.0,32.0,31.29%,98.01%,98.63%,100.00%,518.0,45.0,3,6.67%,2025-10-21,473,240.0,14,148,7,31.29%,2.96%,50.74%
13,2025-10,2025-10-22,1750,369,453,202,115,25.89%,54.74%,25.50%,142,73,137,69.0,35.0,30.24%,96.48%,94.52%,100.00%,495.0,42.0,5,11.90%,2025-10-22,453,215.0,18,137,22,30.24%,3.97%,47.46%
14,2025-10,2025-10-23,1724,356,456,202,112,26.45%,56.74%,28.07%,126,58,126,56.0,31.0,27.63%,100.00%,96.55%,100.00%,517.0,61.0,0,0.00%,2025-10-23,456,184.0,22,126,94,27.63%,4.82%,40.35%
15,2025-10,2025-10-24,166,58,53,28,10,31.93%,48.28%,28.57%,0,0,0,0.0,0.0,0.00%,nan%,nan%,nan%,54.0,1.0,0,0.00%,2025-10-24,53,7.0,0,0,42,0.00%,0.00%,13.21%


## 免审订单转化

In [146]:
# 免审
df_ms1_o = df[(df.is_vip=='1')&(df.status_result=='0')]
df_ms2_o = df2[(df2.is_vip=='1')&(df2.status_result=='0')]
df_ms_o_group = all_models.data_group(df_ms1_o, df_ms2_o, df_risk_examine, '下单日期')
df_ms_o_group = df_ms_o_group[["进件数","人审拒绝","客户取消","出库","待审核","进件出库率","取消率","人审拒绝率",'出库前风控强拒','出库前强拒比例','无法联系','无法联系占比']]
df_ms_o_group_new = df_ms_o_group.reset_index()
df_ms_o_group_new.insert(0, '月份', df_ms_o_group_new['下单日期'].astype(str).str.split('-').str[0]+'-'+df_ms_o_group_new['下单日期'].astype(str).str.split('-').str[1])
# df_ms_o_group_new#.set_index('月份')

#  免押订单转化率 全免押

In [147]:
# 预授权
sql_ysq = '''
    select om.create_time, tmu.id_card_num,top.order_id,om.status
    ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.result') end,'"','') as result 
    from db_digua_business.t_order_pay top
    left join db_digua_business.t_order om on top.order_id = om.id
    left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
    left join db_digua_business.t_order_risk tor on om.id = tor.order_id 
    where top.pay_type = 'ZFBYSQ' 
    -- and  DATE_FORMAT(om.create_time, '%Y-%m-%d')>='2025-09-01'
    -- and  DATE_FORMAT(om.create_time, '%Y-%m-%d')<='2025-09-26'
    and om.create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -15 day )
    and om.user_mobile is not null 
    # and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
    # ,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静','陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静', '李珍珍')   
    -- and top.total_freeze_fund_amount is not null
    -- and trade_no is not null
    '''
df_ysq = clean.query(sql_ysq)
df_ysq.shape

(138736, 5)

In [148]:
df_ysq_t = df_ysq.copy()
df_ysq_t["下单日期"]=df_ysq_t["create_time"].dt.date
df_ysq_t["下单日期"]=pd.to_datetime(df_ysq_t["下单日期"],errors="coerce")
df_ysq_t['是否前置拦截'] = np.where((df_ysq_t.result.str.contains('id_card不得为空') )|(df_ysq_t.result.str.contains('mobile校验不通过')) |(df_ysq_t.result.str.contains('name校验不通过'))
                        |(df_ysq_t.result.str.contains('年龄超过49岁或低于18岁'))|(df_ysq_t.result=='风控未通过')|(df_ysq_t.result=='黑名单用户'),1,0)
df_ysq_t.loc[:,"进件"]=np.where((df_ysq_t["status"]==1)|(df_ysq_t["status"]==13),"未进件","进件")
# 去除是否前置拦截=1且进件=未进件的行
df_ans = df_ysq_t[(df_ysq_t['是否前置拦截']==0)|(df_ysq_t['进件']=='进件')]
# 按下单日期与id_card_num去重
df_ans = df_ans.drop_duplicates(subset=["下单日期","id_card_num"],keep="first")
# 按下单日期分组计算订单数
df_ans = df_ans.groupby(["下单日期"]).agg({"order_id": 'count'}).rename(columns={"order_id": "预授权调起人数"}).reset_index()
# df_ans

In [149]:
# 预授权免押
sql_ysqmy = '''
    select om.create_time, tmu.id_card_num,top.order_id
    from db_digua_business.t_order_pay top
    left join db_digua_business.t_order om on top.order_id = om.id
    left join db_digua_business.t_member_user tmu on om.user_id = tmu.id 
    where top.pay_type = 'ZFBYSQ' 
    -- and  DATE_FORMAT(om.create_time, '%Y-%m-%d')>='2025-09-01'
    -- and  DATE_FORMAT(om.create_time, '%Y-%m-%d')<='2025-09-26'
    and om.create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -15 day )
    and om.user_mobile is not null 
    # and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
    # ,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静','陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静', '李珍珍')   
    and top.total_freeze_fund_amount = 0
    and trade_no is not null
    and top.status in (2, 5)
    '''

df_ysqmy = clean.query(sql_ysqmy)
df_ysqmy["下单日期"]=df_ysqmy["create_time"].dt.date
df_ysqmy["下单日期"]=pd.to_datetime(df_ysqmy["下单日期"],errors="coerce")
df_ysqmy = df_ysqmy.drop_duplicates(subset=["下单日期","id_card_num"],keep="first")
df_ysqmy = df_ysqmy.groupby(["下单日期"]).agg({"order_id": 'count'}).rename(columns={"order_id": "全免押"}).reset_index()
# df_ysqmy

In [150]:
df_qmy = df[df['押金类型']=='全免押']
df_qmy2 = df2[df2['押金类型']=='全免押']
df_qmy_group = all_models.data_group(df_qmy, df_qmy2, df_risk_examine, '下单日期')
df_qmy_group['出库（按下单时间）'] = df_my['全免押'][:-1]
df_qmy_group = df_qmy_group[["进件数","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","待审核", '出库（按下单时间）', '进件出库率', '取消率', '人审转化率', '人审拒绝率']].fillna(0)
df_qmy_group_new = df_qmy_group.reset_index()
df_qmy_group_new.insert(1, '总进件', df_all_new['进件数'])
# df_qmy_group_new.insert(2, '免押进件预授权占比', (df_ysqmy.全免押/df_ans.预授权调起人数).map(lambda x: format(x, '.2%')))
df_qmy_group_new.insert(2, '免押进件预授权占比', (df_ysqmy.全免押 / df_ans.预授权调起人数).apply(lambda x: format(x, '.2%') if pd.notna(x) else None))
# df_qmy_group_new.set_index('下单日期')
df_qmy_group_new.tail()

,下单日期,总进件,免押进件预授权占比,进件数,机审强拒,强拒比例,机审通过件,人审拒绝,风控通过件,风控通过率,客户取消,待审核,出库（按下单时间）,进件出库率,取消率,人审转化率,人审拒绝率
11,2025-10-20,1995,27.22%,1884,1336,70.91%,567.0,299,268.0,14.23%,25,1,179.0,9.45%,1.33%,31.39%,15.87%
12,2025-10-21,1873,27.32%,1759,1280,72.77%,516.0,294,222.0,12.62%,15,9,150.0,8.58%,0.85%,29.26%,16.71%
13,2025-10-22,1750,29.42%,1678,1222,72.82%,493.0,266,227.0,13.53%,20,22,143.0,8.46%,1.19%,28.80%,15.85%
14,2025-10-23,1724,25.93%,1660,1179,71.02%,517.0,245,272.0,16.39%,24,105,124.0,7.59%,1.45%,24.37%,14.76%
15,2025-10-24,166,21.39%,155,105,67.74%,54.0,9,45.0,29.03%,0,45,0.0,0.00%,0.00%,0.00%,5.81%


# 非免押（部分免押）

In [151]:
# 预授权非免押
sql_ysqfmy = '''
    select om.create_time, tmu.id_card_num,top.order_id
    from db_digua_business.t_order_pay top
    left join db_digua_business.t_order om on top.order_id = om.id
    left join db_digua_business.t_member_user tmu on om.user_id = tmu.id 
    where top.pay_type = 'ZFBYSQ' 
    -- and  DATE_FORMAT(om.create_time, '%Y-%m-%d')>='2025-09-01'
    -- and  DATE_FORMAT(om.create_time, '%Y-%m-%d')<='2025-09-26'
    and om.create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -15 day )
    and om.user_mobile is not null 
    # and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
    # ,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静','陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静', '李珍珍')   
    and top.total_freeze_fund_amount > 0
    and trade_no is not null
    and top.status in (2, 5)
    '''
df_ysqfmy = clean.query(sql_ysqfmy)

df_ysqfmy["下单日期"]=df_ysqfmy["create_time"].dt.date
df_ysqfmy["下单日期"]=pd.to_datetime(df_ysqfmy["下单日期"],errors="coerce")
df_ysqfmy = df_ysqfmy.drop_duplicates(subset=["下单日期","id_card_num"],keep="first")
df_ysqfmy = df_ysqfmy.groupby(["下单日期"]).agg({"order_id": 'count'}).rename(columns={"order_id": "部分免押"}).reset_index()
# df_ysqfmy

In [152]:
df_fmy = df[df['押金类型']=='部分免押']
df_fmy2 = df2[df2['押金类型']=='部分免押']
df_fmy_group = all_models.data_group(df_fmy, df_fmy2, df_risk_examine, '下单日期')
df_fmy_group['待审核'] = 0
df_fmy_group = df_fmy_group[["进件数","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消", '待审核']]
try:
    df_fmy_group.loc[:, '出库'] = df_my['部分免押'][:-1]
except:
    df_fmy_group.loc[:, '出库'] = 0
df_fmy_group_new = df_fmy_group.reset_index()
df_fmy_group_new.insert(1, '总进件', df_all_new['进件数'])
df_fmy_group_new.insert(2, '非免押进件预授权占比', (df_ysqfmy.部分免押/df_ans.预授权调起人数).map(lambda x: format(x, '.2%')))
df_fmy_group_new.loc[:, '进件出库率'] = (df_fmy_group_new.出库/df_fmy_group_new.进件数).map(lambda x: format(x, '.2%'))
df_fmy_group_new.loc[:, '取消比例'] = (df_fmy_group_new.客户取消/df_fmy_group_new.进件数).map(lambda x: format(x, '.2%'))
df_fmy_group_new.loc[:, '人审转化率'] = (df_fmy_group_new.出库/df_fmy_group_new.机审通过件).map(lambda x: format(x, '.2%'))
df_fmy_group_new.loc[:, '人审拒绝率'] = (df_fmy_group_new.人审拒绝/df_fmy_group_new.进件数).map(lambda x: format(x, '.2%'))
df_fmy_group_new.tail()
# df_fmy_group_new#.set_index('下单日期')

,下单日期,总进件,非免押进件预授权占比,进件数,机审强拒,强拒比例,机审通过件,人审拒绝,风控通过件,风控通过率,客户取消,待审核,出库,进件出库率,取消比例,人审转化率,人审拒绝率
11,2025-10-20,1995,2.67%,111,111.0,100.00%,0.0,0,0.0,0.00%,0,0,0.0,0.00%,0.00%,nan%,0.00%
12,2025-10-21,1873,2.97%,114,112.0,98.25%,2.0,1,1.0,0.88%,0,0,0.0,0.00%,0.00%,0.00%,0.88%
13,2025-10-22,1750,2.09%,72,71.0,98.61%,2.0,1,1.0,1.39%,0,0,0.0,0.00%,0.00%,0.00%,1.39%
14,2025-10-23,1724,1.94%,64,64.0,100.00%,0.0,0,0.0,0.00%,0,0,0.0,0.00%,0.00%,nan%,0.00%
15,2025-10-24,166,2.57%,11,11.0,100.00%,0.0,0,0.0,0.00%,0,0,NaN,nan%,0.00%,nan%,0.00%


In [153]:
# 将df_qmy_group_new，df_fmy_group_new写入excel中
# path = 'F:/需求/倩姐需求/'
# with pd.ExcelWriter(path + '预授权.xlsx') as writer:
#     df_qmy_group_new.to_excel(writer, sheet_name='免押', startrow=0, index=True)
#     df_fmy_group_new.to_excel(writer, sheet_name='非免押', startrow=0, index=True)

# 取消拒绝原因

In [271]:
# 惜仪需求
path_result = 'F:/myfile/数据提取及分析/'
excel_name = f'取消&拒绝原因_{t_date}.xlsx'
dfms = df2
# 取消原因 总体
qx_zt_df = pd.crosstab(dfms["下单日期"],dfms["取消原因"],margins=True)

# 拒绝原因总体
jj_zt_df = pd.crosstab(dfms["下单日期"],dfms["电审拒绝原因"],margins=True)
#免审进件的电审拒绝原因
dfms=df2[(df2["decision_result"].str.contains(pat="免人审",regex=False)==True)&(df2.is_vip=='1')&(df2.status_result=='0')]
jj_ms_df = pd.crosstab(dfms["下单日期"],dfms["电审拒绝原因"],margins=True)
#免审进件的取消原因
dfms=df2[(df2["decision_result"].str.contains(pat="免人审",regex=False)==True)&(df2.is_vip=='1')&(df2.status_result=='0')]
qx_ms_df = pd.crosstab(dfms["下单日期"],dfms["取消原因"],margins=True)

with pd.ExcelWriter(path_result + excel_name, engine='xlsxwriter') as writer:  
    qx_zt_df.to_excel(writer, sheet_name='取消原因-总体', startrow=0, index=True)
    jj_zt_df.to_excel(writer, sheet_name='拒绝原因-总体', startrow=0, index=True) 
    jj_ms_df.to_excel(writer, sheet_name='拒绝原因-免审', startrow=0, index=True) 
    qx_ms_df.to_excel(writer, sheet_name='取消原因-免审', startrow=0, index=True) 

## 取消原因2&拒绝理由

In [155]:
# from datetime import datetime, timedelta
# # 获取昨日时间并转化为字符串
# from datetime import datetime  

# df2['取消原因2'] = df2['cancel_reason'].str.split('：')
# df2['取消原因2'] = df2['取消原因2'].apply(lambda x: x[-1] if x is not None else x)
# y_date = datetime.today() - timedelta(days=1)

# y_date = y_date.strftime('%Y-%m-%d')
# df2_new = df2[(df2['下单日期'] == y_date)&(df2['拒绝理由'].notna())]
# # 切割拒绝理由内容，取第一个逗号前的内容
# # df2_new['拒绝理由'] = df2_new['拒绝理由'].str.split(',').str[0]
# df_quju = df2_new
# # 取消原因_总体
# qx_zt_df = pd.crosstab(df_quju["下单日期"],df_quju["取消原因2"],margins=False)
# # 拒绝理由_总体
# jj_zt_df = pd.crosstab(df_quju["下单日期"],df_quju["拒绝理由"],margins=False)

# # jj_zt_df[jj_zt_df['下单日期'] == '2025-07-15']

# qx_zt_df
# df_quju.groupby('取消原因2').agg({'order_id': 'count'}).sort_values(by='order_id', ascending=False).rename(columns={'order_id': '订单数'})


# 商家转化数据

In [156]:
df_contain["待审核"] = np.where(df_contain["审核状态"]=='待审核',1,0)
df_contain["前置拦截"] = np.where(df_contain["审核状态"]=='前置拦截',1,0)
df_contain["机审强拒"] = np.where(df_contain["审核状态"]=='机审强拒',1,0)
df_contain["人审拒绝"] = np.where(df_contain["审核状态"]=='人审拒绝',1,0)
df_contain["客户取消"] = np.where(df_contain["审核状态"]=='客户取消',1,0)
df_contain["出库前风控强拒"] = np.where(df_contain["审核状态"]=='出库前风控强拒',1,0)
df_contain["无法联系"] = np.where(df_contain["审核状态"]=='无法联系',1,0)
df_contain["是否进件"] = np.where(df_contain["进件"]=='进件',1,0)
df_contain["是否出库"] = np.where(df_contain["status"].isin([2,3,4,5,6,8,15]),1,0)

df_contain["进件前取消"] = np.where(df_contain["审核状态"]=='进件前取消',1,0)

## 澄心优租

In [157]:
# 澄心优租
cxyz = all_models.merchant_names(df_contain, '澄心优租', '下单日期')
cxyz = cxyz[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","订单出库率"]]
cxyz = clean.continuous_dates(cxyz)
cxyz_new = cxyz.reset_index(names=['下单日期'])

# try:
#     cxyz = all_models.merchant_names(df_contain, '澄心优租', '下单日期')
#     if cxyz.empty:
#         print("warn: 没有数据")
#         cxyz_new = pd.DataFrame(columns=['去重订单数','前置拦截','拦截率','进件数','预授权通过率','机审通过件','人审拒绝','人审拒绝率','待审核','出库','进件出库率','订单出库率'])
#     else:
#         cxyz = cxyz[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","订单出库率"]]
#         cxyz = clean.continuous_dates(cxyz)
#         cxyz_new = cxyz.reset_index(names=['下单日期'])
# except:
#     cxyz_new = pd.DataFrame(columns=['去重订单数','前置拦截','拦截率','进件数','预授权通过率','机审通过件','人审拒绝','人审拒绝率','待审核','出库','进件出库率','订单出库率'])
cxyz_new.tail()

,下单日期,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审通过件,人审拒绝,人审拒绝率,待审核,出库,进件出库率,订单出库率
11,2025-10-20,2.0,0.0,0.00%,1.0,50.00%,1.0,1.0,100.00%,0.0,0.0,0.00%,0.00%
12,2025-10-21,2.0,0.0,0.00%,0.0,0.00%,0.0,0.0,nan%,0.0,0.0,nan%,0.00%
13,2025-10-22,4.0,0.0,0.00%,1.0,25.00%,1.0,1.0,100.00%,0.0,0.0,0.00%,0.00%
14,2025-10-23,2.0,0.0,0.00%,0.0,0.00%,0.0,0.0,nan%,0.0,0.0,nan%,0.00%
15,2025-10-24,2.0,0.0,0.00%,0.0,0.00%,0.0,0.0,nan%,0.0,0.0,nan%,0.00%


## 优优2店

In [158]:
# 优优2店
# yy2 = all_models.merchant_names(df_contain,'优优2店', '下单日期')
# yy2 = yy2[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
# yy2


## 北京海鸟窝科技有限公司

In [159]:
# 北京海鸟窝科技有限公司
hnw = all_models.merchant_names(df_contain,'北京海鸟窝科技有限公司', '下单日期')
hnw = hnw[["去重订单数","进件数","预授权通过率","出库","进件出库率"]]
hnw = clean.continuous_dates(hnw)
hnw_new = hnw.reset_index(names=['下单日期'])
# 修改后更具有健壮性
# try:
#     hnw = all_models.merchant_names(df_contain, '北京海鸟窝科技有限公司', '下单日期')
#     if hnw.empty:
#         print("warn: 没有数据")
#         hnw_new = pd.DataFrame(columns=['去重订单数','进件数','预授权通过率','出库','进件出库率'])
#     else:
#         hnw = hnw[["去重订单数","进件数","预授权通过率","出库","进件出库率"]]
#         hnw = clean.continuous_dates(hnw)
#         hnw_new = hnw.reset_index(names=['下单日期'])
# except:
#     hnw_new = pd.DataFrame(columns=['去重订单数','进件数','预授权通过率','出库','进件出库率'])
hnw_new.tail()


,下单日期,去重订单数,进件数,预授权通过率,出库,进件出库率
11,2025-10-20,0.0,0.0,0,0.0,0
12,2025-10-21,0.0,0.0,0,0.0,0
13,2025-10-22,0.0,0.0,0,0.0,0
14,2025-10-23,1.0,1.0,100.00%,0.0,0.00%
15,2025-10-24,0.0,0.0,0,0.0,0


## 租着用电脑数码

In [160]:
# 租着用电脑数码
# zzy = all_models.merchant_names(df_contain,'租着用电脑数码', '下单日期')
# zzy = zzy[["去重订单数","进件数","预授权通过率","出库","进件出库率"]]
# zzy = clean.continuous_dates(zzy)
# zzy_new = zzy.reset_index(names=['下单日期'])

try:
    zzy = all_models.merchant_names(df_contain, '租着用电脑数码', '下单日期')
    if zzy.empty:
        print("warn: 没有数据")
        zzy_new = pd.DataFrame(columns=['去重订单数','进件数','预授权通过率','出库','进件出库率'])
    else:
        zzy = zzy[["去重订单数","进件数","预授权通过率","出库","进件出库率"]]
        zzy = clean.continuous_dates(zzy)
        zzy_new = zzy.reset_index(names=['下单日期'])
except:
    zzy_new = pd.DataFrame(columns=['去重订单数','进件数','预授权通过率','出库','进件出库率'])
zzy_new.tail()

,下单日期,去重订单数,进件数,预授权通过率,出库,进件出库率
11,2025-10-20,10,2,20.00%,0,0.00%
12,2025-10-21,6,0,0.00%,0,nan%
13,2025-10-22,4,0,0.00%,0,nan%
14,2025-10-23,5,3,60.00%,0,0.00%
15,2025-10-24,2,0,0.00%,0,nan%


## 格木木二奢名品

In [161]:
# 格木木二奢名品
gmm = all_models.merchant_names(df_contain,'格木木二奢名品', '下单日期')
gmm[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]


,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审通过件,客户取消,人审拒绝,人审拒绝率,待审核,出库,进件出库率,取消率,订单出库率
下单日期,,,,,,,,,,,,,,


## 广州康基贸易有限公司

In [162]:
# 广州康基贸易有限公司
kjmy = all_models.merchant_names(df_contain,'广州康基贸易有限公司', '下单日期')
kjmy[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]


,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审通过件,客户取消,人审拒绝,人审拒绝率,待审核,出库,进件出库率,取消率,订单出库率
下单日期,,,,,,,,,,,,,,


## 趣智数码

In [163]:
# 趣智数码
# qzsm = all_models.merchant_names(df_contain,'趣智数码', '下单日期')
# qzsm = qzsm[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
# qzsm = clean.continuous_dates(qzsm)
# qzsm_new = qzsm.reset_index(names=['下单日期'])

try:
    qzsm = all_models.merchant_names(df_contain, '趣智数码', '下单日期')
    if qzsm.empty:
        print("warn: 没有数据")
        qzsm_new = pd.DataFrame(columns=['去重订单数','前置拦截','拦截率','进件数','预授权通过率','机审通过件','客户取消','人审拒绝','人审拒绝率','待审核','出库','进件出库率','取消率','订单出库率'])
    else:
        qzsm = qzsm[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
        qzsm = clean.continuous_dates(qzsm)
        qzsm_new = qzsm.reset_index(names=['下单日期'])
except:
    qzsm_new = pd.DataFrame(columns=['去重订单数','前置拦截','拦截率','进件数','预授权通过率','机审通过件','客户取消','人审拒绝','人审拒绝率','待审核','出库','进件出库率','取消率','订单出库率'])
qzsm_new.tail()


,下单日期,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审通过件,客户取消,人审拒绝,人审拒绝率,待审核,出库,进件出库率,取消率,订单出库率
11,2025-10-20,2.0,0.0,0.00%,0.0,0.00%,0.0,0.0,0.0,nan%,0.0,0.0,nan%,nan%,0.00%
12,2025-10-21,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,0
13,2025-10-22,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,0
14,2025-10-23,3.0,0.0,0.00%,1.0,33.33%,1.0,0.0,0.0,0.00%,0.0,1.0,100.00%,0.00%,33.33%
15,2025-10-24,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,0


## 汇客好租

In [164]:
# # 汇客好租
# hkhz = all_models.merchant_names(df_contain,'汇客好租', '下单日期')
# hkhz = hkhz[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
# hkhz = clean.continuous_dates(hkhz)
# hkhz_new = hkhz.reset_index(names=['下单日期'])
# # 电脑
# df_contain_dy = df_contain[(df_contain.归属渠道=='抖音渠道')&(df_contain.classify_id.isin([125,126,127,185]))]
# hkhz_dy = all_models.merchant_names(df_contain_dy,'汇客好租', '下单日期')
# hkhz_dy = hkhz_dy[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
# hkhz_dy = clean.continuous_dates(hkhz_dy)
# hkhz_dy_new = hkhz_dy.reset_index(names=['下单日期'])


try:
    # 汇客好租
    hkhz = all_models.merchant_names(df_contain, '汇客好租', '下单日期')
    if hkhz.empty:
        print("warn: 没有数据")
        hkhz_new = pd.DataFrame(columns=['去重订单数','前置拦截','拦截率','进件数','预授权通过率','机审通过件','客户取消','人审拒绝','人审拒绝率','待审核','出库','进件出库率','取消率','订单出库率'])
    else:
        hkhz = hkhz[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
        hkhz = clean.continuous_dates(hkhz)
        hkhz_new = hkhz.reset_index(names=['下单日期'])
except:
    hkhz_new = pd.DataFrame(columns=['去重订单数','前置拦截','拦截率','进件数','预授权通过率','机审通过件','客户取消','人审拒绝','人审拒绝率','待审核','出库','进件出库率','取消率','订单出库率'])

try:
    # 电脑
    df_contain_dy = df_contain[(df_contain.归属渠道=='抖音渠道')&(df_contain.classify_id.isin([125,126,127,185]))]
    hkhz_dy = all_models.merchant_names(df_contain_dy, '汇客好租', '下单日期')
    if hkhz_dy.empty:
        print("warn: 没有数据")
        hkhz_dy_new = pd.DataFrame(columns=['去重订单数','前置拦截','拦截率','进件数','预授权通过率','机审通过件','客户取消','人审拒绝','人审拒绝率','待审核','出库','进件出库率','取消率','订单出库率'])
    else:
        hkhz_dy = hkhz_dy[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
        hkhz_dy = clean.continuous_dates(hkhz_dy)
        hkhz_dy_new = hkhz_dy.reset_index(names=['下单日期'])
except:
    hkhz_dy_new = pd.DataFrame(columns=['去重订单数','前置拦截','拦截率','进件数','预授权通过率','机审通过件','客户取消','人审拒绝','人审拒绝率','待审核','出库','进件出库率','取消率','订单出库率'])

hkhz_new.tail()

warn: 没有数据


,下单日期,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审通过件,客户取消,人审拒绝,人审拒绝率,待审核,出库,进件出库率,取消率,订单出库率
11,2025-10-20,4,0,0.00%,1,25.00%,1,0,1,100.00%,0,0,0.00%,0.00%,0.00%
12,2025-10-21,7,0,0.00%,1,14.29%,1,0,0,0.00%,0,1,100.00%,0.00%,14.29%
13,2025-10-22,8,0,0.00%,1,12.50%,1,0,0,0.00%,1,0,0.00%,0.00%,0.00%
14,2025-10-23,2,0,0.00%,0,0.00%,0,0,0,nan%,0,0,nan%,nan%,0.00%
15,2025-10-24,1,0,0.00%,1,100.00%,1,0,0,0.00%,1,0,0.00%,0.00%,0.00%


In [165]:
hkhz_dy_new

,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审通过件,客户取消,人审拒绝,人审拒绝率,待审核,出库,进件出库率,取消率,订单出库率


In [166]:
# df_contain.merchant_name.value_counts()
# hkhz_yf = all_models.merchant_names(df_contain, '汇客好租', '下单月份')
# hkhz_yf
# df_contain[df_contain.merchant_name=='汇客好租'].shape
# hkhz = all_models.merchant_names(df_contain, '汇客好租', '下单日期')
# hkhz

### 临时需求：获取抖音订单的已发货客户信息

In [167]:
# # 抖音订单：order_type = 'DY_ORDER'
# # 已发货 status notin (1,2,10,12,13)其中4为租赁中
# df_hkhz_dy = df_contain[(df_contain.merchant_name=='汇客好租')&(df_contain.order_type=='DY_ORDER')]
# # 剔除status为1,2,10,12,13的订单
# df_hkhz_dy = df_hkhz_dy[~df_hkhz_dy.status.isin([1,2,10,12,13])]
# df_hkhz_dy.loc[:, '订单状态'] = np.where(df_hkhz_dy.status==4, '租赁中', '已发货')
# df_hkhz_dy.loc[:, '是否逾期'] = np.where((df_hkhz_dy.overdue==1)&(df_hkhz_dy.status==4), '逾期', '未逾期')
# df_hkhz_dy.loc[:, '订单类型'] = np.where(df_hkhz_dy.order_type=='DY_ORDER', '抖音订单', '非抖音订单')
# # df_hkhz_dy_new = df_hkhz_dy[['merchant_name','订单类型','order_number', '订单状态', 'true_name', 'user_mobile', 'id_card_num', '是否逾期', '下单日期']].rename(columns={'merchant_name': '商户名称', 'order_number': '订单号', 'true_name': '姓名', 'user_mobile': '手机号', 'id_card_num': '身份证号'})
# # df_hkhz_dy_new.to_excel(f'F:/需求/倩姐需求/汇客好租抖音订单_{t_date}.xlsx', index=False)

In [168]:
# df_hkhz = df_contain[(df_contain.merchant_name=='汇客好租')]
# # 获取租赁中且逾期的订单
# df_hkhz_yq = df_hkhz[(df_hkhz.status==4)&(df_hkhz.overdue==1)]
# df_hkhz_yq.loc[:,'逾期金额'] = df_hkhz_yq.new_actual_money - df_hkhz_yq.已收租金 - df_hkhz_yq.已付买断尾款
# # 按下单月份分组统计逾期订单数和逾期金额
# df_hkhz_yq_group = df_hkhz_yq.groupby(df_hkhz_yq['下单月份']).agg({'order_number': 'count', '逾期金额': 'sum'}).rename(columns={'order_number': '逾期订单数'}).reset_index()
# df_hkhz_yq_group#.to_excel(f'F:/需求/倩姐需求/汇客好租逾期订单_{t_date}.xlsx', index=False)
# df_hkhz_yq[['order_number','下单日期','new_actual_money','已收租金','已付买断尾款','逾期金额']]
# df_hkhz_yq['new_actual_money'].mean()

## 小蚂蚁租机

In [169]:
# 小蚂蚁租机
xmy = all_models.merchant_names(df_contain,'小蚂蚁租机', '下单日期')
xmy = xmy[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
if len(xmy)!=0:
    xmy = clean.continuous_dates(xmy)
xmy_new = xmy.reset_index(names=['下单日期'])


## 乙辉数码

In [170]:
# 乙辉数码
yhsm = all_models.merchant_names(df_contain,'乙辉数码', '下单日期')
yhsm = yhsm[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
# yhsm = clean.continuous_dates(yhsm)
yhsm_new = yhsm.reset_index()
yhsm_new.dtypes


下单日期      datetime64[ns]
去重订单数              int64
前置拦截             float64
拦截率              float64
进件数              float64
预授权通过率           float64
机审通过件            float64
客户取消             float64
人审拒绝             float64
人审拒绝率            float64
待审核              float64
出库               float64
进件出库率            float64
取消率              float64
订单出库率            float64
dtype: object

## 兴鑫兴通讯

In [171]:
# 兴鑫兴通讯
xxx = all_models.merchant_names(df_contain,'兴鑫兴通讯', '下单日期')
xxx = xxx[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
xxx_new = xxx.reset_index()
# xxx_new

## 呱子笔记本电脑

In [172]:
# 呱子笔记本电脑
# gz = all_models.merchant_names(df_contain,'呱子笔记本电脑', '下单日期')
# gz = gz[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
# gz = clean.continuous_dates(gz)
# gz_new = gz.reset_index(names=['下单日期'])
# gz_new.set_index('下单日期')

## 南京聚格网络科技

In [173]:
jgwl = all_models.merchant_names(df_contain,'南京聚格网络科技', '下单日期')
jgwl = jgwl[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
try:
    jgwl = clean.continuous_dates(jgwl)
except:
    print("warn: 没有数据")
jgwl_new = jgwl.reset_index(names=['下单日期'])
# jgwl#_new#.set_index('下单日期')

warn: 没有数据


In [174]:
# df_contain['merchant_name'].unique()


## 人人享租

In [175]:
# rrxz = all_models.merchant_names(df_contain,'人人享租', '下单日期')
# rrxz = rrxz[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
# # rrxz = clean.continuous_dates(rrxz)
# rrxz_new = rrxz.reset_index(names=['下单日期'])
# # rrxz_new

try:
    rrxz = all_models.merchant_names(df_contain, '人人享租', '下单日期')
    if rrxz.empty:
        print("warn: 没有数据")
        rrxz_new = pd.DataFrame(columns=['去重订单数','前置拦截','拦截率','进件数','预授权通过率','机审通过件','客户取消','人审拒绝','人审拒绝率','待审核','出库','进件出库率','取消率','订单出库率'])
    else:
        rrxz = rrxz[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
        rrxz = clean.continuous_dates(rrxz)
        rrxz_new = rrxz.reset_index(names=['下单日期'])
except:
    rrxz_new = pd.DataFrame(columns=['去重订单数','前置拦截','拦截率','进件数','预授权通过率','机审通过件','客户取消','人审拒绝','人审拒绝率','待审核','出库','进件出库率','取消率','订单出库率'])
rrxz_new#.tail()

,下单日期,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审通过件,客户取消,人审拒绝,人审拒绝率,待审核,出库,进件出库率,取消率,订单出库率
0,2025-10-09,119.0,0.0,0.00%,114.0,95.80%,24.0,0.0,21.0,18.42%,0.0,8.0,7.02%,0.00%,6.72%
1,2025-10-10,108.0,0.0,0.00%,108.0,100.00%,10.0,0.0,10.0,9.26%,0.0,3.0,2.78%,0.00%,2.78%
2,2025-10-11,121.0,0.0,0.00%,119.0,98.35%,10.0,0.0,10.0,8.40%,0.0,0.0,0.00%,0.00%,0.00%
3,2025-10-12,96.0,0.0,0.00%,96.0,100.00%,10.0,0.0,9.0,9.38%,0.0,2.0,2.08%,0.00%,2.08%
4,2025-10-13,105.0,0.0,0.00%,105.0,100.00%,7.0,0.0,7.0,6.67%,0.0,0.0,0.00%,0.00%,0.00%
5,2025-10-14,109.0,0.0,0.00%,104.0,95.41%,8.0,0.0,8.0,7.69%,0.0,3.0,2.88%,0.00%,2.75%
6,2025-10-15,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,0
7,2025-10-16,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,0
8,2025-10-17,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,0
9,2025-10-18,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,0


## 崇胜数码

In [176]:
# cssm = all_models.merchant_names(df_contain,'崇胜数码', '下单日期')
# cssm = cssm[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
# cssm_new = cssm.reset_index(names=['下单日期'])

## 喜卓灵租机

In [177]:

xzl = all_models.merchant_names(df_contain,'喜卓灵租机', '下单日期')
xzl = xzl[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
if len(xzl)!=0:
    xzl = clean.continuous_dates(xzl)
xzl_new = xzl.reset_index(names=['下单日期'])

# xzl_new

## 喜卓灵新租机

In [178]:
# xzl_news = all_models.merchant_names(df_contain,'喜卓灵新租机', '下单日期')
# xzl_news = xzl_news[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
# # xzl_new = clean.continuous_dates(xzl_new)
# xzl_news = xzl_news.reset_index(names=['下单日期'])
# xzl_news

try:
    xzl_news = all_models.merchant_names(df_contain, '喜卓灵新租机', '下单日期')
    if xzl_news.empty:
        print("warn: 没有数据")
        xzl_news = pd.DataFrame(columns=["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"])
    else:
        xzl_news = xzl_news[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
        # xzl_new = clean.continuous_dates(xzl_new)
        xzl_news = xzl_news.reset_index(names=['下单日期'])
except:
    xzl_news = pd.DataFrame(columns=["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"])
xzl_news.tail()

,下单日期,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审通过件,客户取消,人审拒绝,人审拒绝率,待审核,出库,进件出库率,取消率,订单出库率
11,2025-10-20,199,0,0.00%,199,100.00%,21,0,21,10.55%,0,5,2.51%,0.00%,2.51%
12,2025-10-21,208,0,0.00%,208,100.00%,24,0,23,11.06%,0,6,2.88%,0.00%,2.88%
13,2025-10-22,207,0,0.00%,207,100.00%,22,0,20,9.66%,0,9,4.35%,0.00%,4.35%
14,2025-10-23,182,0,0.00%,182,100.00%,32,0,22,12.09%,10,3,1.65%,0.00%,1.65%
15,2025-10-24,7,0,0.00%,7,100.00%,0,0,0,0.00%,0,0,0.00%,0.00%,0.00%


## 星晟数码

In [179]:
# # 入驻时间:2025-07-21 
# xcsm = all_models.merchant_names(df_contain,'星晟数码', '下单日期')
# xcsm = xcsm[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
# xcsm = clean.continuous_dates(xcsm)
# xcsm_new = xcsm.reset_index(names=['下单日期'])
# # xcsm_new

try:
    xcsm = all_models.merchant_names(df_contain, '星晟数码', '下单日期')
    if xcsm.empty:
        print("warn: 没有数据")
        xcsm_new = pd.DataFrame(columns=["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"])
    else:
        xcsm = xcsm[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
        xcsm = clean.continuous_dates(xcsm)
        xcsm_new = xcsm.reset_index(names=['下单日期'])
except:
    xcsm_new = pd.DataFrame(columns=["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"])
xcsm_new.tail()

,下单日期,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审通过件,客户取消,人审拒绝,人审拒绝率,待审核,出库,进件出库率,取消率,订单出库率
11,2025-10-20,307,0,0.00%,115,37.46%,111,5,38,33.04%,2,6,5.22%,4.35%,1.95%
12,2025-10-21,245,0,0.00%,79,32.24%,78,3,24,30.38%,9,4,5.06%,3.80%,1.63%
13,2025-10-22,214,0,0.00%,95,44.39%,95,1,32,33.68%,24,1,1.05%,1.05%,0.47%
14,2025-10-23,258,0,0.00%,103,39.92%,103,0,22,21.36%,53,2,1.94%,0.00%,0.78%
15,2025-10-24,69,0,0.00%,26,37.68%,26,0,0,0.00%,21,0,0.00%,0.00%,0.00%


### 支付宝订单

In [180]:
# df_contain['order_type'].value_counts()
df_contain_zfb = df_contain[df_contain['order_type'] == 'ZFB_ORDER'] 
xssm_zfb = all_models.merchant_names(df_contain_zfb,'星晟数码', '下单日期')
xssm_zfb = xssm_zfb[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
xssm_zfb = clean.continuous_dates(xssm_zfb)
xcsm_zfb_new = xssm_zfb.reset_index(names=['下单日期'])
xcsm_zfb_new.tail()

,下单日期,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审通过件,客户取消,人审拒绝,人审拒绝率,待审核,出库,进件出库率,取消率,订单出库率
11,2025-10-20,191,0,0.00%,14,7.33%,14,1,1,7.14%,0,1,7.14%,7.14%,0.52%
12,2025-10-21,174,0,0.00%,18,10.34%,18,1,1,5.56%,2,2,11.11%,5.56%,1.15%
13,2025-10-22,126,0,0.00%,17,13.49%,17,0,3,17.65%,2,0,0.00%,0.00%,0.00%
14,2025-10-23,159,0,0.00%,19,11.95%,19,0,3,15.79%,6,0,0.00%,0.00%,0.00%
15,2025-10-24,46,0,0.00%,5,10.87%,5,0,0,0.00%,2,0,0.00%,0.00%,0.00%


### 京东订单

In [181]:
# df_contain['order_type'].value_counts()
df_contain_jd = df_contain[df_contain['order_type'] == 'JD_ORDER'] 
xssm_jd = all_models.merchant_names(df_contain_jd,'星晟数码', '下单日期')
xssm_jd = xssm_jd[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
xssm_jd = clean.continuous_dates(xssm_jd)
xcsm_jd_new = xssm_jd.reset_index(names=['下单日期'])
xcsm_jd_new.tail()

,下单日期,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审通过件,客户取消,人审拒绝,人审拒绝率,待审核,出库,进件出库率,取消率,订单出库率
11,2025-10-20,116,0,0.00%,101,87.07%,97,4,37,36.63%,2,5,4.95%,3.96%,4.31%
12,2025-10-21,71,0,0.00%,61,85.92%,60,2,23,37.70%,7,2,3.28%,3.28%,2.82%
13,2025-10-22,88,0,0.00%,78,88.64%,78,1,29,37.18%,22,1,1.28%,1.28%,1.14%
14,2025-10-23,99,0,0.00%,84,84.85%,84,0,19,22.62%,47,2,2.38%,0.00%,2.02%
15,2025-10-24,23,0,0.00%,21,91.30%,21,0,0,0.00%,19,0,0.00%,0.00%,0.00%


## 蘑菇时间  

In [182]:
# # 入驻时间:2025-07-08
# mgsj = all_models.merchant_names(df_contain,'蘑菇时间', '下单日期')
# mgsj = mgsj[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
# mgsj = clean.continuous_dates(mgsj)
# mgsj_New = mgsj.reset_index(names=['下单日期'])
# # mgsj_New

try:
    mgsj = all_models.merchant_names(df_contain, '蘑菇时间', '下单日期')
    if mgsj.empty:
        print("warn: 没有数据")
        mgsj_New = pd.DataFrame(columns=["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"])
    else:
        mgsj = mgsj[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
        mgsj = clean.continuous_dates(mgsj)
        mgsj_New = mgsj.reset_index(names=['下单日期'])
except:
    mgsj_New = pd.DataFrame(columns=["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"])
mgsj_New.tail()

,下单日期,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审通过件,客户取消,人审拒绝,人审拒绝率,待审核,出库,进件出库率,取消率,订单出库率
11,2025-10-20,6.0,0.0,0.00%,1.0,16.67%,1.0,0.0,1.0,100.00%,0.0,0.0,0.00%,0.00%,0.00%
12,2025-10-21,2.0,0.0,0.00%,2.0,100.00%,2.0,0.0,1.0,50.00%,0.0,1.0,50.00%,0.00%,50.00%
13,2025-10-22,4.0,0.0,0.00%,1.0,25.00%,1.0,0.0,0.0,0.00%,0.0,0.0,0.00%,0.00%,0.00%
14,2025-10-23,7.0,0.0,0.00%,2.0,28.57%,2.0,0.0,2.0,100.00%,0.0,0.0,0.00%,0.00%,0.00%
15,2025-10-24,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,0


## 云启德曜

In [183]:
try:
    yqdy = all_models.merchant_names(df_contain, '云启德曜', '下单日期')
    if yqdy.empty:
        print("warn: 没有数据")
        yqdy_New = pd.DataFrame(columns=["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"])
    else:
        yqdy = yqdy[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
        yqdy = clean.continuous_dates(yqdy)
        yqdy_New = yqdy.reset_index(names=['下单日期'])
except:
    yqdy_New = pd.DataFrame(columns=["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"])
yqdy_New.tail()

,下单日期,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审通过件,客户取消,人审拒绝,人审拒绝率,待审核,出库,进件出库率,取消率,订单出库率
11,2025-10-20,93,0,0.00%,93,100.00%,22,0,22,23.66%,0,5,5.38%,0.00%,5.38%
12,2025-10-21,95,0,0.00%,95,100.00%,16,0,16,16.84%,0,3,3.16%,0.00%,3.16%
13,2025-10-22,104,0,0.00%,104,100.00%,15,0,15,14.42%,0,3,2.88%,0.00%,2.88%
14,2025-10-23,83,0,0.00%,81,97.59%,19,0,19,23.46%,0,0,0.00%,0.00%,0.00%
15,2025-10-24,7,0,0.00%,7,100.00%,0,0,0,0.00%,0,0,0.00%,0.00%,0.00%


## 艾欧尼亚

In [184]:
try:
    aony = all_models.merchant_names(df_contain, '艾欧尼亚数码', '下单日期')
    if aony.empty:
        print("warn: 没有数据")
        aony_New = pd.DataFrame(columns=["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"])
    else:
        aony = aony[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
        aony = clean.continuous_dates(aony)
        aony_New = aony.reset_index(names=['下单日期'])
except:
    aony_New = pd.DataFrame(columns=["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"])
aony_New.tail()

,下单日期,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审通过件,客户取消,人审拒绝,人审拒绝率,待审核,出库,进件出库率,取消率,订单出库率
11,2025-10-20,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,0
12,2025-10-21,1.0,0.0,0.00%,0.0,0.00%,0.0,0.0,0.0,nan%,0.0,0.0,nan%,nan%,0.00%
13,2025-10-22,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,0
14,2025-10-23,1.0,0.0,0.00%,0.0,0.00%,0.0,0.0,0.0,nan%,0.0,0.0,nan%,nan%,0.00%
15,2025-10-24,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,0


# 每日各渠道订单统计 导出

In [185]:
from datetime import datetime  
today = datetime.today() 
t_date = today.date().strftime('%Y-%m-%d')
t_date

'2025-10-24'

In [186]:
path = r'\\digua\迪瓜租机\20.日报数据相关输出/'
# 每日各渠道去重订单统计结果表格导出
# df = df[df.归属渠道=='搜索渠道']
gsqd_qcdd = pd.crosstab(df["下单日期"],df["归属渠道"],margins=True)
lyqd_qcdd = pd.crosstab(df["下单日期"],df["来源渠道"],margins=True)
hdmc_qcdd = pd.crosstab(df["下单日期"],df["activity_name"],margins=True)
#每日各渠道进件统计结果表格导出
# df_j = df_j[df_j.归属渠道=='搜索渠道']
gsqd_jj = pd.crosstab(df_j["下单日期"],df_j["归属渠道"],margins=True)
lyqd_jj = pd.crosstab(df_j["下单日期"],df_j["来源渠道"],margins=True)
hdmc_jj = pd.crosstab(df_j["下单日期"],df_j["activity_name"],margins=True)
# 各归属渠道 出库 导出
# dfck = dfck[dfck.归属渠道=='搜索渠道']
gsqd_ck = pd.crosstab(dfck["下单日期"],dfck["归属渠道"],margins=True)
lyqd_ck = pd.crosstab(dfck["下单日期"],dfck["来源渠道"],margins=True)
hdmc_ck = pd.crosstab(dfck["下单日期"],dfck["activity_name"],margins=True)
# 商品ID
qd_list = ['搜索-租手机','搜索-租苹果手机','搜索下挂-商品','搜索直达banner','搜索下挂券','顶部搜索框的搜索结果页','搜索侠客行-核心词','搜索侠客行-首页','搜索邦道-销售线索','搜索邦道-小程序推广']
sp_jj = pd.pivot_table(df_j[df_j.来源渠道.isin(qd_list)],values='order_id', index=["下单日期", '商品ID'], columns='来源渠道', aggfunc='count')
sp_ck = pd.pivot_table(dfck[dfck.来源渠道.isin(qd_list)],values='order_id', index=["下单日期", '商品ID'], columns='来源渠道', aggfunc='count')
try:
    with pd.ExcelWriter(path + f'日报相关数据输出_{t_date}.xlsx', engine='xlsxwriter') as writer:
        # 去重订单
        gsqd_qcdd.to_excel(writer, sheet_name='归属渠道去重订单统计', index=True)  
        lyqd_qcdd.to_excel(writer, sheet_name='来源渠道去重订单统计', index=True)
        hdmc_qcdd.to_excel(writer, sheet_name='活动名称去重订单统计', index=True)
        # 进件
        gsqd_jj.to_excel(writer, sheet_name='归属渠道进件统计', index=True)
        lyqd_jj.to_excel(writer, sheet_name='来源渠道进件统计', index=True)
        hdmc_jj.to_excel(writer, sheet_name='活动名称进件统计', index=True)
        # 出库
        gsqd_ck.to_excel(writer, sheet_name='归属渠道出库统计', index=True)
        lyqd_ck.to_excel(writer, sheet_name='来源渠道出库统计', index=True)
        hdmc_ck.to_excel(writer, sheet_name='活动名称出库统计', index=True)
        # 商品ID
        sp_jj.to_excel(writer, sheet_name='来源渠道商品ID进件统计', index=True)
        sp_ck.to_excel(writer, sheet_name='来源渠道商品ID出库统计', index=True)
except PermissionError:
    print(PermissionError)

In [187]:
# sp_jj.tail()
df["下单日期"].unique()

<DatetimeArray>
['2025-10-09 00:00:00', '2025-10-10 00:00:00', '2025-10-11 00:00:00',
 '2025-10-12 00:00:00', '2025-10-13 00:00:00', '2025-10-14 00:00:00',
 '2025-10-15 00:00:00', '2025-10-16 00:00:00', '2025-10-17 00:00:00',
 '2025-10-18 00:00:00', '2025-10-19 00:00:00', '2025-10-20 00:00:00',
 '2025-10-21 00:00:00', '2025-10-22 00:00:00', '2025-10-23 00:00:00',
 '2025-10-24 00:00:00']
Length: 16, dtype: datetime64[ns]

In [188]:
# dfck[(dfck['来源渠道']=='搜索下挂-商品')&(dfck['退货拒签']==1)].shape

# 总体 "租完即送"占比

In [189]:
# 获取拒量的出库和非拒量的出库
dfck_fjl = dfck[~dfck.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True]
dfck_jl = dfck[(dfck.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True)&(dfck.tips.str.contains('联合拒量订单')==False)]
# 可能哪天会发现连续15天没有拒量出库的数据，可以将时间范围扩大21-30天临时解决先跑下日报

In [190]:
dfck_jl.shape#[dfck_jl['下单日期']=='2025-09-03']

(4, 133)

In [191]:
# #总体 "租完即送"占比（拒量）
# df_rent_jlck = pd.crosstab(dfck_jl["下单日期"],dfck_jl["租赁方案"],margins=True)
# # 填充不连续日期的数据

# df_rent_jlck = clean.continuous_dates(df_rent_jlck.iloc[:-1, :])
# df_rent_jlck = df_rent_jlck.reset_index(names=['下单日期'])
# df_rent_jlck['下单日期'] = pd.to_datetime(df_rent_jlck['下单日期'])
# df_rent_jlck = df_rent_jlck.set_index('下单日期')
# df_rent_jlck.rename(columns={'租完即送': '租完即送_拒量', '租完归还': '租完归还_拒量', 'All': 'All_拒量'}, inplace=True)
# try:
#     df_rent_jlck.loc[:, '租完即送出库占比_拒量'] = (df_rent_jlck.租完即送_拒量/df_rent_jlck.All_拒量).map(lambda x: format(x, '.2%'))
# except:
#     df_rent_jlck.loc[:, '租完即送_拒量'] = 0
#     df_rent_jlck.loc[:, '租完即送出库占比_拒量'] = (df_rent_jlck.租完即送_拒量/df_rent_jlck.All_拒量).map(lambda x: format(x, '.2%'))
# try:
#     df_rent_jlck.loc[:, '租完即归还库占比_拒量'] = (df_rent_jlck.租完归还_拒量/df_rent_jlck.All_拒量).map(lambda x: format(x, '.2%'))
# except:
#     df_rent_jlck.loc[:, '租完归还_拒量'] = 0
#     df_rent_jlck.loc[:, '租完即归还库占比_拒量'] = (df_rent_jlck.租完归还_拒量/df_rent_jlck.All_拒量).map(lambda x: format(x, '.2%'))


In [192]:
import pandas as pd
from datetime import datetime, timedelta

df_rent_jlck = pd.crosstab(dfck_jl["下单日期"], dfck_jl["租赁方案"], margins=True)

# 如果 df_rent_jlck 为空，则进行处理
if df_rent_jlck.empty:
    # 获取当前日期
    current_date = datetime.now().date()
    
    # 计算15天前的日期
    start_date = current_date - timedelta(days=15)  # 15天不包括今天，所以是15天前
    
    # 生成最近15天的所有日期
    date_range = pd.date_range(start=start_date, end=current_date, freq='D')
    
    # 创建一个新的 DataFrame，列名与原始 df_rent_jlck 一致
    df_rent_jlck = pd.DataFrame(index=date_range, columns=['租完即送_拒量', '租完归还_拒量', 'All_拒量', '租完即送出库占比_拒量', '租完即归还库占比_拒量'])
    
    # 将所有数据填充为 0 或 '0%'
    df_rent_jlck.fillna(0, inplace=True)
    # df_rent_jlck['租完即送出库占比_拒量'] = 0
    # df_rent_jlck['租完即归还库占比_拒量'] = 0
    
    # 设置索引名称
    df_rent_jlck.index.name = '下单日期'
else:
    # 原有处理逻辑
    # df_rent_jlck_clean = df_rent_jlck.iloc[:-1, :]#.dropna(subset=['下单日期']) 
    df_rent_jlck = clean.continuous_dates(df_rent_jlck.iloc[:-1, :])
    df_rent_jlck = df_rent_jlck.reset_index(names=['下单日期'])
    df_rent_jlck['下单日期'] = pd.to_datetime(df_rent_jlck['下单日期'])
    df_rent_jlck = df_rent_jlck.set_index('下单日期')
    df_rent_jlck.rename(columns={'租完即送': '租完即送_拒量', '租完归还': '租完归还_拒量', 'All': 'All_拒量'}, inplace=True)
    
    try:
        df_rent_jlck.loc[:, '租完即送出库占比_拒量'] = (df_rent_jlck.租完即送_拒量/df_rent_jlck.All_拒量).map(lambda x: format(x, '.2%'))
    except:
        df_rent_jlck.loc[:, '租完即送_拒量'] = 0
        df_rent_jlck.loc[:, '租完即送出库占比_拒量'] = (df_rent_jlck.租完即送_拒量/df_rent_jlck.All_拒量).map(lambda x: format(x, '.2%'))
    
    try:
        df_rent_jlck.loc[:, '租完即归还库占比_拒量'] = (df_rent_jlck.租完归还_拒量/df_rent_jlck.All_拒量).map(lambda x: format(x, '.2%'))
    except:
        df_rent_jlck.loc[:, '租完归还_拒量'] = 0
        df_rent_jlck.loc[:, '租完即归还库占比_拒量'] = (df_rent_jlck.租完归还_拒量/df_rent_jlck.All_拒量).map(lambda x: format(x, '.2%'))

In [193]:
# df_rent_jlck

In [194]:
#总体 "租完即送"占比（排除拒量）
df_rent  = pd.crosstab(df_j["下单日期"],df_j["租赁方案"],margins=True)
df_rent=df_rent.rename(columns={"租完即送":"租完即送进件","租完归还":"租完归还进件","All":"合计进件",})
df_rent_ck=pd.crosstab(dfck_fjl["下单日期"],dfck_fjl["租赁方案"],margins=True)
df_rent_ck=df_rent_ck.rename(columns={"租完即送":"租完即送出库","租完归还":"租完归还出库","All":"合计出库",})
df_r=pd.merge(df_rent,df_rent_ck,left_index=True, right_index=True)
df_r["租完即送进件占比"]=df_r["租完即送进件"]/df_r["合计进件"]
df_r["租完即送进件占比"]=df_r["租完即送进件占比"].apply(lambda x:format(x,".2%"))
df_r["租完归还进件占比"]=df_r["租完归还进件"]/df_r["合计进件"]
df_r["租完归还进件占比"]=df_r["租完归还进件占比"].apply(lambda x:format(x,".2%"))
df_r["租完即送出库占比"]=df_r["租完即送出库"]/df_r["合计出库"]
df_r["租完即送出库占比"]=df_r["租完即送出库占比"].apply(lambda x:format(x,".2%"))
df_r["租完归还出库占比"]=df_r["租完归还出库"]/df_r["合计出库"]
df_r["租完归还出库占比"]=df_r["租完归还出库占比"].apply(lambda x:format(x,".2%"))
df_r["租完即送转化率"]=df_r["租完即送出库"]/df_r["租完即送进件"]
df_r["租完即送转化率"]=df_r["租完即送转化率"].apply(lambda x:format(x,".2%"))
df_r["租完归还转化率"]=df_r["租完归还出库"]/df_r["租完归还进件"]
df_r["租完归还转化率"]=df_r["租完归还转化率"].apply(lambda x:format(x,".2%"))
df_r = df_r.iloc[:-1, :]
df_r = df_r.reset_index()
df_r.loc[:, '下单日期'] = pd.to_datetime(df_r['下单日期'])
df_r.set_index('下单日期', inplace=True)
# 拼接拒量出库
df_r1 = df_r.merge(df_rent_jlck, on='下单日期', how='left')

df_r1 = df_r1[["租完即送进件","租完归还进件","租完归还进件占比","租完即送进件占比","租完即送出库","租完归还出库", '租完即送_拒量', '租完归还_拒量', '租完即归还库占比_拒量', '租完即送出库占比_拒量', "租完归还出库占比","租完即送出库占比", '租完即送转化率','租完归还转化率']]#.iloc[:-1, :]
df_r1 = clean.continuous_dates(df_r1)
df_r_new = df_r1.reset_index(names=['下单日期'])


# 总体出库订单碎屏险购买数据

## 总体

In [195]:
def ck(dfck, name):
    # '服务订单状态：1、待支付；2、待确认；3、服务中；4、已失效；5、已取消；6、已退款',
    dfck[f'已取消'] = np.where(dfck["service_status"]==5,1,0)
    dfck[f'已退款'] = np.where(dfck["service_status"]==6,1,0)
    dfck[f'待支付'] = np.where(dfck["service_status"]==1,1,0)
    dfck[f'待确认'] = np.where(dfck["service_status"]==2,1,0)
    dfck[f'服务中'] = np.where(dfck["service_status"]==3,1,0)

    df_s3  =  dfck.groupby(["下单日期"]).agg({f'已取消':'sum',f'已退款':'sum', f'待支付':'sum', f'待确认':'sum',f'服务中':'sum'})
    
    df_s3[f"最终实际支付碎屏险出库_{name}"]=df_s3[f"待确认"]+df_s3[f"服务中"] +df_s3[f"待支付"]
    return df_s3

In [196]:
# 不包含京享租
dfck_fjl_fjd = dfck_fjl[dfck_fjl.归属渠道!='京东渠道']
dfck_jl_fjd = dfck_jl[dfck_jl.归属渠道!='京东渠道']
#总体出库订单碎屏险购买数据

df_bx_ck=pd.crosstab(dfck_fjl_fjd["下单日期"],dfck_fjl_fjd["buy_service_product"],margins=True)

if df_bx_ck.empty:
    # 获取当前日期
    current_date = datetime.now().date()
    
    # 计算15天前的日期
    start_date = current_date - timedelta(days=15)  # 15天不包括今天，所以是15天前
    
    # 生成最近15天的所有日期
    date_range = pd.date_range(start=start_date, end=current_date, freq='D')
    
    # 创建一个新的 DataFrame，列名与原始 df_rent_jlck 一致
    df_bx_ck = pd.DataFrame(index=date_range, columns=['不购买碎屏险出库', '购买碎屏险出库', 'All', '合计出库'])
    
    # 将所有数据填充为 0 或 '0%'
    df_bx_ck.fillna(0, inplace=True)
    
    
    # 设置索引名称
    df_bx_ck.index.name = '下单日期'
else:
    df_bx_ck=df_bx_ck.rename(columns={1:"不购买碎屏险出库",2:"购买碎屏险出库","All":"合计出库",})
    
#总体出库订单碎屏险购买数据——拒量
df_bx_ck_jl=pd.crosstab(dfck_jl_fjd["下单日期"],dfck_jl_fjd["buy_service_product"],margins=True)

if df_bx_ck_jl.empty:
    # 获取当前日期
    current_date = datetime.now().date()
    
    # 计算15天前的日期
    start_date = current_date - timedelta(days=15)  # 15天不包括今天，所以是15天前
    
    # 生成最近15天的所有日期
    date_range = pd.date_range(start=start_date, end=current_date, freq='D')
    
    # 创建一个新的 DataFrame，列名与原始 df_rent_jlck 一致
    df_bx_ck_jl = pd.DataFrame(index=date_range, columns=['不购买碎屏险出库', '购买碎屏险出库', 'All', '合计出库_拒量'])
    
    # 将所有数据填充为 0 或 '0%'
    df_bx_ck_jl.fillna(0, inplace=True)
    
    
    # 设置索引名称
    df_bx_ck_jl.index.name = '下单日期'
else:

    df_bx_ck_jl=df_bx_ck_jl.rename(columns={1:"不购买碎屏险出库",2:"购买碎屏险出库","All":"合计出库_拒量",})

# 非拒量
df_s3 = ck(dfck_fjl_fjd, 'All')
# 拒量
df_s3_jl = ck(dfck_jl_fjd, '拒量')

# df_s3=pd.crosstab(dfck["下单日期"],dfck["服务订单状态"],margins=True)
# df_s3=df_s3.drop(columns={"All"})
df_s2merge=pd.merge(df_bx_ck,df_s3,left_index=True, right_index=True)
df_s3merge=pd.merge(df_bx_ck_jl,df_s3_jl,left_index=True, right_index=True)
df_s2merge = df_s2merge.merge(df_s3merge[['合计出库_拒量', '最终实际支付碎屏险出库_拒量']], left_index=True, right_index=True, how='left')
df_s2merge["实际支付碎屏险比例"]=df_s2merge["最终实际支付碎屏险出库_All"]/df_s2merge["合计出库"]
df_s2merge["实际支付碎屏险比例"]=df_s2merge["实际支付碎屏险比例"].apply(lambda x:format(x,".2%"))
df_s2merge.loc[:, '实际支付碎屏险比例_拒量'] = (df_s2merge.最终实际支付碎屏险出库_拒量/df_s2merge.合计出库_拒量).map(lambda x: format(x, '.2%'))

df_s2merge = df_s2merge[['不购买碎屏险出库','购买碎屏险出库','合计出库', '合计出库_拒量','已取消','已退款', '待支付', '待确认','服务中','最终实际支付碎屏险出库_拒量','最终实际支付碎屏险出库_All', '实际支付碎屏险比例_拒量',  '实际支付碎屏险比例']]
df_s2merge = clean.continuous_dates(df_s2merge)
df_s2merge = df_s2merge.reset_index(names=['下单日期'])
# df_s2merge
df_s2merge.tail()

,下单日期,不购买碎屏险出库,购买碎屏险出库,合计出库,合计出库_拒量,已取消,已退款,待支付,待确认,服务中,最终实际支付碎屏险出库_拒量,最终实际支付碎屏险出库_All,实际支付碎屏险比例_拒量,实际支付碎屏险比例
11,2025-10-20,1.0,145.0,146.0,0.0,1.0,1.0,0.0,49.0,94.0,0.0,143.0,nan%,97.95%
12,2025-10-21,1.0,106.0,107.0,0.0,0.0,0.0,0.0,80.0,26.0,0.0,106.0,nan%,99.07%
13,2025-10-22,1.0,112.0,113.0,0.0,0.0,0.0,0.0,109.0,3.0,0.0,112.0,nan%,99.12%
14,2025-10-23,0.0,91.0,91.0,0.0,0.0,0.0,0.0,91.0,0.0,0.0,91.0,nan%,100.00%
15,2025-10-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0


In [197]:
# df_s2merge
# dfck[dfck["归属渠道"]=="芝麻租物"]

In [198]:
#芝麻租物出库订单碎屏险购买数据
dfckzw=dfck[dfck["归属渠道"]=="芝麻租物"]
df_zw3=pd.crosstab(dfckzw["下单日期"],dfckzw["buy_service_product"],margins=True)
df_zw3=df_zw3.rename(columns={1:"不购买碎屏险出库",2:"购买碎屏险出库","All":"合计出库"})

dfckzw['已取消'] = np.where(dfckzw["service_status"]==5,1,0)
dfckzw['已退款'] = np.where(dfckzw["service_status"]==6,1,0)
dfckzw['待支付'] = np.where(dfckzw["service_status"]==1,1,0)
dfckzw['待确认'] = np.where(dfckzw["service_status"]==2,1,0)
dfckzw['服务中'] = np.where(dfckzw["service_status"]==3,1,0)

df_zws3  =   dfckzw.groupby(["下单日期"]).agg({'已取消':'sum','已退款':'sum', '待支付':'sum', '待确认':'sum','服务中':'sum'})  
# =pd.crosstab(dfckzw["下单日期"],dfckzw["服务订单状态"],margins=True)
# df_zws3=df_zws3.drop(columns={"All"})

df_zws2merge = pd.merge(df_zw3,df_zws3,left_index=True, right_index=True)
df_zws2merge["最终实际支付碎屏险出库"] = df_zws2merge["待确认"]+df_zws2merge["服务中"]+df_zws2merge["待支付"]
# 如果发现合计出库为0，或者没有芝麻租物的订单，则可能是每日各渠道订单统计导出中设置了出库渠道仅有搜索渠道
df_zws2merge["实际支付碎屏险比例"] = df_zws2merge["最终实际支付碎屏险出库"]/df_zws2merge["合计出库"]
df_zws2merge["实际支付碎屏险比例"] = df_zws2merge["实际支付碎屏险比例"].apply(lambda x:format(x,".2%"))


df_zws2merge = df_zws2merge[['不购买碎屏险出库','购买碎屏险出库','合计出库','已取消','已退款', '待支付', '待确认','服务中','最终实际支付碎屏险出库','实际支付碎屏险比例']]
df_zws2merge = clean.continuous_dates(df_zws2merge)
df_zws2merge_new = df_zws2merge.reset_index(names=['下单日期1'])
# df_zws2merge_new = df_zws2merge_new.rename(columns={'下单日期': '下单日期1'})
df_zws2merge_new.tail()

,下单日期1,不购买碎屏险出库,购买碎屏险出库,合计出库,已取消,已退款,待支付,待确认,服务中,最终实际支付碎屏险出库,实际支付碎屏险比例
11,2025-10-20,0.0,93.0,93.0,1.0,1.0,0.0,32.0,59.0,91.0,97.85%
12,2025-10-21,1.0,72.0,73.0,0.0,0.0,0.0,52.0,20.0,72.0,98.63%
13,2025-10-22,0.0,73.0,73.0,0.0,0.0,0.0,71.0,2.0,73.0,100.00%
14,2025-10-23,0.0,56.0,56.0,0.0,0.0,0.0,56.0,0.0,56.0,100.00%
15,2025-10-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


## 整合

In [199]:
df_s2merge_new = df_s2merge#.reset_index()
df_s2merge_new.loc[:, '碎屏险实际出库占比'] = (df_s2merge_new.最终实际支付碎屏险出库_All/df_s2merge_new.购买碎屏险出库).map(lambda x: format(x,".2%"))
df_s2merge_new.loc[:, '非芝麻实际支付碎屏险比例'] = ((df_s2merge_new.购买碎屏险出库-df_zws2merge_new.购买碎屏险出库)/(df_s2merge_new.合计出库-df_zws2merge_new.合计出库)).map(lambda x: format(x,".2%"))
df_s2merge_all_new = pd.concat([df_s2merge_new, df_zws2merge_new], axis=1)
# df_s2merge_all_new#.set_index('下单日期')

## 京享租

In [200]:
df_jd2_ck = df_jd2[df_jd2.是否出库==1]
df_jd2_sp=pd.crosstab(df_jd2_ck["下单日期"],df_jd2_ck["buy_service_product"],margins=True)
df_jd2_sp=df_jd2_sp.rename(columns={1:"不购买碎屏险出库",2:"购买碎屏险出库","All":"合计出库",})
df_jdsp = ck(df_jd2_ck, 'All')
df_jd_merge=pd.merge(df_jd2_sp,df_jdsp,left_index=True, right_index=True)
df_jd_merge["实际支付碎屏险比例"]=(df_jd_merge["最终实际支付碎屏险出库_All"]/df_jd_merge["合计出库"]).map(lambda x: format(x, '.2%'))
df_jd_merge = df_jd_merge.reset_index()
df_jd_merge.loc[:, '下单日期'] = df_jd_merge.下单日期.astype('object')
df_jd_group_new = df_jd_group_new.merge(df_jd_merge[['下单日期', '合计出库', '最终实际支付碎屏险出库_All', '实际支付碎屏险比例']], on='下单日期', how='left')
df_jd_group_new.tail()

,下单日期,创建订单数,创建进件数,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审强拒,强拒比例,机审通过件,人审拒绝,风控通过件,风控通过率,客户取消,无法联系,出库前风控强拒,待审核,出库（按下单时间）,出库（按转化时间）,进件出库率,取消率,人审拒绝率,出库前强拒比例,无法联系占比,订单出库率,合计出库,最终实际支付碎屏险出库_All,实际支付碎屏险比例
11,2025-10-20,2048,987,1465,559,38.16%,838,57.20%,712.0,84.96%,126.0,78.0,48.0,5.73%,4,11,0.0,0,33,33.0,3.94%,0.48%,9.31%,0.00%,1.31%,2.25%,33.0,33.0,100.00%
12,2025-10-21,1696,845,1286,484,37.64%,750,58.32%,628.0,83.73%,122.0,70.0,52.0,6.93%,2,4,0.0,3,43,41.0,5.73%,0.27%,9.33%,0.00%,0.53%,3.34%,43.0,42.0,97.67%
13,2025-10-22,1774,842,1321,541,40.95%,730,55.26%,612.0,83.84%,118.0,68.0,50.0,6.85%,6,7,0.0,7,30,31.0,4.11%,0.82%,9.32%,0.00%,0.96%,2.27%,30.0,29.0,96.67%
14,2025-10-23,1944,916,1444,565,39.13%,809,56.02%,680.0,84.05%,129.0,63.0,66.0,8.16%,5,1,0.0,26,34,39.0,4.20%,0.62%,7.79%,0.00%,0.12%,2.35%,34.0,33.0,97.06%
15,2025-10-24,234,85,142,74,52.11%,59,41.55%,47.0,79.66%,12.0,3.0,9.0,15.25%,0,0,0.0,9,0,0.0,0.00%,0.00%,5.08%,0.00%,0.00%,0.00%,NaN,NaN,NaN


# 拒量（廖志彧）

## 何薪华需求

In [201]:
# df.columns

In [202]:
# # 拒量的订单2024年12月到2025年6月正常还款的（租赁中）用户没有逾期的订单，需要订单号，用户下单的手机号码，审核人，还款期数（已还）
# df_jl = df[(df.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True)&df.status==4]
# df_jl = df_jl[~df_jl.merchant_name.isin(["小蚂蚁租机","兴鑫兴通讯","人人享租","崇胜数码","喜卓灵租机"])]
# # 


# df_name_new = df_name.sort_values('update_time', ascending=False).groupby('order_id').head(1)
# # 内连接，获取出库单分配人
# df_jl_name = df_jl_new[df_jl_new.是否出库==1].merge(df_name_new, on='order_id', how='inner')
# df_jl_name_new = df_jl_name[['下单日期', 'order_id', 'order_number', '策略命中等级', '分配人']]

## 拒量数据(转化数据、明细)

In [203]:
# # # 设置时间段
# now_day = datetime.now()
# seven_ago = now_day - pd.Timedelta(days=8)
# one_ago = now_day - pd.Timedelta(days=1)
# # 获取特定时间段的拒量订单的转化数据和明细

# df_jl = df[df.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True]
# df_jl = df_jl[~df_jl.merchant_name.isin(["小蚂蚁租机","兴鑫兴通讯","人人享租","崇胜数码","喜卓灵租机"])]
# df_jl_new = df_jl#[~df_jl.tips.str.contains('策略241205命中(1)', regex=False)]
# df_jl_new['策略命中等级'] = df_jl_new['tips'].str.extract(r'(策略241205命中\(\d+\)?|策略241212命中\(\d+\)?|命中自有模型回捞策略|回捞策略250330命中?)')[0]
# # 进件数，出库数，出库率，风险等级
# df_jl_new_group = df_jl_new.groupby('下单日期').agg({'是否进件': 'sum', '是否出库': 'sum'})
# df_jl_new_group.rename(columns={'是否进件': '进件数', '是否出库': '出库'}, inplace=True)
# df_jl_new_group['进件出库率'] = (df_jl_new_group.出库/df_jl_new_group.进件数).map(lambda x: format(x, '.2%'))
# df_jl_new2 = df_jl_new[['下单日期', 'order_number', '策略命中等级']]
# df_jl_new2_group = df_jl_new2.groupby('策略命中等级').agg(数量=('order_number', 'count'))

# # 获取当前是星期几
# Monday = pd.to_datetime(datetime.now().strftime('%Y-%m-%d')).day_name()
# if Monday == 'Monday':
#     with pd.ExcelWriter(f'F:/需求/廖志彧需求/拒量数据_{Today_zfb}.xlsx', engine='openpyxl') as writer:
#         df_jl_new_group.to_excel(writer, sheet_name='转化数据')
#         df_jl_new2_group.to_excel(writer, sheet_name='拒量数据明细')
#         df_jl_new['order_number'].to_excel(writer, sheet_name='拒量明细', index=False)
# df_name['分配人'].unique()


## 出库单分配人

In [204]:
# # # 分配人名单：蔡斯静 林婉婷 胡春燕 何薪华 刘慧琪 钟桂洪
# name_list = ['蔡斯静', '林婉婷', '胡春燕', '何薪华', '刘慧琪', '钟桂洪', '王伟鸿']
# # 筛选出库单分配人
# df_name = df_name[df_name['分配人'].isin(name_list)]
# # df_name.分配人.unique()
# df_jl = df[df.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True]
# df_jl = df_jl[~df_jl.merchant_name.isin(["小蚂蚁租机","兴鑫兴通讯","人人享租","崇胜数码","喜卓灵租机"])]
# df_jl_new = df_jl#[~df_jl.tips.str.contains('策略241205命中(1)', regex=False)]
# df_jl_new['策略命中等级'] = df_jl_new['tips'].str.extract(r'(策略241205命中\(\d+\)?|策略241212命中\(\d+\)?|命中自有模型回捞策略|回捞策略250330命中?)')[0]

# df_name_new = df_name.sort_values('update_time', ascending=False).groupby('order_id').head(1)
# # 内连接，获取出库单分配人
# df_jl_name = df_jl_new[df_jl_new.是否出库==1].merge(df_name_new, on='order_id', how='inner')
# df_jl_name_new = df_jl_name[['下单日期', 'order_id', 'order_number', '策略命中等级', '分配人']]
# with pd.ExcelWriter(f'F:/需求/廖志彧需求/分配数据_{t_date}.xlsx', engine='openpyxl') as writer:
#     df_jl_name_new.to_excel(writer, sheet_name='出库单分配人', index=False)
# # 

In [205]:
# df_jl_new.下单日期.unique()

## 首逾

In [206]:
# # 在没有其它变化的情况下该口径是OK的：2025.9.3
# now_day = datetime.now()

# df_jl = df[df.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True]
# df_jl = df_jl[~df_jl.merchant_name.isin(["小蚂蚁租机","兴鑫兴通讯","人人享租","崇胜数码","喜卓灵租机"])]

# name_list = ['蔡斯静', '林婉婷', '胡春燕', '何薪华', '刘慧琪', '钟桂洪','王伟鸿']
# df_name = df_name[df_name['分配人'].isin(name_list)]
# df_name_new = df_name.sort_values('update_time', ascending=False).groupby('order_id').head(1)
# # 首逾
# date1 = '2025-07'
# date2 = '2025-08'
# df_new = df_jl[df_jl.下单月份==date1]

# # df_new["出库"] = np.where(df_new["status"].isin([2, 3, 4, 5, 6, 8, 15]), 1, 0)

# df_new = df_new[df_new.是否出库==1]

# df_stages_new = df_stages[df_stages['order_id'].notna()]
# df_stages_new['order_id'] = df_stages_new['order_id'].astype(int)
# df_stages_new.loc[:, '预计还款月份'] = df_stages_new.refund_date.dt.strftime('%Y-%m')
# df_new_merge = df_new.merge(df_stages_new, on='order_id', how='left')
# # df_jl_stages_new = df_new_merge[(df_new_merge.预计还款月份==date2)&(df_new_merge.sort==2)]
# df_jl_stages_new = df_new_merge[(df_new_merge.sort==2)]
# df_jl_stages_new.loc[:, 'now_day'] = pd.to_datetime(now_day.strftime('%Y-%m-%d'))
# # df_jl_stages_new = df_jl_stages_new[df_jl_stages_new.refund_date<=df_jl_stages_new.now_day]
# df_jl_stages_new.loc[:, 'overdue_day'] = np.where(df_jl_stages_new.reality_refund_date.notna(), 
#                                                 np.where(df_jl_stages_new.reality_refund_date>df_jl_stages_new.refund_date, (df_jl_stages_new.reality_refund_date-df_jl_stages_new.refund_date), 0)
#                                                 , np.where(df_jl_stages_new.refund_date>df_jl_stages_new.now_day, 0, (df_jl_stages_new.now_day-df_jl_stages_new.refund_date)))
# df_jl_stages_new.loc[:, 'overdue_day'] = df_jl_stages_new['overdue_day'].apply(lambda x: x.days)
# # df_jl_stages_new = df_jl_stages_new[df_jl_stages_new.overdue_day>0]

# # df_jl_stages_news = df_jl_stages_new[df_jl_stages_new.overdue_day>0][['order_id', 'order_number', '下单日期', 'refund_date', 'reality_refund_date',  'overdue_day']].rename(columns={'order_id': '订单id', 'order_number': '订单号', 'refund_date': '应还日期', 'reality_refund_date': '实还日期', 'overdue_day': '逾期天数'})
# df_jl_stages_news = df_jl_stages_new[['order_id', 'order_number', '下单日期', 'refund_date', 'reality_refund_date',  'overdue_day', 'status2']].rename(columns={'order_id': '订单id', 'order_number': '订单号', 'refund_date': '应还日期', 'reality_refund_date': '实还日期', 'overdue_day': '逾期天数'})
# df_jl_stages_news = df_jl_stages_news.merge(df_name_new[['order_id', '分配人']], left_on='订单id', right_on='order_id', how='inner')
# df_jl_stages_news.loc[:, '是否逾期'] = np.where((df_jl_stages_news.status2=='租赁中')&(df_jl_stages_news.实还日期.isna())&(df_jl_stages_news.逾期天数>0), 1, 0)
# df_jl_stages_news_g = df_jl_stages_news.groupby('分配人').agg({'order_id': 'count', '是否逾期': 'sum'}).rename(columns={'order_id': '出库'})
# df_jl_stages_news_g.loc[:, '逾期/出库'] = (df_jl_stages_news_g.是否逾期/df_jl_stages_news_g.出库).map(lambda x: format(x, '.2%'))
# with pd.ExcelWriter(f'F:/需求/廖志彧需求/首逾_{t_date}.xlsx', engine='xlsxwriter') as writer:
#     df_jl_stages_news.to_excel(writer, sheet_name='出库订单', index=False)
#     df_jl_stages_news_g.to_excel(writer, sheet_name='首逾订单')
# # df_jl_stages_news

In [207]:
# df_jl.tail()

## wxz总体拒量程序

In [208]:
# 设置时间段
# now_day = datetime.now()
# seven_ago = now_day - pd.Timedelta(days=8)
# one_ago = now_day - pd.Timedelta(days=1)
# # 获取特定时间段的拒量订单的转化数据和明细
# # df_jl = df[(df.下单日期>=seven_ago)&(df.下单日期<=one_ago)&(df.tips.str.contains(r'策略2412|命中自有模型回捞策略')==True)] 多半无用
# df_jl = df[df.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True]
# df_jl = df_jl[~df_jl.merchant_name.isin(["小蚂蚁租机","兴鑫兴通讯","人人享租","崇胜数码","喜卓灵租机"])]
# df_jl_new = df_jl#[~df_jl.tips.str.contains('策略241205命中(1)', regex=False)]
# df_jl_new['策略命中等级'] = df_jl_new['tips'].str.extract(r'(策略241205命中\(\d+\)?|策略241212命中\(\d+\)?|命中自有模型回捞策略|回捞策略250330命中?)')[0]
# # 进件数，出库数，出库率，风险等级
# df_jl_new_group = df_jl_new.groupby('下单日期').agg({'是否进件': 'sum', '是否出库': 'sum'})
# df_jl_new_group.rename(columns={'是否进件': '进件数', '是否出库': '出库'}, inplace=True)
# df_jl_new_group['进件出库率'] = (df_jl_new_group.出库/df_jl_new_group.进件数).map(lambda x: format(x, '.2%'))
# df_jl_new2 = df_jl_new[['下单日期', 'order_number', '策略命中等级']]
# df_jl_new2_group = df_jl_new2.groupby('策略命中等级').agg(数量=('order_number', 'count'))

# 获取当前是星期几
# Monday = pd.to_datetime(datetime.now().strftime('%Y-%m-%d')).day_name()
# if Monday == 'Monday':
#     with pd.ExcelWriter(f'F:/需求/廖志彧需求/拒量数据_{Today_zfb}.xlsx', engine='openpyxl') as writer:
#         df_jl_new_group.to_excel(writer, sheet_name='转化数据')
#         df_jl_new2_group.to_excel(writer, sheet_name='拒量数据明细')
#         df_jl_new['order_number'].to_excel(writer, sheet_name='拒量明细', index=False)


# df_name_new = df_name.sort_values('update_time', ascending=False).groupby('order_id').head(1)
# df_jl_name = df_jl_new[df_jl_new.是否出库==1].merge(df_name_new, on='order_id', how='left')
# df_jl_name_new = df_jl_name[['下单日期', 'order_id', 'order_number', '策略命中等级', '分配人']]
# with pd.ExcelWriter(f'F:/需求/廖志彧需求/分配数据_{Today_zfb}.xlsx', engine='openpyxl') as writer:
#     df_jl_name_new.to_excel(writer, sheet_name='出库单分配人', index=False)
# df_jl_name_new

# # 首逾
# date1 = '2025-06'
# date2 = '2025-07'
# df_new = df_jl[df_jl.下单月份==date1]
# df_new = df_new[df_new.是否出库==1]
# df_stages_new = df_stages[df_stages['order_id'].notna()]
# df_stages_new['order_id'] = df_stages_new['order_id'].astype(int)
# df_stages_new.loc[:, '预计还款月份'] = df_stages_new.refund_date.dt.strftime('%Y-%m')
# df_new_merge = df_new.merge(df_stages_new, on='order_id', how='left')
# df_jl_stages_new = df_new_merge[(df_new_merge.预计还款月份==date2)&(df_new_merge.sort==2)]
# df_jl_stages_new = df_new_merge[(df_new_merge.sort==2)]
# df_jl_stages_new.loc[:, 'now_day'] = pd.to_datetime(now_day.strftime('%Y-%m-%d'))
# # df_jl_stages_new = df_jl_stages_new[df_jl_stages_new.refund_date<=df_jl_stages_new.now_day]
# df_jl_stages_new.loc[:, 'overdue_day'] = np.where(df_jl_stages_new.reality_refund_date.notna(), 
#                                                 np.where(df_jl_stages_new.reality_refund_date>df_jl_stages_new.refund_date, (df_jl_stages_new.reality_refund_date-df_jl_stages_new.refund_date), 0)
#                                                 , np.where(df_jl_stages_new.refund_date>df_jl_stages_new.now_day, 0, (df_jl_stages_new.now_day-df_jl_stages_new.refund_date)))
# df_jl_stages_new.loc[:, 'overdue_day'] = df_jl_stages_new['overdue_day'].apply(lambda x: x.days)
# # df_jl_stages_new = df_jl_stages_new[df_jl_stages_new.overdue_day>0]

# # df_jl_stages_news = df_jl_stages_new[df_jl_stages_new.overdue_day>0][['order_id', 'order_number', '下单日期', 'refund_date', 'reality_refund_date',  'overdue_day']].rename(columns={'order_id': '订单id', 'order_number': '订单号', 'refund_date': '应还日期', 'reality_refund_date': '实还日期', 'overdue_day': '逾期天数'})
# df_jl_stages_news = df_jl_stages_new[['order_id', 'order_number', '下单日期', 'refund_date', 'reality_refund_date',  'overdue_day', 'status2']].rename(columns={'order_id': '订单id', 'order_number': '订单号', 'refund_date': '应还日期', 'reality_refund_date': '实还日期', 'overdue_day': '逾期天数'})
# df_jl_stages_news = df_jl_stages_news.merge(df_name_new[['order_id', '分配人']], left_on='订单id', right_on='order_id', how='left')
# df_jl_stages_news.loc[:, '是否逾期'] = np.where((df_jl_stages_news.status2=='租赁中')&(df_jl_stages_news.实还日期.isna())&(df_jl_stages_news.逾期天数>0), 1, 0)
# df_jl_stages_news_g = df_jl_stages_news.groupby('分配人').agg({'order_id': 'count', '是否逾期': 'sum'}).rename(columns={'order_id': '出库'})
# df_jl_stages_news_g.loc[:, '逾期/出库'] = (df_jl_stages_news_g.是否逾期/df_jl_stages_news_g.出库).map(lambda x: format(x, '.2%'))
# with pd.ExcelWriter(f'F:/需求/廖志彧需求/首逾_{t_date}.xlsx', engine='xlsxwriter') as writer:
#     df_jl_stages_news.to_excel(writer, sheet_name='出库订单', index=False)
#     df_jl_stages_news_g.to_excel(writer, sheet_name='首逾订单')
# df_jl_stages_news

In [209]:
# df_jl_new[['下单日期', 'order_number', '策略命中等级', '是否出库']].to_excel(f'F:/需求/总需求/拒量回捞数据_{t_date}.xlsx', index=False)
# df_jl_stages_news_g

## 拒量日报

In [210]:
# df_jl
df[(df['tips'].str.contains(r'联合拒量订单|支付宝联合运营订单', na=False))].shape

(1992, 114)

In [211]:
df_jl = df[(df.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True)&(~df['tips'].str.contains(r'联合拒量订单|支付宝联合运营订单', na=False))]
# 剔除回捞被拒漏到商家的数据9.1
reject_merchants = ["小蚂蚁租机", "兴鑫兴通讯", "人人享租", "崇胜数码", "喜卓灵租机", "喜卓灵新租机"]
df_jl = df_jl[~df_jl['merchant_name'].isin(reject_merchants)]

df_jl_group = df_jl.groupby('下单日期').agg({'是否进件': 'sum', '人审拒绝': 'sum', '风控通过件': 'sum', '客户取消': 'sum', '无法联系': 'sum', '出库前风控强拒': 'sum', '待审核': 'sum', '是否出库': 'sum', '已退款': 'sum'}).rename(columns={'是否进件': '进件数', '是否出库': '出库'})
df_jl_group.loc[:, '客户取消'] = df_jl_group.客户取消+df_jl_group.已退款
df_jl_group.loc[:, '风控通过率'] = (df_jl_group.风控通过件/df_jl_group.进件数).map(lambda x: format(x, '.2%'))
df_jl_group.loc[:, '进件出库率'] = (df_jl_group.出库/df_jl_group.进件数).map(lambda x: format(x, '.2%'))
df_jl_group.loc[:, '取消比例'] = (df_jl_group.客户取消/df_jl_group.进件数).map(lambda x: format(x, '.2%'))
df_jl_group.loc[:, '电审拒绝率'] = (df_jl_group.人审拒绝/df_jl_group.进件数).map(lambda x: format(x, '.2%'))
df_jl_group.loc[:, '出库前强拒比例'] = (df_jl_group.出库前风控强拒/df_jl_group.进件数).map(lambda x: format(x, '.2%'))
df_jl_group.loc[:, '无法联系占比'] = (df_jl_group.无法联系/df_jl_group.进件数).map(lambda x: format(x, '.2%'))
df_jl_group_new = df_jl_group[['进件数', '人审拒绝', '风控通过件', '风控通过率', '客户取消', '无法联系', '出库前风控强拒', '待审核', '出库', '进件出库率', '取消比例', '电审拒绝率', '出库前强拒比例', '无法联系占比']].reset_index()

df_jl_group_new.tail()

,下单日期,进件数,人审拒绝,风控通过件,风控通过率,客户取消,无法联系,出库前风控强拒,待审核,出库,进件出库率,取消比例,电审拒绝率,出库前强拒比例,无法联系占比
11,2025-10-20,105,79,11,10.48%,10,1,0,0,0,0.00%,9.52%,75.24%,0.00%,0.95%
12,2025-10-21,127,84,28,22.05%,27,0,0,1,0,0.00%,21.26%,66.14%,0.00%,0.00%
13,2025-10-22,103,41,49,47.57%,46,1,0,2,0,0.00%,44.66%,39.81%,0.00%,0.97%
14,2025-10-23,70,25,37,52.86%,18,0,1,18,0,0.00%,25.71%,35.71%,1.43%,0.00%
15,2025-10-24,11,0,9,81.82%,0,0,0,9,0,0.00%,0.00%,0.00%,0.00%,0.00%


# 挽留

In [212]:
df_wl_merge = df_wl.merge(df[['order_number', '是否出库']], on='order_number', how='left')
df_wl_merge.loc[:, '下单日期'] = pd.to_datetime(df_wl_merge.下单日期)
df_wl_merge.loc[:, '挽留成功'] = np.where(df_wl_merge.task_status=='SUCCESS', 1, 0)
df_wl_g = df_wl_merge.groupby('下单日期').agg({'order_number': 'nunique', '挽留成功': 'sum', '是否出库': 'sum'}).rename(columns={'order_number': '挽留数', '是否出库': '出库'})
df_wl_g_merge = df_all[['去重订单数', '进件数', '客户取消']].merge(df_wl_g, on='下单日期', how='left')
df_wl_g_merge.loc[:, '进件出库率'] = (df_wl_g_merge.出库/df_wl_g_merge.进件数).map(lambda x: format(x, '.2%'))
df_wl_g_merge.loc[:, '订单出库率'] = (df_wl_g_merge.出库/df_wl_g_merge.去重订单数).map(lambda x: format(x, '.2%'))
df_wl_g_merge.loc[:, '挽留成功率'] = (df_wl_g_merge.挽留成功/df_wl_g_merge.挽留数).map(lambda x: format(x, '.2%'))
df_wl_g_merge.loc[:, '挽留出库率'] = (df_wl_g_merge.出库/df_wl_g_merge.挽留数).map(lambda x: format(x, '.2%'))
df_wl_g_merge = clean.continuous_dates(df_wl_g_merge)
df_wl_g_merge_new = df_wl_g_merge.reset_index(names=['下单日期'])
df_wl_g_merge_new.tail()

,下单日期,去重订单数,进件数,客户取消,挽留数,挽留成功,出库,进件出库率,订单出库率,挽留成功率,挽留出库率
11,2025-10-20,7168,1995,25,6.0,1.0,1.0,0.05%,0.01%,16.67%,16.67%
12,2025-10-21,6768,1873,15,2.0,0.0,0.0,0.00%,0.00%,0.00%,0.00%
13,2025-10-22,6324,1750,20,0.0,0.0,0.0,nan%,nan%,nan%,nan%
14,2025-10-23,6443,1724,24,5.0,1.0,1.0,0.06%,0.02%,20.00%,20.00%
15,2025-10-24,902,166,0,0.0,0.0,0.0,nan%,nan%,nan%,nan%


# 获取日报文件名称并修改

In [213]:
now = datetime.now()
now_day = pd.to_datetime(now.strftime("%Y-%m-%d"))
now_date = now.strftime('%m%d')
before_date = (now-timedelta(days=1)).strftime('%m%d')
before_date_two = (now-timedelta(days=2)).strftime('%m%d')
before_date_four = (now-timedelta(days=4)).strftime('%m%d')
monday = pd.to_datetime(now.strftime('%Y-%m-%d')).day_name()

In [214]:
# 指定目录路径
directory = 'F:/日报/日报'

# 查找所有 .xlsx 和 .xls 文件
excel_files = glob.glob(os.path.join(directory, '*.xlsx')) + glob.glob(os.path.join(directory, '*.xls'))

for old_file_path in excel_files:
    # 提取文件名（不带路径）
    old_filename = os.path.basename(old_file_path)
    old_date = old_filename.split('.')[0][-4:]
    new_filename = old_filename.replace(old_date, before_date)
    # 将新的文件名添加到路径
    new_file_path = os.path.join(directory, new_filename)

    # 确保新文件名不存在，以避免覆盖文件
    if not os.path.exists(new_file_path):
        os.rename(old_file_path, new_file_path)
        print(f"文件 {old_filename} 已重命名为: {new_filename}")
    else:
        print(f"跳过文件 {old_filename}，因为新文件名已存在。")

文件 new商户订单统计表截至1022.xlsx 已重命名为: new商户订单统计表截至1023.xlsx
文件 new迪瓜租机订单日报截止1022.xlsx 已重命名为: new迪瓜租机订单日报截止1023.xlsx
文件 ~$new迪瓜租机订单日报截止1022.xlsx 已重命名为: ~$new迪瓜租机订单日报截止1023.xlsx


# 读取日报

## 获取日期及星期

In [215]:

# 中文星期映射 获取星期几

# 设置需要打开的文件和密码
# file_path1 = f'F:/日报/日报/迪瓜租机订单日报截止{before_date}.xlsx'
file_path1 = f'F:/日报/日报/new迪瓜租机订单日报截止{before_date}.xlsx'

# file_path2 = f'F:/日报/日报/商户订单统计表截至{before_date}.xlsx'
file_path2 = f'F:/日报/日报/new商户订单统计表截至{before_date}.xlsx'
password = '20240101'

## 日报数据

In [216]:
def Open_Excel2(df_list, path, password, col_list, key_list, key):
    '''
    对日报数据进行自动更新
    :param df: 日报各个渠道的df
    :param path: 日报的excel文件路径
    :param password: excel文件的密码
    :param sheet_name: excel的sheet页名称
    :param col: 列的位置，如A列，B列等
    :param col_len: 每个模块之间的间隔宽度
    :param key: 判断是否有进行合计
    :return:
    '''
    # 定义文件路径和密码
    file_path = path
    password = password
    # 使用 xlwings 打开加密的 Excel 文件
    app = xw.App(visible=False)
    # 获取今天是星期几
    week = pd.to_datetime(datetime.now().strftime('%Y-%m-%d')).day_name()
    # 初始化数据列长
    col_len = 1
    try:
        wb = xw.Book(file_path, password=password)
        # 获取工作表
        sheet_names = wb.sheet_names
        num = 1 if key=='日报' else 0
        for idx, df_lis in enumerate(df_list):
            sheet = wb.sheets[sheet_names[idx+num]]
            # print(sheet_names[idx])
            col = col_list[idx]
            key = key_list[idx]
            for idx1, df in enumerate(df_lis):
                # 如果df没有数据则不执行操作
                # print(len(df))
                if len(df)!=0:
                    # print(len(df))
                    # 获取该工作表的最后一行，并插入一行
                    # col_len:判断是否是第一列第一个需要插入的数据；
                    # key：值等于1用来判断是否是免人审数据，免审订单转化，押金类型这三个数据页在最后一行的前三行插入； 值不等于1则在最后一行插入并且起始行数最后减4
                    # 获取最后一行的日期数据进行判断
                    last_row = sheet.range(col + str(sheet.cells.last_cell.row)).end('up').row
                    # 改进代码（查找特定列中的最后一个非空单元格）
                    # last_row = sheet.range(f'{col}{sheet.cells.last_cell.row}').end('up').row
                    # print(last_row)
                    # 更改
                    last_date = sheet.range(f'{col}{last_row}').value if key==0 else sheet.range(f'{col}{last_row - 4}').value
                    while last_date is None and last_row > 1: # 更改{
                        last_row -= 1
                        last_date = sheet.range(f'{col}{last_row}').value # 更改}
                    # 获取当前列的长度
                    col_len = sheet.range(f'{col}{1}').end('down').row
                    print(f"工作表: {sheet.name},current_key: {key}, col: {col}, last_row: {last_row}, last_date: {last_date}!")
                    # 获取前一天的日期
                    before_date = pd.to_datetime((datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d'))
                    if col_len == 1:
                        if before_date != last_date:
                            # print(f'before_date: {before_date},last_date:{last_date}')
                            diff = (before_date - last_date).days
                            if key==0:
                                # 判断最后一行日期是不是前一天的日期，如不是则差几天就插入几行，如是则不插入新的行
                                sheet.api.Rows(last_row).Insert() if diff == 1 else sheet.api.Rows(f'{last_row}:{last_row + diff - 1}').Insert()
                            else:
                                sheet.api.Rows(last_row - 3).Insert() if diff == 1 else sheet.api.Rows(f'{last_row - diff + 1}:{last_row}').Insert()

                    # 重新获取最后一行
                    last_row = sheet.used_range.last_cell.row
                    # 将 DataFrame 写入固定的位置
                    # 对应Excel中的行和列，起始行数
                    start_row = last_row - len(df) - 1 if key==0 else last_row - len(df) - 3
                    # print(start_row)
                    # 打印关键变量，便于调试
                    print(f"写入时，工作表: {sheet.name}, last_row: {last_row}, len(df): {len(df)}, current_key: {key}, start_row: {start_row}")
                    # 定位写入区域左上角单元格（行, 列）
                    target_range = sheet.range((start_row, col_len))
                    # 一次性写入整个DataFrame数据
                    target_range.options(index=False, header=False).value = df
                    # 获取上一个df的宽度并相加
                    col_len += 1 + len(df.columns)
            # 新的sheet页，数据列长重新赋值
            col_len = 1
        
        # 保存并重新加密文件
        # wb.save(password=password)
        # 确保关闭工作簿和应用程序实例
        wb.close()
        app.quit()
    finally:
        # 确保关闭工作簿和应用程序实例
        wb.close()
        app.quit()


## 创建新的sheet页

In [217]:



# # 自定义列名转数字的函数（替代xlwings.utils.col_name_to_num）
# def col_name_to_num(col_name):
#     """
#     将Excel列名（如'A', 'B', 'AA'）转换为数字（1, 2, 27）
#     """
#     num = 0
#     for c in col_name:
#         num = num * 26 + (ord(c.upper()) - ord('A') + 1)
#     return num

# def write_with_index_and_header(df_list, file_path, password, new_sheet_name, 
#                                start_col='A', start_row=1):
#     '''
#     创建新sheet页，同时写入行索引和列索引（表头），兼容所有xlwings版本
#     :param df_list: 待写入的DataFrame列表
#     :param file_path: Excel文件路径
#     :param password: Excel密码
#     :param new_sheet_name: 新sheet页名称
#     :param start_col: 起始列（默认'A'）
#     :param start_row: 起始行（默认1）
#     '''
#     app = xw.App(visible=False, add_book=False)
#     try:
#         # 打开加密文件
#         wb = xw.Book(file_path, password=password)
        
#         # 删除同名sheet（若存在）
#         for sheet in wb.sheets:
#             if sheet.name == new_sheet_name:
#                 sheet.delete()
        
#         # 创建新sheet
#         new_sheet = wb.sheets.add(name=new_sheet_name)
        
#         # 使用自定义函数将列名（如'A'）转换为数字（如1）
#         current_col = col_name_to_num(start_col)  # 替换原xlwings.utils的方法
        
#         for df in df_list:
#             if len(df) == 0:
#                 continue  # 跳过空数据
            
#             # 确定写入区域左上角（行=start_row，列=current_col）
#             target_range = new_sheet.range((start_row, current_col))
            
#             # 写入数据（保留行索引和列索引）
#             target_range.options(
#                 index=False,    # 不保留行索引
#                 header=True,   # 保留列索引（表头）
#                 chunksize=1000 # 大数据量优化
#             ).value = df
            
#             # 计算下一个数据的起始列（当前数据总列数 + 2列间隔）
#             total_cols = 1 + df.shape[1]  # 1（行索引） + 数据列数
#             current_col += total_cols + 2  # 加2列间隔
        
#         wb.save(password=password)
#         print(f"已成功写入新sheet《{new_sheet_name}》，包含行索引和列索引（表头）")
    
#     except Exception as e:
#         print(f"错误：{str(e)}")
#     finally:
#         wb.close()
#         app.quit()
        
# # 日报数据 - 在新sheet页写入
# # write_with_index_and_header([df_qudao_group[:-1]], file_path1, password, "拒收")

# # # 商家数据 - 在新sheet页写入
# # write_to_new_sheet(sj_df_list[0], file_path2, password, "商家数据汇总")

## 加入了写入空表的逻辑

In [218]:
def Open_Excel(df_list, path, password, col_list, key_list, key):
    '''
    对日报数据进行自动更新（增强版，处理空工作表）
    :param df_list: 日报各个渠道的df列表
    :param path: 日报的excel文件路径
    :param password: excel文件的密码
    :param col_list: 列的位置，如A列，B列等
    :param key_list: 每个sheet页的处理方式标志
    :param key: 判断是否有进行合计
    :return:
    '''
    # 定义文件路径和密码
    file_path = path
    password = password
    # 使用 xlwings 打开加密的 Excel 文件
    app = xw.App(visible=False)
    # 获取今天是星期几
    week = pd.to_datetime(datetime.now().strftime('%Y-%m-%d')).day_name()
    # 初始化数据列长
    col_len = 1
    try:
        wb = xw.Book(file_path, password=password)
        # 获取工作表
        sheet_names = wb.sheet_names
        num = 1 if key == '日报' else 0
        
        for idx, df_lis in enumerate(df_list):
            # 确保索引不越界
            if idx + num >= len(sheet_names):
                print(f"警告: 工作表索引 {idx+num} 超出范围，跳过")
                continue
                
            sheet = wb.sheets[sheet_names[idx+num]]
            col = col_list[idx]
            current_key = key_list[idx]  # 避免与函数参数key冲突
            
            # 检查工作表是否为空
            is_empty_sheet = sheet.used_range.last_cell.row == 1 and sheet.used_range.last_cell.column == 1
            
            for df in df_lis:
                if len(df) == 0:
                    continue
                
                if is_empty_sheet:
                    # 空工作表处理逻辑：直接写入数据，保留原格式
                    print(f"提示: 工作表 '{sheet.name}' 为空，直接写入数据")
                    start_row = 1  # 从第一行开始写入
                    print(f"工作表: {sheet.name}, len(df): {len(df)}, current_key: {current_key}, start_row: {start_row}")
                    # 写入数据（保留索引和表头）
                    target_range = sheet.range((start_row, col_len))
                    # 一次性写入整个DataFrame数据
                    target_range.options(index=False, header=True).value = df
                    
                    # 更新列位置
                    col_len += 1 + len(df.columns)
                    continue
                # 非空工作表的原有逻辑
                # 获取该工作表的最后一行，并插入一行
                    # col_len:判断是否是第一列第一个需要插入的数据；
                    # key：值等于1用来判断是否是免人审数据，免审订单转化，押金类型这三个数据页在最后一行的前三行插入； 值不等于1则在最后一行插入并且起始行数最后减4
                    # 获取最后一行的日期数据进行判断
                
                # 获取最后一行的日期数据进行判断
                last_row = sheet.range(col + str(sheet.cells.last_cell.row)).end('up').row
                last_date = sheet.range(f'{col}{last_row}').value if current_key==0 else sheet.range(f'{col}{last_row - 4}').value
                # 更改
                while last_date is None and sheet.name in ['拒收', '星晟数码', '蘑菇时间']: # 更改{
                    last_row -= 1                    
                    last_date = sheet.range(f'{col}{last_row}').value # 更改}
                print(f"工作表: {sheet.name},current_key: {current_key}, col:{col},last_row: {last_row}, last_date: {last_date}!")
                # if last_date is None:
                #     # 日期为空：插入一行并使用前一天作为默认日期 姑且看看
                #     if current_key == 0:
                # 非空工作表的原有逻辑
                # 获取最后一行
                # last_cell = sheet.range(col + str(sheet.cells.last_cell.row)).end('up')
                # last_row = last_cell.row无用
                
                # # 获取最后一行的日期
                # last_date_value = last_cell.value
                
                # # 处理日期值为空或无效的情况
                # if last_date_value is None:
                #     print(f"警告: 工作表 '{sheet.name}' 的日期列 {col} 为空，将插入新行")
                #     last_date = None
                # else:
                #     try:
                #         # 尝试将Excel日期值转换为pandas Timestamp
                #         if isinstance(last_date_value, str):
                #             # 处理字符串格式的日期
                #             last_date = pd.to_datetime(last_date_value).normalize()
                #         elif isinstance(last_date_value, (datetime, pd.Timestamp)):
                #             # 处理datetime或Timestamp对象
                #             last_date = pd.Timestamp(last_date_value).normalize()
                #         else:
                #             # 处理Excel数值格式的日期（浮点数）
                #             last_date = xw.utils.xl_to_datetime(last_date_value).normalize()
                #     except Exception as e:
                #         print(f"警告: 无法解析工作表 '{sheet.name}' 的日期值 '{last_date_value}': {str(e)}")
                #         last_date = None ！！！！！
                
                # 获取前一天的日期
                before_date = pd.to_datetime((datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d'))
                
                # 处理日期为空或无效的情况
                if col_len == 1:
                    if last_date is None:
                        # 日期为空：插入一行并使用前一天作为默认日期
                        if current_key == 0:
                            sheet.api.Rows(last_row).Insert()
                        
                        else:
                            sheet.api.Rows(last_row - 3).Insert()
                        # 写入默认日期
                        sheet.range(f"{col}{last_row}").value = before_date.strftime('%Y-%m-%d')
                        last_date = before_date  # 更新last_date避免后续错误
                    # 判断最后一行日期是不是前一天的日期，如不是则差几天就插入几行，如是则不插入新的行
                    elif before_date != last_date:
                        # 日期不匹配：计算差值并插入相应行数
                        diff = (before_date - last_date).days
                        if diff > 0:  # 确保差值为正数
                            if current_key == 0:
                                sheet.api.Rows(last_row).Insert() if diff == 1 else sheet.api.Rows(f'{last_row}:{last_row + diff - 1}').Insert()
                            else:
                                sheet.api.Rows(last_row - 3).Insert() if diff == 1 else sheet.api.Rows(f'{last_row - diff + 1}:{last_row}').Insert()
                
                # 重新获取最后一行
                last_row = sheet.used_range.last_cell.row
                
                # 计算起始行
                start_row = last_row - len(df) - 1 if current_key == 0 else last_row - len(df) - 3
                start_row = max(start_row, 3)  # 确保行号至少为3，避免0或负数
                # 打印关键变量，便于调试
                print(f"写入时，工作表: {sheet.name}, last_row: {last_row}, len(df): {len(df)}, current_key: {current_key}, start_row: {start_row}")
                # 写入数据（保留索引）
                target_range = sheet.range((start_row, col_len))
                target_range.options(index=False, header=False).value = df
                
                # 更新列位置# 获取上一个df的宽度并相加
                col_len += 1 + len(df.columns)
            
            # 新的sheet页重置列位置
            col_len = 1
        
        # 保存并重新加密文件
        print("已保存并重新加密文件")
        wb.save(password=password)
        # # 确保关闭工作簿和应用程序实例
        # wb.close()
        # app.quit()
    finally:
        # 确保关闭工作簿和应用程序实例
        wb.close()
        app.quit()

## 将日报数据写入excel

In [219]:
# 日报数据
df_list = [[df_all_new[:-1], df_ss_group_new[:-1],df_dr_group_new[:-1], df_zm_group_new[:-1], df_zw_group_new[:-1], df_tc[:-1], df_dy_group_new[:-1], df_zfb_group1[:-1]],
            [df_ms_new3[:-1]], [df_ms_o_group_new[:-1]],[df_qmy_group_new[:-1], df_fmy_group_new[:-1]], [df_r_new[:-1]], [df_s2merge_all_new[:-1]], [df_jl_group_new[:-1]], 
            [df_wl_g_merge_new[:-1]], [df_jd_group_new[:-1], df_jdsc_group_new[:-1]], [df_qudao_group[:]]]# , [df_qudao_group[:-1]]
col_list = ['C', 'B', 'B', 'A', 'A', 'A', 'A', 'A', 'A', 'A']
key_list = [0, 1, 1, 1, 0, 0, 0, 0, 0, 0]

Open_Excel(df_list, file_path1, password, col_list, key_list, key='日报')
del df_list
gc.collect()
print('内存回收成功')

# # 日报数据
# df_list = [[df_all_new[:-1], df_ss_group_new[:-1], df_dr_group_new[:-1], df_zm_group_new[:-1], df_zw_group_new[:-1], df_tc[:-1], df_dy_group_new[:-1], df_zfb_group1[:-1]], 
#             [df_ms_new3[:-1]], [df_ms_o_group_new[:-1]],[df_qmy_group_new[:-1], df_fmy_group_new[:-1]], [df_r_new[:-1]], [df_s2merge_all_new[:-1]], [df_jl_group_new[:-1]], 
#             [df_wl_g_merge_new[:-1]], [df_jd_group_new[:-1], df_jdsc_group_new[:-1]]]
# col_list = ['C', 'B', 'B', 'A', 'A', 'A', 'A', 'A', 'A']
# key_list = [0, 1, 1, 1, 0, 0, 0, 0, 0]
# Open_Excel2(df_list, file_path1, password, col_list, key_list, key='日报')
# 商家数据
sj_df_list = [[cxyz_new[:-1]], [hnw_new[:-1]], [zzy_new[:-1]], [qzsm_new[:-1]], [hkhz_new[:-1], hkhz_dy_new[:-1]], [xmy_new[:-1]], [rrxz_new[:-1]], [jgwl_new[:-1]], [xzl_new[:-1]],
                [xzl_news[:-1]], [xcsm_new[:-1], xcsm_zfb_new[:-1], xcsm_jd_new[:-1]], [mgsj_New[:-1]], [yqdy_New[:-1]], [aony_New[:-1]]]
sj_col_list = ['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A']
sj_key_list = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Open_Excel(sj_df_list, file_path2, password, sj_col_list, sj_key_list, key='商家')
del sj_df_list
gc.collect()
print('内存回收成功')


工作表: 2025年,current_key: 0, col:C,last_row: 298, last_date: 2025-10-22 00:00:00!
写入时，工作表: 2025年, last_row: 301, len(df): 15, current_key: 0, start_row: 285
工作表: 2025年,current_key: 0, col:C,last_row: 299, last_date: 2025-10-23 00:00:00!
写入时，工作表: 2025年, last_row: 301, len(df): 15, current_key: 0, start_row: 285
工作表: 2025年,current_key: 0, col:C,last_row: 299, last_date: 2025-10-23 00:00:00!
写入时，工作表: 2025年, last_row: 301, len(df): 15, current_key: 0, start_row: 285
工作表: 2025年,current_key: 0, col:C,last_row: 299, last_date: 2025-10-23 00:00:00!
写入时，工作表: 2025年, last_row: 301, len(df): 15, current_key: 0, start_row: 285
工作表: 2025年,current_key: 0, col:C,last_row: 299, last_date: 2025-10-23 00:00:00!
写入时，工作表: 2025年, last_row: 301, len(df): 15, current_key: 0, start_row: 285
工作表: 2025年,current_key: 0, col:C,last_row: 299, last_date: 2025-10-23 00:00:00!
写入时，工作表: 2025年, last_row: 301, len(df): 15, current_key: 0, start_row: 285
工作表: 2025年,current_key: 0, col:C,last_row: 299, last_date: 2025-10-23 

In [220]:
# sheet_name = '2025年'
# # 自动写入日报数据 总体

# col_len = all_models.Open_Excel2(df_all_new[:-1], file_path1, password, sheet_name, 'C')
# # 搜索渠道
# col_len1 = all_models.Open_Excel2(df_ss_group_new[:-1], file_path1, password, sheet_name, 'C', col_len)
# # 单人会话
# df_dr_group_new = df_dr_group.reset_index()
# col_len2 = all_models.Open_Excel2(df_dr_group_new[:-1], file_path1, password, sheet_name, 'C', col_len1)
# # 芝麻租物
# col_len3 = all_models.Open_Excel2(df_zm_group_new[:-1], file_path1, password, sheet_name, 'C', col_len2)
# # 纯租物
# df_zw_group_new = df_zw_group.reset_index()
# col_len4 = all_models.Open_Excel2(df_zw_group_new[:-1], file_path1, password, sheet_name, 'C',  col_len3)
# # 总体剔除直播
# df_tc_new = df_tc.reset_index()
# col_len5 = all_models.Open_Excel2(df_tc_new[:-1], file_path1, password, sheet_name, 'C',  col_len4)
# # 抖音渠道
# df_dy_group_new = df_dy_group.reset_index()
# col_len6 = all_models.Open_Excel2(df_dy_group_new[:-1], file_path1, password, sheet_name, 'C', col_len5)
# #  支付宝直播
# df_zfb_group_new = df_zfb_group1.reset_index()
# col_len7 = all_models.Open_Excel2(df_zfb_group_new[:-1], file_path1, password, sheet_name, 'C',  col_len6)
# ## 免人审数据
# all_models.Open_Excel2(df_ms_new3[:-1], file_path1, password, '免人审数据', 'B',  key=1)
# ## 免审订单转化统计
# all_models.Open_Excel2(df_ms_o_group_new[:-1], file_path1, password, '免审订单转化统计', 'B',  key=1)
# # 全免押
# yj_col_len = all_models.Open_Excel2(df_qmy_group_new[:-1], file_path1, password, '押金类型', 'A',  key='押金')
# # # 非免押
# all_models.Open_Excel2(df_fmy_group_new[:-1], file_path1, password, '押金类型', 'A', col_len=yj_col_len, key='押金')
# ## 租完即送占比
# all_models.Open_Excel2(df_r_new[:-1], file_path1, password, '租完即送占比', 'A', key='租完即送')
# ## 碎屏险数据
# all_models.Open_Excel2(df_s2merge_all_new[:-1], file_path1, password, '碎屏险数据', 'A', key='碎屏险')
# ## 拒量
# all_models.Open_Excel2(df_jl_group_new[:-1], file_path1, password, '拒量', 'A', key='拒量')
# ## 挽留
# all_models.Open_Excel2(df_wl_g_merge_new[:-1], file_path1, password, '挽留', 'A', key='挽留')
# ## 京享租
# col_len = all_models.Open_Excel2(df_jd_group_new[:-1], file_path1, password, '京享租', 'A', key='京享租')
# # 京东商城
# all_models.Open_Excel2(df_jdsc_group_new[:-1], file_path1, password, '京享租', 'A', key='京享租', col_len=col_len)


## 商家数据

In [221]:
# def merchant(df, name):
#     df_max_date = df.下单日期.max()
#     if df_max_date<(now_day-timedelta(days=1)):
#         df.loc[len(df)] = np.nan
#         # return df
#         all_models.Open_Excel2(df=df, path=file_path2, password=password, sheet_name=name, col='A', key='商家')
#     else:
#         all_models.Open_Excel2(df=df, path=file_path2, password=password, sheet_name=name, col='A', key='商家')
#         # return df

In [222]:
# # 澄心优租
# merchant(cxyz_new[:-1], '澄心优租')
# # 北京海鸟窝科技有限公司
# merchant(hnw_new[:-1], '海鸟窝')
# # 租着用
# merchant(zzy_new[:-1], '租着用')
# # 趣智数码
# merchant(qzsm_new[:-1], '趣智数码')
# # 汇客好租
# merchant(hkhz_new[:-1], '汇客好租')
# # 小蚂蚁租机
# merchant(xmy_new[:-1], '小蚂蚁租机')
# # 乙辉数码
# # merchant(yhsm_new, '乙辉数码')
# # 兴鑫兴通讯
# # merchant(xxx_new, '兴鑫兴通讯')

# # 南京聚格网络科技
# merchant(jgwl_new[:-1], '南京聚格网络科技')
# # 呱子笔记本电脑
# # merchant(gz_new[:-1], '呱子笔记本电脑')
# # 人人享租
# merchant(rrxz_new[:-1], '人人享租')
# # 崇胜数码
# # merchant(cssm_new[:-1], '崇胜数码')
# # 喜卓灵租机
# merchant(xzl_new[:-1], '喜卓灵租机')